In [ ]:
import scipy.stats as stats
import scipy.special as special
import numpy as np
import scipy.optimize as opt
import scipy.stats as st
import scipy.stats
import numpy as np
from scipy import stats
from scipy.stats import pearsonr, spearmanr, iqr, median_abs_deviation, beta, norm, nct, t, f, ncf, chi2,  median_test, hmean, wilcoxon, hypergeom, rankdata, mannwhitneyu, binom, fisher_exact, barnard_exact, chi2_contingency
from scipy.special import hyp2f1
from numpy import around, array2string
import numpy as np
import numpy as geek
import math
import random
import itertools
import statistics
import researchpy as rp
import pingouin as pg
from pingouin import cochran
import pandas as pd
import itertools as it
from collections import Counter
import scipy.stats as stats
import scipy.optimize as optimize
from scipy.optimize import newton, root_scalar
from scipy.stats.contingency import odds_ratio, relative_risk
import rpy2.robjects as robjects
from statsmodels.stats.contingency_tables import mcnemar, cochrans_q
from statsmodels.stats.proportion import proportion_confint, confint_proportions_2indep
from itertools import product



#Function for counting negative and positive ranks
def SumNegative(x):
    sum = 0
    for i in x:
        if i < 0:
            sum += i
    return sum

def SumPositive(x):
    sum = 0
    for i in x:
        if i > 0:
            sum += i
    return sum

def Wide_to_Long_2_coloumns(x,y):
    sample_size_1 = np.empty(len(x), dtype = int)
    sample_size_2 = np.empty(len(y), dtype = int)
    sample_size_1.fill(1)
    sample_size_2.fill(0)
    y = np.append(x,y)
    x = np.append(sample_size_1,sample_size_2)

    return x,y


# This is a function that converts Z_scores into pvalues
def calculate_p_value_from_z_score(score):
    p_value = st.t.sf((abs(score)), 100000) * 2
    return min(float(p_value), 0.99999)

# This is a function that converts t_scores into pvalues
def calculate_p_value_from_t_score(t_score, df):
    p_value = st.t.sf((abs(t_score)), df) * 2
    return min(float(p_value), 0.99999)

# This is a function that converts chi_square values into pvalues
def calculate_p_value_from_chi_score(chi_score, df):
    p_value = st.chi2.sf((abs(chi_score)), df)
    return min(float(p_value), 0.99999)


# This is a function that calculates the Confidence Intervals of the Effect size in One Sample Z_score test (or two dependent samples)
#Note that since the effect size in the population and its standart deviation are unknown we can estimate it based on the sample. For the one sample case we will use the Hedges and Olkin 1985 Formula to estimate the standart deviation of the effect size
def calculate_central_ci_from_cohens_d_one_sample(cohens_d, sample_size, confidence_level):
    standard_error_es = np.sqrt((1 / sample_size) + ((cohens_d**2 / (2 * sample_size))))
    z_critical_value = st.norm.ppf(confidence_level + ((1 - confidence_level) / 2))
    ci_lower = cohens_d - standard_error_es * z_critical_value
    ci_upper = cohens_d + standard_error_es * z_critical_value
    return ci_lower, ci_upper, standard_error_es

def calculate_central_ci_from_cohens_d_two_samples(cohens_d, sample_size_1, sample_size_2, confidence_level):
    standard_error_es = np.sqrt(((sample_size_1 + sample_size_2)/(sample_size_1 * sample_size_2)) + ((cohens_d ** 2 / (2 * (sample_size_1 + sample_size_2)))))
    z_critical_value = st.norm.ppf(confidence_level + ((1-confidence_level)/2))
    ci_lower = cohens_d - standard_error_es * z_critical_value
    ci_upper = cohens_d + standard_error_es * z_critical_value
    return ci_lower, ci_upper, standard_error_es

def calculate_cohens_d_from_z_score(z_score, sample_size):
    return abs(z_score / np.sqrt(sample_size))

#This is the Correction factor for Hedges g
def correction_factor(df):
    return math.exp(math.lgamma(df/2) - math.log(math.sqrt(df/2)) - math.lgamma((df-1)/2))


#This is a function for the non-central pivotal CI's

def pivotal_ci_t(t_Score, df, sample_size, confidence_level):
    is_negative = False
    
    if t_Score < 0:
        is_negative = True
        t_Score = abs(t_Score)
    
    upper_limit = 1 - (1 - confidence_level) / 2
    lower_limit = (1 - confidence_level) / 2

    lower_criterion = [-t_Score, t_Score / 2, t_Score]
    upper_criterion = [t_Score, 2 * t_Score, 3 * t_Score]

    while nct.cdf(t_Score, df, lower_criterion[0]) < upper_limit:
        lower_criterion = [lower_criterion[0] - t_Score, lower_criterion[0], lower_criterion[2]]
    
    while nct.cdf(t_Score, df, upper_criterion[0]) < lower_limit:
        if nct.cdf(t_Score, df) < lower_limit:
            lower_ci = [0, nct.cdf(t_Score, df)]
            upper_criterion = [upper_criterion[0] / 4, upper_criterion[0], upper_criterion[2]]
    
    while nct.cdf(t_Score, df, upper_criterion[2]) > lower_limit:
        upper_criterion = [upper_criterion[0], upper_criterion[2], upper_criterion[2] + t_Score]

    # Finding lower limit for the ncp
    lower_ci = 0.0
    diff_lower = 1
    while diff_lower > 0.00001:
        if nct.cdf(t_Score, df, lower_criterion[1]) < upper_limit:
            lower_criterion = [lower_criterion[0], (lower_criterion[0] + lower_criterion[1]) / 2, lower_criterion[1]]
        else:
            lower_criterion = [lower_criterion[1], (lower_criterion[1] + lower_criterion[2]) / 2, lower_criterion[2]]
        diff_lower = abs(nct.cdf(t_Score, df, lower_criterion[1]) - upper_limit)
        lower_ci = lower_criterion[1] / (np.sqrt(sample_size))

    # Finding upper limit for the ncp
    upper_ci = 0.0
    diff_upper = 1
    while diff_upper > 0.00001:
        if nct.cdf(t_Score, df, upper_criterion[1]) < lower_limit:
            upper_criterion = [upper_criterion[0], (upper_criterion[0] + upper_criterion[1]) / 2, upper_criterion[1]]
        else:
            upper_criterion = [upper_criterion[1], (upper_criterion[1] + upper_criterion[2]) / 2, upper_criterion[2]]
        diff_upper = abs(nct.cdf(t_Score, df, upper_criterion[1]) - lower_limit)
        upper_ci = upper_criterion[1] / (np.sqrt(sample_size))

    if is_negative:
        return -upper_ci, -lower_ci
    else:
        return lower_ci, upper_ci
    

## These Functions are for the Robust_Effect_Size
def density(x):
    x = np.array(x)
    return x**2 * norm.pdf(x)

def pbvar(x, beta=0.2):
    w = np.abs(x - np.median(x))
    omega = np.sort(w)[int(np.floor((1 - beta) * len(x)))]
    y = (x - np.median(x)) / omega
    z = np.minimum(1, np.maximum(-1, y))
    pbvar = len(x) * omega**2 * np.sum(z**2) / len(x[np.abs(y) < 1])**2
    return pbvar

def area_under_function(f, a, b, *args, function_a=None, function_b=None, limit=10, eps=1e-5):
    if function_a is None:
        function_a = f(a, *args)
    if function_b is None:
        function_b = f(b, *args)
    midpoint = (a + b) / 2
    f_midpoint = f(midpoint, *args)
    area_trapezoidal = ((function_a + function_b) * (b - a)) / 2
    area_simpson = ((function_a + 4 * f_midpoint + function_b) * (b - a)) / 6
    if abs(area_trapezoidal - area_simpson) < eps or limit == 0:
        return area_simpson
    return area_under_function(f, a, midpoint, *args, function_a=function_a, function_b=f_midpoint, limit=limit-1, eps=eps) + area_under_function(f, midpoint, b, *args, function_a=f_midpoint, function_b=function_b, limit=limit-1, eps=eps)

def WinsorizedVariance(x, trimming_level=0.2):
    y = np.sort(x)
    n = len(x)
    ibot = int(np.floor(trimming_level * n)) + 1
    itop = n - ibot + 1
    xbot = y[ibot-1] 
    xtop = y[itop-1]
    y = np.where(y <= xbot, xbot, y)
    y = np.where(y >= xtop, xtop, y)
    winvar = np.std(y, ddof=1)**2
    return winvar

def WinsorizedCorrelation(x, y, trimming_level=0.2):
    sample_size = len(x)
    x_sorted = np.sort(x)
    y_sorted = np.sort(y)
    trimming_size = int(np.floor(trimming_level * sample_size)) + 1
    x_lower = x_sorted[trimming_size - 1]
    x_upper = x_sorted[sample_size - trimming_size]
    y_lower = y_sorted[trimming_size - 1]
    y_upper = y_sorted[sample_size - trimming_size]
    x_winzsorized = np.clip(x, x_lower, x_upper)
    y_winzsorized = np.clip(y, y_lower, y_upper)
    winsorized_correlation = np.corrcoef(x_winzsorized, y_winzsorized)[0, 1]
    winsorized_covariance = np.cov(x_winzsorized, y_winzsorized)[0, 1]
    test_statistic = winsorized_correlation * np.sqrt((sample_size - 2) / (1 - winsorized_correlation**2))
    Number_of_trimmed_values = int(np.floor(trimming_level * sample_size))
    p_value = 2 * (1 - t.cdf(np.abs(test_statistic), sample_size - 2*Number_of_trimmed_values - 2))
    return {'cor': winsorized_correlation, 'cov': winsorized_covariance, 'p.value': p_value, 'n': sample_size, 'test_statistic': test_statistic}


#Yuens_Test Tyuen statistic with the relvant effect size - note that for the independent variable a bootstrapping is used (with 100 repetitions) if n1 and n2 are not equal which might yield slightly different results
#This function caculates the Yuen Statistics for the Trimmed Means
def Yuens_Test_Trimmed_Means_Dependent(x, y, trimming_level=0.2, confidence_level = 0.95):
    sample_size = len(x)
    non_winsorized_sample_size = len(x) - 2 * np.floor(trimming_level * len(x))
    trimmed_mean_1 = stats.trim_mean(x, trimming_level)
    trimmed_mean_2 = stats.trim_mean(y, trimming_level)
    sort_values = np.concatenate((x, y))
    Variance_Trimmed_Means = (np.std(np.array([trimmed_mean_1, trimmed_mean_2]), ddof=1))**2
    correction = area_under_function(density, norm.ppf(trimming_level), norm.ppf(1 - trimming_level)) + 2 * (norm.ppf(trimming_level)**2) * trimming_level
    Winzorized_Variance = WinsorizedVariance(sort_values, trimming_level = trimming_level)
    Explained_variance = Variance_Trimmed_Means/(Winzorized_Variance/correction)
    Robust_Explonatory_Effect_Size = np.sqrt(Explained_variance)
    if (Robust_Explonatory_Effect_Size > 1):
        x0 = np.concatenate((np.ones(len(x)), np.ones(len(y)) * 2))
        y0 = np.concatenate((x, y))
        Robust_Explonatory_Effect_Size = np.sqrt(WinsorizedCorrelation(x0, y0, trimming_level = trimming_level)['cor']**2)
    winsorized_variance_1 = WinsorizedVariance(x, trimming_level) 
    winsorized_variance_2 = WinsorizedVariance(y, trimming_level)
    winsorized_correlation = WinsorizedCorrelation(x, y, trimming_level)['cor']
    winsorized_correlation_pvalue = WinsorizedCorrelation(x, y, trimming_level)['p.value']
    q1 = (len(x) - 1) * winsorized_variance_1
    q2 = (len(y) - 1) * winsorized_variance_2
    q3 = (len(x) - 1) * WinsorizedCorrelation(x, y, trimming_level)['cov']
    df = non_winsorized_sample_size - 1
    Yuen_Standrd_Error = np.sqrt((q1 + q2 - 2 * q3) / (non_winsorized_sample_size * (non_winsorized_sample_size - 1)))    
    difference_trimmed_means = trimmed_mean_1 - trimmed_mean_2
    Yuen_Statistic = difference_trimmed_means / Yuen_Standrd_Error
    Yuen_p_value = 2 * (1 - t.cdf(np.abs(Yuen_Statistic), df))
    return np.array([trimmed_mean_1, trimmed_mean_2, winsorized_variance_1, winsorized_variance_2, sample_size, Yuen_Statistic, df, Yuen_Standrd_Error, Yuen_p_value, \
           difference_trimmed_means, winsorized_correlation, winsorized_correlation_pvalue, Explained_variance, Robust_Explonatory_Effect_Size])

#Build CI for the yuens statistics using the Akp 2005 method with using the yuens and its df's and multiplying by the factor 

import numpy as np
from scipy import stats
from scipy.stats import t, norm

def Yuens_Test_Trimmed_Means_Independent(x, y, trimming_level=0.2, confidence_level=0.95, PB=False):
    sample_size_1 = len(x)
    sample_size_2 = len(y)
    h1 = sample_size_1 - 2 * np.floor(trimming_level * sample_size_1)
    h2 = sample_size_2 - 2 * np.floor(trimming_level * sample_size_2)
    trimmed_mean_1 = stats.trim_mean(x, trimming_level)
    trimmed_mean_2 = stats.trim_mean(y, trimming_level)
    difference_trimmed_means = trimmed_mean_1 - trimmed_mean_2
    winsorized_variance_1 = WinsorizedVariance(x, trimming_level)
    winsorized_variance_2 = WinsorizedVariance(y, trimming_level)
    correction = area_under_function(density, norm.ppf(trimming_level), norm.ppf(1 - trimming_level)) + 2 * (norm.ppf(trimming_level)**2) * trimming_level
    q1 = (sample_size_1 - 1) * winsorized_variance_1 / (h1 * (h1 - 1))
    q2 = (sample_size_2 - 1) * winsorized_variance_2 / (h2 * (h2 - 1))
    df = (q1 + q2)**2 / ((q1**2 / (h1 - 1)) + (q2**2 / (h2 - 1)))
    Yuen_Standard_Error = np.sqrt(q1 + q2)
    Yuen_Statistic = np.abs(difference_trimmed_means / Yuen_Standard_Error)
    Yuen_p_value = 2 * (1 - t.cdf(abs(Yuen_Statistic), df))
    sort_values = np.concatenate((x, y))
    Variance_Trimmed_Means = np.var(np.concatenate(([trimmed_mean_1], [trimmed_mean_2])))
    Winzorized_Variance = WinsorizedVariance(sort_values, trimming_level=trimming_level) / correction if not PB else pbvar(sort_values) / 1.06
    
    # Check if winsorized variance is equal to 1 and set explanatory robust effect size and explained variance to 0
    if Winzorized_Variance == 0:
        Explained_Variance = 0
        Robust_Explonatory_Effect_Size = 0
    else:
        Explained_Variance = Variance_Trimmed_Means / Winzorized_Variance
        Robust_Explonatory_Effect_Size = np.sqrt(Explained_Variance)

    standardizer_Robust_Cohens_d = np.sqrt(((sample_size_1 - 1) * winsorized_variance_1 + (sample_size_1 - 2) * winsorized_variance_2) / (sample_size_1 + sample_size_2 - 2))
    Robust_Cohens_d = np.sqrt(correction) * difference_trimmed_means / standardizer_Robust_Cohens_d
    Robust_Cohens_d_corrected_1 = np.sqrt(correction) * difference_trimmed_means / np.sqrt(winsorized_variance_1)
    Robust_Cohens_d_corrected_2 = np.sqrt(correction) * difference_trimmed_means / np.sqrt(winsorized_variance_2)
    
    if Explained_Variance > 1:
        x0 = np.concatenate((np.repeat(1, sample_size_1), np.repeat(2, sample_size_2)))
        y0 = np.concatenate((x, y))
        Explained_Variance = WinsorizedCorrelation(x0, y0, trimming_level=trimming_level)['cor']**2
        Robust_Explonatory_Effect_Size = np.sqrt(Explained_Variance)
    
    if sample_size_1 != sample_size_2:
        N = min(sample_size_1, sample_size_2)
        vals = np.zeros(100)
        for i in range(100):
            sample_x = np.random.choice(x, size=N, replace=False)
            sample_y = np.random.choice(y, size=N, replace=False) 
            m1_sample = stats.trim_mean(sample_x, trimming_level)
            m2_sample = stats.trim_mean(sample_y, trimming_level)
            Variance_Trimmed_Means = np.var(np.concatenate(([m1_sample], [m2_sample])), ddof=1) if sample_y is not None else np.var([m1_sample])
            correction = area_under_function(density, norm.ppf(trimming_level), norm.ppf(1 - trimming_level)) + 2 * (norm.ppf(trimming_level)**2) * trimming_level
            Winzorized_Variance = WinsorizedVariance(np.concatenate((sample_x, sample_y)), trimming_level=trimming_level) / correction if not PB else pbvar(np.concatenate((sample_x, sample_y))) / 1.06
            vals[i] = Variance_Trimmed_Means / Winzorized_Variance
        Explained_Variance = np.median(vals)
        Robust_Explonatory_Effect_Size = np.sqrt(Explained_Variance)

    return np.array([trimmed_mean_1, trimmed_mean_2, winsorized_variance_1, winsorized_variance_2, sample_size_1, sample_size_2, Yuen_Statistic, df, Yuen_Standard_Error, Yuen_p_value, difference_trimmed_means, Explained_Variance, Robust_Explonatory_Effect_Size, Robust_Cohens_d_corrected_2, Robust_Cohens_d_corrected_1, Robust_Cohens_d, standardizer_Robust_Cohens_d, h1, h2, correction])

#Add a note as if one of the winsorized samples has no variabilty - the explanotry robust effect is equal to zero in such a case
#Also dont forget to add the robust correlation for unequal variances based on sample size of one group (seperate effect sizes)
#Add bootstrapping

def sign_test_wilcoxon_method (x,y,confidence_level=0.95, population_median=0): #this is the original method by wilcoxon that ignores ties
    if y is None:
        difference = x - population_median
    else:
        difference = x - y
    sample_size = len(x)
    sign = np.where(difference < 0, -1, 1)
    difference = difference[difference != 0]
    difference_abs = np.abs(difference)
    total_n = difference.shape[0]
    positive_n = difference[difference > 0].shape[0]
    negative_n = difference[difference < 0].shape[0]
    zero_n = difference[difference == 0].shape[0]
    ranked = scipy.stats.rankdata(difference_abs)
    sign = np.where(difference < 0, -1, 1)
    ranked_sign = (sign * ranked)
    total_sum_ranks = ranked.sum()
    positive_sum_ranks = ranked[difference > 0].sum()
    negative_sum_ranks = ranked[difference < 0].sum()
    mean_positive_ranks = positive_sum_ranks / positive_n
    mean_negative_ranks = negative_sum_ranks / negative_n
    zero_sum_ranks = ranked[difference == 0].sum()
    sign2 = np.where(difference == 0, 0, sign)
    ranked2 = sign2 * ranked
    ranked2 = np.where(difference == 0, 0, ranked2)
    var_adj_T = (ranked2 * ranked2).sum()
    Adjusted_Var = (1/4) * var_adj_T
    Unadjusted_Var = ((total_n * (total_n + 1)) * (2 * total_n + 1)) /24
    var_zero_adj_T_pos = -1 * ((zero_n * (zero_n + 1)) * (2 * zero_n + 1)) /24
    var_ties_adj = Adjusted_Var - Unadjusted_Var - var_zero_adj_T_pos
    MeanW = ((positive_n + negative_n) * (positive_n + negative_n + 1)) / 4
    RBCwilcoxon = (positive_sum_ranks - negative_sum_ranks) / (positive_sum_ranks + negative_sum_ranks) #This is the match paired rank biserial correlation (Kerby, 2014)

    #Calculate The Z score wilcox
    Z_adjusted = (positive_sum_ranks - MeanW) /  np.sqrt(Adjusted_Var)
    Z_Unadjusted = (positive_sum_ranks - MeanW) /  np.sqrt(Unadjusted_Var)
    Z_adjustedCorrected = (positive_sum_ranks - MeanW - 0.5 * np.sign(positive_sum_ranks - MeanW)) /  np.sqrt(Adjusted_Var)
    Z_Unadjusted_Corrected = (positive_sum_ranks - MeanW - 0.5 * np.sign(positive_sum_ranks - MeanW)) /  np.sqrt(Unadjusted_Var)
    p_value_adjusted = norm.cdf(Z_adjusted)*2
    p_value_unadjusted = norm.cdf(Z_Unadjusted)*2
    p_value_adjusted_corrected = norm.cdf(Z_adjustedCorrected)*2
    p_value_unadjusted_corrected = norm.cdf(Z_Unadjusted_Corrected)*2
    RBC_adjusted = Z_adjusted / np.sqrt(sample_size)
    RBC_unadjusted = Z_Unadjusted / np.sqrt(sample_size)
    RBC_unajusted_corrected = Z_adjustedCorrected / np.sqrt(sample_size)
    RBC_adjusted_corrected = Z_Unadjusted_Corrected / np.sqrt(sample_size)

    return np.array([positive_n, negative_n, total_n, positive_sum_ranks, negative_sum_ranks, total_sum_ranks, \
                     mean_positive_ranks,  mean_negative_ranks, MeanW, Adjusted_Var, Unadjusted_Var, RBCwilcoxon, \
                     Z_adjusted, Z_Unadjusted, Z_adjustedCorrected, Z_Unadjusted_Corrected, \
                     p_value_adjusted, p_value_unadjusted, p_value_adjusted_corrected, p_value_unadjusted_corrected, \
                     RBC_adjusted, RBC_unadjusted, RBC_unajusted_corrected, RBC_adjusted_corrected])

def sign_test_pratt_method (x,y, confidence_level): #this is the original method by pratt that consider ties in the data
    difference = x-y
    sample_size = len(difference)
    sign = np.where(difference < 0, -1, 1)
    difference_abs = np.abs(difference)
    total_n = difference.shape[0]
    positive_n = difference[difference > 0].shape[0]
    negative_n = difference[difference < 0].shape[0]
    zero_n = difference[difference == 0].shape[0]
    ranked = scipy.stats.rankdata(difference_abs)
    sign = np.where(difference < 0, -1, 1)
    ranked_sign = (sign * ranked)
    total_sum_ranks = ranked.sum()
    positive_sum_ranks = ranked[difference > 0].sum()
    negative_sum_ranks = ranked[difference < 0].sum()
    mean_positive_ranks = positive_sum_ranks / positive_n
    mean_negative_ranks = negative_sum_ranks / negative_n
    zero_sum_ranks = ranked[difference == 0].sum()
    sign2 = np.where(difference == 0, 0, sign)
    ranked2 = sign2 * ranked
    ranked2 = np.where(difference == 0, 0, ranked2)
    var_adj_T = (ranked2 * ranked2).sum()
    Adjusted_Var = (1/4) * var_adj_T
    Unadjusted_Var = ((total_n * (total_n + 1)) * (2 * total_n + 1)) /24
    var_zero_adj_T_pos = -1 * ((zero_n * (zero_n + 1)) * (2 * zero_n + 1)) /24
    var_ties_adj = Adjusted_Var - Unadjusted_Var - var_zero_adj_T_pos
    MeanW = (positive_sum_ranks+negative_sum_ranks)/2
    RBCpratt = (positive_sum_ranks - negative_sum_ranks) / (positive_sum_ranks + negative_sum_ranks + zero_sum_ranks) #This is the matched pairs rank biserial correlation


    #Calculate The Z score pratt
    Z_adjusted = (positive_sum_ranks - MeanW) /  np.sqrt(Adjusted_Var)
    Z_Unadjusted = (positive_sum_ranks - MeanW) /  np.sqrt(Unadjusted_Var)
    Z_adjusted_Corrected = (positive_sum_ranks - MeanW - 0.5 * np.sign(positive_sum_ranks - MeanW)) /  np.sqrt(Adjusted_Var)
    Z_Unadjusted_Corrected = (positive_sum_ranks - MeanW - 0.5 * np.sign(positive_sum_ranks - MeanW)) /  np.sqrt(Unadjusted_Var)
    p_value_adjusted = norm.cdf(Z_adjusted)*2
    p_value_unadjusted = norm.cdf(Z_Unadjusted)*2
    p_value_adjusted_corrected = norm.cdf(Z_adjusted_Corrected)*2
    p_value_unadjusted_corrected = norm.cdf(Z_Unadjusted_Corrected)*2
    RBC_adjusted = Z_adjusted / np.sqrt(sample_size)
    RBC_unadjusted = Z_Unadjusted / np.sqrt(sample_size)
    RBC_unajusted_corrected = Z_Unadjusted_Corrected / np.sqrt(sample_size)
    RBC_adjusted_corrected = Z_adjusted_Corrected / np.sqrt(sample_size)

    return np.array([positive_n, negative_n, total_n, positive_sum_ranks, negative_sum_ranks, total_sum_ranks, \
                     mean_positive_ranks,  mean_negative_ranks, MeanW, Adjusted_Var, Unadjusted_Var, RBCpratt, \
                     Z_adjusted, Z_Unadjusted, Z_adjusted_Corrected, Z_Unadjusted_Corrected, \
                     p_value_adjusted, p_value_unadjusted, p_value_adjusted_corrected, p_value_unadjusted_corrected, \
                     RBC_adjusted, RBC_unadjusted, RBC_unajusted_corrected, RBC_adjusted_corrected])

def Common_Language_Effect_Sizes_Dependent(x,y,confidence_level):
    difference = (x- y)
    sample_size = len(x)
    mean_difference = np.mean(difference)
    cohens_dz = mean_difference/np.std(difference, ddof = 1)
    
    #Mcgraw and Wang common language Effect size (calculate dz)
    cles = norm.cdf(cohens_dz) * 100

    #Probabilty of superiority PSdep
    superiority_counts = np.where(x > y, 1, np.where(x < y, 0, 0.5)) #This line  gives 1 for superior values, 0 to inferior values and 0.5 to ties. 
    totalp = np.sum(np.where(superiority_counts == 0.5, 0, superiority_counts))
    PSdep = totalp/sample_size
    
    #Vargha and Delany Axy, 2000 (the dependent groups version)
    VDA_xy = sum(superiority_counts)/sample_size

    #Cliffs Delta dependent version (Feng ordinal delta, 2007)
    cliffs_delta = (sum(np.where(x>y,1,0)) - sum(np.where(y>x,1,0)))/sample_size

    return np.array([PSdep, cles, VDA_xy, cliffs_delta])



#######CI's for proportions depndent 

########### These functions are for the schilon_doi CI's

###########################################################################



def PSdep_dependent_CI(x,y,confidence_level):
    sample_size = len(x)
    difference = x - y
    superiority_counts = np.where(x > y, 1, np.where(x < y, 0, 0.5)) #This line  gives 1 for superior values, 0 to inferior values and 0.5 to ties. 
    totalp = np.sum(np.where(superiority_counts == 0.5, 0, superiority_counts))

###Agrest_Cull CI's
    p = totalp/sample_size
    q = 1 - p
    n_tag = sample_size + norm.ppf(1 - confidence_level)**2
    w_tag = totalp + norm.ppf(1 - confidence_level)**2 / 2
    ptag = w_tag/n_tag
    qtag = 1 - ptag
    agresti_cull_upper = ptag + norm.ppf(1 - confidence_level) *(np.sqrt((ptag*qtag)/n_tag))
    agresti_cull_lower = ptag - norm.ppf(1 - confidence_level) *(np.sqrt((ptag*qtag)/n_tag))
    AgrestiCull_lower = agresti_cull_lower
    AgrestiCull_upper = agresti_cull_upper

    if AgrestiCull_lower < 0:
        AgrestiCull_lower = 0
    if AgrestiCull_upper > 1:
        AgrestiCull_upper = 1

    ###Wilson_CI's
    ctagsquare = norm.ppf(1 - confidence_level)**2
    ctag = norm.ppf(1 - confidence_level)
    element1 = ctag*np.sqrt(((totalp*(sample_size-totalp))/sample_size**3)+ctagsquare/(4*sample_size**2))
    lowerwilson = (sample_size/(sample_size+ctagsquare)) *(p+ctagsquare/(2*sample_size) - element1)
    upperwilson = (sample_size/(sample_size+ctagsquare)) *(p+ctagsquare/(2*sample_size) + element1)
    
    if lowerwilson < 0:
        lowerwilson = 0
    if upperwilson > 1:
        upperwilson = 1

    ###Wald_CI's
    term1 = ctag * np.sqrt(p*q)/np.sqrt(sample_size)
    lowerwald = p - term1
    upperwald = p + term1
    
    if lowerwald < 0:
        lowerwald = 0
    if upperwald > 1:
        upperwald = 1

    ###jeffreys CI's
    lowerjeffreys = beta.ppf(confidence_level, totalp+0.5, sample_size-totalp+0.5)
    upperjeffreys = beta.ppf(1-confidence_level, totalp+0.5, sample_size-totalp+0.5)

    if lowerjeffreys < 0:
        lowerjeffreys = 0
    if upperjeffreys > 1:
        upperjeffreys = 1

###clopper-pearson CI's
    lowerCP = beta.ppf(confidence_level, totalp, sample_size-totalp+1)
    upperCP= beta.ppf(1-confidence_level, totalp+1, sample_size-totalp)

    if lowerCP < 0:
        lowerCP = 0
    if upperCP > 1:
        upperCP = 1

###arcsine CI's 1 (Kulynskaya)
    ptilde = (totalp + 0.375)/(sample_size + 0.75)
    arcsinelower = math.sin(math.asin(np.sqrt(ptilde)) - 0.5*ctag/np.sqrt(sample_size))**2
    arcsineupper = math.sin(math.asin(np.sqrt(ptilde)) + 0.5*ctag/np.sqrt(sample_size))**2

    if arcsinelower < 0:
        arcsinelower = 0
    if arcsineupper > 1:
        arcsineupper = 1

    ### logit CI's
    lambdahat = math.log(totalp/(sample_size-totalp))
    term1 = sample_size/(totalp*(sample_size-totalp))
    lambdalow = lambdahat - ctag*np.sqrt(term1)
    lambdaupper = lambdahat + ctag*np.sqrt(term1)
    logitlower = math.exp(lambdalow)/(1 + math.exp(lambdalow))
    logitupper = math.exp(lambdaupper)/(1 + math.exp(lambdaupper))

    if logitupper > 1:
        logitupper = 1
    
    if totalp == 0:
        logitlower = "**Error**"
        logitupper = "Cant calculte logit CI's when p is equal to zero"
    
    
    ### Pratt CI's
    A = ((totalp + 1)/(sample_size-totalp))**2
    B = 81 * (totalp+1) * (sample_size-totalp) - 9*sample_size - 8
    C = -3 * ctag * np.sqrt(9*(totalp+1)*(sample_size - totalp) * (9*sample_size + 5 - ctagsquare) + sample_size + 1)
    D = 81 * (totalp + 1)**2 - 9 *(totalp+1)* (2+ctagsquare) + 1
    E = 1 + A * ((B+C)/D)**3
    A2 = (totalp/ (sample_size-totalp-1)) **2
    B2 = 81 * (totalp) * (sample_size-totalp-1) - 9*sample_size - 8
    C2 = 3 * ctag * np.sqrt(9*totalp*(sample_size-totalp-1) * (9*sample_size + 5 - ctagsquare) + sample_size + 1)
    D2 = 81 * totalp**2 - (9 *totalp) * (2+ctagsquare) + 1
    E2 = 1 + A2 * ((B2+C2)/D2)**3

    upperPratt = 1/E    
    lowerPratt = 1/E2

    if totalp == 1:
        lowerPratt = 1 - (1-confidence_level)**(1/sample_size)
        upperPratt = 1 - (confidence_level)**(1/sample_size)

    if totalp == 0:
        lowerpratt = 0
        upperPratt = beta.ppf(1-confidence_level, totalp+1, sample_size-totalp)

    if totalp == sample_size-1:
        lowerpratt = (confidence_level)**(1/sample_size)
        upperPratt = (1-confidence_level)**(1/sample_size)
    
    if totalp == sample_size:
        lowerpratt = (confidence_level*2)**(1/sample_size)
        upperPratt = 1

    if lowerPratt < 0:
        lowerPratt = 0
    if upperPratt > 1:
        upperPratt = 1
    

#Goodman-Kruskal Gamma function
def goodman(x, y):
    x,y =Wide_to_Long_2_coloumns(x,y)
    Rx = np.sign(np.subtract.outer(x, x))
    Ry = np.sign(np.subtract.outer(y, y))
    S1 = np.multiply(Rx, Ry)
    return np.sum(S1) / np.sum(np.abs(S1))


def CLES_Aparametric_Independent_Samples (x,y,confidence_level, bootstrap=False):
    sample_size_1 = len(x)
    sample_size_2 = len(y)
    sample_size = sample_size_1 + sample_size_2
    Number_of_comparisons_between_x_and_y = sample_size_1 * sample_size_2
    Matrix = np.subtract.outer(x, y)
    Poistive_Compariosons = Matrix > 0
    nonties_comparisons = Matrix != 0 
    Superioirty_num = np.count_nonzero(Poistive_Compariosons) ## Proportions of cases group1 is larger
    Number_Of_Ties = (Number_of_comparisons_between_x_and_y - np.count_nonzero(nonties_comparisons)) # Amount of ties in the data
    proportion_of_ties = Number_Of_Ties / Number_of_comparisons_between_x_and_y
    Inferioirtiy_num = Number_of_comparisons_between_x_and_y - Superioirty_num - Number_Of_Ties # Proportions of cases group2 is larger

    # Applying the Haldane-Anscombe correction only if bootstrap is True and Inferioirtiy_num is zero
    if bootstrap and Inferioirtiy_num == 0:
        Inferioirtiy_num = 0.05

    # Applying the Haldane-Anscombe correction only if bootstrap is True and Superioirty_num is zero
    if bootstrap and Superioirty_num == 0:
        Superioirty_num = 0.05
    
    ##Calculating the effect sizes of POS = Probability_Of_Superiority
    POS_Grissom_Kim = Superioirty_num / Number_of_comparisons_between_x_and_y
    POS_Vargha_Delaney = (Superioirty_num + Number_Of_Ties *0.5) / Number_of_comparisons_between_x_and_y
    POS_Gamma_Based_Metsamuuronen = 1 - goodman(x,y) * 0.5 + 0.5

    ##Calculating the odds ratio effect sizes
    WMW_agresti = Superioirty_num/Inferioirtiy_num

    #Add anotification of the meanings - text that will describe the meaning
    WMW_Churilov = ((Superioirty_num  / Number_of_comparisons_between_x_and_y) + 0.5 * proportion_of_ties) /((Inferioirtiy_num / Number_of_comparisons_between_x_and_y) + 0.5 * proportion_of_ties)

    #Cliffs delta is a transformation of the probabilty of superiority:
    cliffs_delta = 1 - 2 * POS_Grissom_Kim

    if bootstrap: #i need to return that as an array for the bootstrap
        return POS_Grissom_Kim, POS_Vargha_Delaney, POS_Gamma_Based_Metsamuuronen, WMW_agresti, WMW_Churilov, cliffs_delta

    else: 
    
        # Central Confidence Intervals
        ##############################
        #The t_value_for the CI's
        Critical_t_value = scipy.stats.t.ppf(1 - ((1-confidence_level)/2) , sample_size-2)

        #1. Probabilty of Superiority
        #A. Ruscio CI's:
        Standard_Error_Ruscio = np.sqrt((1/ sample_size_1+ 1/sample_size_2 +1/(sample_size_1*sample_size_2))/12)

        # Grissom & Kim PS CI
        ci_lower_Grissom_Kim = POS_Grissom_Kim - (Standard_Error_Ruscio * Critical_t_value)
        ci_upper_Grissom_Kim = POS_Grissom_Kim + (Standard_Error_Ruscio * Critical_t_value)
        ci_lower_central_Grissom_Kim = max(ci_lower_Grissom_Kim, 0)
        ci_upper_central_Grissom_Kim = min(ci_upper_Grissom_Kim, 1)

        # Probability of Superiority CI (Vargha-Delaney)
        ci_lower_Vargha_Delaney = POS_Vargha_Delaney - (Standard_Error_Ruscio * Critical_t_value)
        ci_upper_Vargha_Delaney = POS_Vargha_Delaney + (Standard_Error_Ruscio * Critical_t_value)
        ci_lower_central_Vargha_Delaney = max(ci_lower_Vargha_Delaney, 0)
        ci_upper_central_Vargha_Delaney = min(ci_upper_Vargha_Delaney, 1)

        # PS Gamma Rubia CI
        ci_lower_Gamma = POS_Gamma_Based_Metsamuuronen - (Standard_Error_Ruscio * Critical_t_value)
        ci_upper_Gamma = POS_Gamma_Based_Metsamuuronen + (Standard_Error_Ruscio * Critical_t_value)
        ci_lower_central_Gamma = max(ci_lower_Gamma, 0)
        ci_upper_central_Gamma = min(ci_upper_Gamma, 1)

        #B Metsammuuronen/Rubia
        gamma = goodman(x,y)

        sample_size_1 = len(x)
        sample_size_2 = len(y)
        sample_size = sample_size_1 + sample_size_2

        x_combined = np.append(x, y)
        freq = Counter(x_combined)
        frequencies = list(freq.values())
        tiescorrection = [(f ** 3 - f) for f in frequencies]
        multiplicity_factor = sum(tiescorrection)

        data_dict = {'X': x, 'Y': y}
        group_freq = {group: Counter(data_dict[group]) for group in data_dict.keys()}
        group_x_df = pd.DataFrame.from_dict(group_freq['X'], orient='index', columns=['Group1'])
        group_y_df = pd.DataFrame.from_dict(group_freq['Y'], orient='index', columns=['Group2'])
        cont_table = pd.concat([group_x_df, group_y_df], axis=1).fillna(0)

        data_long = Wide_to_Long_2_coloumns(x,y)
        data_ASE = pd.DataFrame({'Score': data_long[1], 'Group': data_long[0]})
        data_ASE['nj'] = data_ASE.groupby('Score')['Score'].transform('count')
        data_ASE['ni'] = data_ASE.groupby('Group')['Group'].transform('count')
        count_x_greater = [sum([y_val < x_val for y_val in y]) for x_val in x]
        count_y_greater = [sum([x_val < y_val for x_val in x]) for y_val in y]
        count_x_smaller = [sum([y_val > x_val for y_val in y]) for x_val in x]
        count_y_smaller = [sum([x_val > y_val for x_val in x]) for y_val in y]

        data_ASE['Concordant'] = count_x_greater + count_y_greater
        data_ASE['Disoncordant'] = count_x_smaller + count_y_smaller

        group_0_rows = data_ASE.loc[data_ASE['Group'] == 0, ['Concordant', 'Disoncordant']]
        group_0_rows[['Concordant', 'Disoncordant']] = group_0_rows[['Disoncordant', 'Concordant']].values
        data_ASE.loc[data_ASE['Group'] == 0, ['Concordant', 'Disoncordant']] = group_0_rows

        data_ASE['C-D'] = data_ASE['Concordant'] - data_ASE['Disoncordant']
        Q_concordant = sum(data_ASE['Disoncordant'])
        P_disconcordant = sum(data_ASE['Concordant'])

        ni_squared = sum(cont_table.sum(axis=1) ** 2)
        nj_squared = sum(cont_table.sum(axis=0) ** 2)
        Dx = sample_size ** 2 - ni_squared
        Dg = sample_size ** 2 - nj_squared

        data_ASE['N-nj'] = sample_size - data_ASE['nj']
        data_ASE['N-ni'] = sample_size - data_ASE['ni']
        data_ASE['ASE1formula'] = ((Dx * data_ASE['C-D']) - (data_ASE['N-nj'] * (P_disconcordant - Q_concordant))) ** 2
        data_ASE['ASE2formula'] = ((Dg * data_ASE['C-D']) - (data_ASE['N-ni'] * (P_disconcordant - Q_concordant))) ** 2
        data_ASE['C*Q - P*D'] = (data_ASE['Concordant'] * Q_concordant - data_ASE['Disoncordant'] * P_disconcordant) ** 2

        #Standart Errors of the Gamma and Delta correlations
        Standart_Error_Gamma_Metsamuuronen = ((2 / (Q_concordant + P_disconcordant))) * (np.sqrt((sum(data_ASE['C-D'] ** 2)) - ((1 / sample_size) * ((P_disconcordant - Q_concordant) ** 2))))
        Standart_Error_Delta_Metsamuuronen = ((2 / Dx)) * (np.sqrt((sum(data_ASE['C-D'] ** 2)) - ((1 / sample_size) * ((P_disconcordant - Q_concordant) ** 2))))

        #Extra - Standart Error for the H0 Hypothesis
        Standard_Error_1_Delta_H0 = (2 / Dx ** 2) * np.sqrt(sum(data_ASE['ASE1formula']))
        Standard_Error_2_Delta_H0 = ((2 / Dg)) * (np.sqrt((sum(data_ASE['C-D'] ** 2)) - ((1 / sample_size) * ((P_disconcordant - Q_concordant) ** 2))))
        Standart_Error_Gamma_H0 = (4 / (Q_concordant + P_disconcordant) ** 2) * (np.sqrt(sum(data_ASE['C*Q - P*D'])))

        # PS Gamma Metsamuuronen CI
        ci_lower_Gamma_Metsamuuronen = POS_Gamma_Based_Metsamuuronen - ((Standart_Error_Gamma_Metsamuuronen*0.5+0.5) * Critical_t_value)
        ci_upper_Gamma_Metsamuuronen = POS_Gamma_Based_Metsamuuronen + ((Standart_Error_Gamma_Metsamuuronen*0.5+0.5) * Critical_t_value)
        ci_lower_Gamma_Metsamuuronen = max(ci_lower_Gamma_Metsamuuronen, 0)
        ci_upper_Gamma_Metsamuuronen = min(ci_upper_Gamma_Metsamuuronen, 1)

        # Somers_Delta Metsamuuronen
        ci_lower_Grissom_Kim_Metsamuuronen = POS_Grissom_Kim - ((Standart_Error_Delta_Metsamuuronen*0.5+0.5) * Critical_t_value)
        ci_upper_Grissom_Kim_Metsamuuronen = POS_Grissom_Kim + ((Standart_Error_Delta_Metsamuuronen*0.5+0.5) * Critical_t_value)
        ci_lower_Grissom_Kim_Metsamuuronen = max(ci_lower_Grissom_Kim_Metsamuuronen, 0)
        ci_upper_Grissom_Kim_Metsamuuronen = min(ci_upper_Grissom_Kim_Metsamuuronen, 1)

        return POS_Grissom_Kim, POS_Vargha_Delaney, POS_Gamma_Based_Metsamuuronen, WMW_agresti, WMW_Churilov, cliffs_delta, \
                ci_lower_central_Grissom_Kim, ci_upper_central_Grissom_Kim,ci_lower_central_Vargha_Delaney, ci_upper_central_Vargha_Delaney,  \
                ci_lower_central_Gamma,ci_upper_central_Gamma, ci_lower_Gamma_Metsamuuronen,  ci_upper_Gamma_Metsamuuronen, \
                ci_lower_Grissom_Kim_Metsamuuronen, ci_upper_Grissom_Kim_Metsamuuronen


#Cohensd based effect sizes
def CLES_Cohens_based_Independent_samples(sample_mean_1, sample_mean_2, population_diff, sample_size_1, sample_size_2, sample_sd_1,sample_sd_2, df, confidence_level, bootstrap=False):
    sample_size = sample_size_1 + sample_size_2
    cohens_ds = (sample_mean_1 - sample_mean_2 - population_diff)  / (np.sqrt((((sample_size_1-1)*(sample_sd_1**2)) + ((sample_size_2-1)*(sample_sd_2**2))) / df))
    cohens_dpop = cohens_ds / np.sqrt(df/(df+2))
    t_score = cohens_ds / (np.sqrt(1/sample_size_1 + 1/sample_size_2))

    cohens_U2_dpop = (norm.cdf(abs(cohens_dpop)/2))
    cohens_U2_ds = (norm.cdf(abs(cohens_ds)/2))
    cohens_U1_dpop = (2 * abs(cohens_U2_dpop) - 1) / (cohens_U2_dpop)
    cohens_U1_ds =  (2*abs(cohens_U2_ds) - 1) / cohens_U2_ds
    cohens_U3_dpop = norm.cdf(abs(cohens_dpop))
    cohens_U3_ds = norm.cdf(abs(cohens_ds))

    probabilty_of_superiority_dpop = norm.cdf(cohens_dpop/np.sqrt(2))        
    probabilty_of_superiority_ds = norm.cdf(cohens_ds/np.sqrt(2))

    proportion_of_overlap_dpop = 2 * norm.cdf(-abs(cohens_dpop) / 2)
    proportion_of_overlap_ds = 2 * norm.cdf(-abs(cohens_ds) / 2)

    Mcgraw_Wong_CLES = norm.cdf((sample_mean_1 - sample_mean_2 - population_diff) - np.sqrt(sample_sd_1**2 + sample_sd_2**2))

    if bootstrap: #i need to return that as an array for the bootstrap
        return proportion_of_overlap_ds, proportion_of_overlap_dpop, probabilty_of_superiority_ds, probabilty_of_superiority_dpop, cohens_U3_ds, cohens_U3_dpop, cohens_U1_ds, cohens_U1_dpop, cohens_U2_ds, cohens_U2_dpop

    else: 
        # 1. Central CI's 
        Standart_Error_dpop =  np.sqrt(((sample_size_1+sample_size_2)/(sample_size_1*sample_size_2)) + ((cohens_dpop**2/(2*(sample_size_1+sample_size_2)))))
        Standart_Error_ds =  np.sqrt(((sample_size_1+sample_size_2)/(sample_size_1*sample_size_2)) + ((cohens_ds**2/(2*(sample_size_2+sample_size_1)))))
        tcrit = scipy.stats.t.ppf(1-(1-confidence_level)/2, df) #this is the critical value for calculating the confidence interval for t score
        upper_central_ci_dpop = abs(cohens_dpop + tcrit * Standart_Error_dpop)
        upper_central_ci_ds = abs(cohens_ds + tcrit * Standart_Error_ds)
        lower_central_ci_dpop = abs(cohens_dpop - tcrit * Standart_Error_dpop)
        lower_central_ci_ds = abs(cohens_ds - tcrit * Standart_Error_ds)

        #Convert the upper limits to the effect sizes based on d
        upper_central_ci_cohens_U2_dpop =  (norm.cdf(upper_central_ci_dpop/2))
        upper_central_ci_cohens_U2_ds = (norm.cdf(upper_central_ci_ds/2))
        upper_central_ci_cohens_U1_dpop = ((2 * upper_central_ci_cohens_U2_dpop - 1) / upper_central_ci_cohens_U2_dpop)
        upper_central_ci_cohens_U1_ds =   ((2 * upper_central_ci_cohens_U2_ds - 1)   / upper_central_ci_cohens_U2_ds)
        upper_central_ci_cohens_U3_dpop = norm.cdf(upper_central_ci_dpop)
        upper_central_ci_cohens_U3_ds = norm.cdf(upper_central_ci_ds)
        upper_central_ci_POS_dpop = norm.cdf(upper_central_ci_dpop/np.sqrt(2))
        upper_central_ci_POS_ds = norm.cdf(upper_central_ci_ds/np.sqrt(2))
        upper_central_ci_POO_dpop = 2 * norm.cdf(-abs(upper_central_ci_dpop) / 2)
        upper_central_ci_POO_ds = 2 * norm.cdf(-abs(upper_central_ci_ds) / 2)
        if upper_central_ci_cohens_U2_dpop > 100:
                upper_central_ci_cohens_U2_dpop = 100
        if upper_central_ci_cohens_U2_ds > 100:
                upper_central_ci_cohens_U2_ds = 100
        if upper_central_ci_cohens_U1_dpop > 100:
                upper_central_ci_cohens_U1_dpop = 100
        if upper_central_ci_cohens_U1_ds > 100:
                upper_central_ci_cohens_U1_ds = 100
        if upper_central_ci_cohens_U3_dpop > 100:
                upper_central_ci_cohens_U3_dpop = 100
        if upper_central_ci_cohens_U3_ds > 100:
                upper_central_ci_cohens_U3_ds = 100
        if upper_central_ci_POS_dpop > 1:
                upper_central_ci_POS_dpop = 1
        if upper_central_ci_POS_ds > 1:
                upper_central_ci_POS_ds = 1
        if upper_central_ci_POO_dpop > 1:
                upper_central_ci_POO_dpop = 1
        if upper_central_ci_POO_ds > 1:
                upper_central_ci_POO_ds = 1
        
        #Convert the lower limits to the effect sizes based on d
        lower_central_ci_cohens_U2_dpop = (norm.cdf(lower_central_ci_dpop/2))
        lower_central_ci_cohens_U2_ds = (norm.cdf(lower_central_ci_ds/2))
        lower_central_ci_cohens_U1_dpop = ((2 * (lower_central_ci_cohens_U2_dpop) - 1) / (lower_central_ci_cohens_U2_dpop))
        lower_central_ci_cohens_U1_ds = ((2 * (lower_central_ci_cohens_U2_ds) - 1) / (lower_central_ci_cohens_U2_ds))
        lower_central_ci_cohens_U3_dpop = norm.cdf(lower_central_ci_dpop)
        lower_central_ci_cohens_U3_ds = norm.cdf(lower_central_ci_ds)
        lower_central_ci_POS_dpop = norm.cdf(lower_central_ci_dpop/np.sqrt(2))        
        lower_central_ci_POS_ds = norm.cdf(lower_central_ci_ds/np.sqrt(2))
        lower_central_ci_POO_dpop = 2 * norm.cdf(-abs(lower_central_ci_dpop) / 2)
        lower_central_ci_POO_ds = 2 * norm.cdf(-abs(lower_central_ci_ds) / 2)
        if lower_central_ci_cohens_U2_dpop < 0:
                lower_central_ci_cohens_U2_dpop = 0
        if lower_central_ci_cohens_U2_ds < 0:
                lower_central_ci_cohens_U2_ds = 0
        if lower_central_ci_cohens_U1_dpop < 0:
                lower_central_ci_cohens_U1_dpop = 0
        if lower_central_ci_cohens_U1_ds < 0:
                lower_central_ci_cohens_U1_ds = 0
        if lower_central_ci_cohens_U3_dpop < 0:
                lower_central_ci_cohens_U3_dpop = 0
        if lower_central_ci_cohens_U3_ds < 0:
                lower_central_ci_cohens_U3_ds = 0
        if lower_central_ci_POS_dpop < 0:
                lower_central_ci_POS_dpop = 0
        if lower_central_ci_POS_ds  < 0:
                lower_central_ci_POS_ds = 0
        if lower_central_ci_POO_dpop < 0:
                lower_central_ci_POO_dpop = 0
        if lower_central_ci_POO_ds < 0:
                lower_central_ci_POO_ds = 0

        #2 Pivotal_CI's
        lower_pivotal_ci_ds, upper_pivotal_ci_ds = pivotal_ci_t(abs(t_score), df, sample_size, confidence_level)
        lower_pivotal_ci_ds = (lower_pivotal_ci_ds * np.sqrt(sample_size)) / np.sqrt(sample_size/4)
        upper_pivotal_ci_ds = (upper_pivotal_ci_ds * np.sqrt(sample_size)) / np.sqrt(sample_size/4)

        lower_pivotal_ci_dpop = lower_pivotal_ci_ds / np.sqrt(df/(df+2)) 
        upper_pivotal_ci_dpop = upper_pivotal_ci_ds / np.sqrt(df/(df+2)) 
        
        upper_pivotal_ci_cohens_U2_ds = (norm.cdf(upper_pivotal_ci_ds/2))
        upper_pivotal_ci_cohens_U1_ds = ((2 * (upper_pivotal_ci_cohens_U2_ds) - 1) / (upper_pivotal_ci_cohens_U2_ds))
        upper_pivotal_ci_cohens_U3_ds = norm.cdf(upper_pivotal_ci_ds)
        upper_pivotal_ci_POS_ds = norm.cdf(upper_pivotal_ci_ds/np.sqrt(2))
        upper_pivotal_ci_POO_ds = 2 * norm.cdf(-abs(upper_pivotal_ci_ds) / 2)
        if upper_pivotal_ci_cohens_U2_ds > 100:
                upper_pivotal_ci_cohens_U2_ds = 100
        if upper_pivotal_ci_cohens_U1_ds > 100:
                upper_pivotal_ci_cohens_U1_ds = 100
        if upper_pivotal_ci_cohens_U3_ds > 100:
                upper_pivotal_ci_cohens_U3_ds = 100
        if upper_pivotal_ci_POS_ds > 1:
                upper_pivotal_ci_POS_ds = 1
        if upper_pivotal_ci_POO_ds > 1:
                upper_pivotal_ci_POO_ds = 1
        
        #Convert the lower limits to the effect sizes based on d
        lower_pivotal_ci_cohens_U2_ds = (norm.cdf(lower_pivotal_ci_ds/2))
        lower_pivotal_ci_cohens_U1_ds = ((2 * (lower_pivotal_ci_ds) - 1) / (lower_pivotal_ci_cohens_U2_ds))
        lower_pivotal_ci_cohens_U3_ds = norm.cdf(lower_pivotal_ci_ds)
        lower_pivotal_ci_POS_ds = norm.cdf(lower_pivotal_ci_ds/np.sqrt(2))        
        lower_pivotal_ci_POO_ds = 2 * norm.cdf(-abs(lower_pivotal_ci_ds) / 2)
        if lower_pivotal_ci_cohens_U2_ds < 0:
                lower_pivotal_ci_cohens_U2_ds = 0
        if lower_pivotal_ci_cohens_U1_ds < 0:
                lower_pivotal_ci_cohens_U1_ds = 0
        if lower_pivotal_ci_cohens_U3_ds < 0:
                lower_pivotal_ci_cohens_U3_ds = 0
        if lower_pivotal_ci_POS_ds < 0:
                lower_pivotal_ci_POS_ds = 0
        if lower_pivotal_ci_POO_ds < 0:
                lower_pivotal_ci_POO_ds = 0


        upper_pivotal_ci_cohens_U2_dpop = (norm.cdf(upper_pivotal_ci_dpop/2))
        upper_pivotal_ci_cohens_U1_dpop = ((2 * (upper_pivotal_ci_cohens_U2_dpop) - 1) / (upper_pivotal_ci_cohens_U2_dpop))
        upper_pivotal_ci_cohens_U3_dpop = norm.cdf(upper_pivotal_ci_dpop)
        upper_pivotal_ci_POS_dpop = norm.cdf(upper_pivotal_ci_dpop/np.sqrt(2))
        upper_pivotal_ci_POO_dpop = 2 * norm.cdf(-abs(upper_pivotal_ci_dpop) / 2)

        if upper_pivotal_ci_cohens_U2_dpop > 100:
            upper_pivotal_ci_cohens_U2_dpop = 100
        if upper_pivotal_ci_cohens_U1_dpop > 100:
            upper_pivotal_ci_cohens_U1_dpop = 100
        if upper_pivotal_ci_cohens_U3_dpop > 100:
            upper_pivotal_ci_cohens_U3_dpop = 100
        if upper_pivotal_ci_POS_dpop > 1:
            upper_pivotal_ci_POS_dpop = 1
        if upper_pivotal_ci_POO_dpop > 1:
            upper_pivotal_ci_POO_dpop = 1

        # Convert the lower limits to the effect sizes based on dpop
        lower_pivotal_ci_cohens_U2_dpop = (norm.cdf(lower_pivotal_ci_dpop/2))
        lower_pivotal_ci_cohens_U1_dpop = ((2 * (lower_pivotal_ci_cohens_U2_dpop) - 1) / (lower_pivotal_ci_cohens_U2_dpop))
        lower_pivotal_ci_cohens_U3_dpop = norm.cdf(lower_pivotal_ci_dpop)
        lower_pivotal_ci_POS_dpop = norm.cdf(lower_pivotal_ci_dpop/np.sqrt(2))        
        lower_pivotal_ci_POO_dpop = 2 * norm.cdf(-abs(lower_pivotal_ci_dpop) / 2)

        if lower_pivotal_ci_cohens_U2_dpop < 0:
            lower_pivotal_ci_cohens_U2_dpop = 0
        if lower_pivotal_ci_cohens_U1_dpop < 0:
            lower_pivotal_ci_cohens_U1_dpop = 0
        if lower_pivotal_ci_cohens_U3_dpop < 0:
            lower_pivotal_ci_cohens_U3_dpop = 0
        if lower_pivotal_ci_POS_dpop < 0:
            lower_pivotal_ci_POS_dpop = 0
        if lower_pivotal_ci_POO_dpop < 0:
            lower_pivotal_ci_POO_dpop = 0

        return upper_pivotal_ci_ds, lower_pivotal_ci_ds, t_score, cohens_ds, cohens_dpop, \
               Mcgraw_Wong_CLES, cohens_U1_ds, cohens_U2_ds, cohens_U3_ds, probabilty_of_superiority_ds, probabilty_of_superiority_dpop, proportion_of_overlap_ds, \
               cohens_U1_dpop, cohens_U2_dpop, cohens_U3_dpop, probabilty_of_superiority_dpop, proportion_of_overlap_dpop, \
               \
               lower_central_ci_cohens_U2_ds, upper_central_ci_cohens_U2_ds, lower_central_ci_cohens_U1_ds, upper_pivotal_ci_cohens_U1_ds, \
               lower_central_ci_cohens_U3_ds, upper_central_ci_cohens_U3_ds, lower_central_ci_POS_ds, upper_central_ci_POS_ds, \
               lower_central_ci_POO_ds, upper_central_ci_POO_ds, lower_central_ci_cohens_U2_dpop, upper_central_ci_cohens_U2_dpop, \
               lower_central_ci_cohens_U1_dpop, upper_central_ci_cohens_U1_dpop, lower_central_ci_cohens_U3_dpop, upper_central_ci_cohens_U1_ds, \
               lower_central_ci_POS_dpop, upper_central_ci_POS_dpop, lower_central_ci_POO_dpop, upper_central_ci_POO_dpop,\
               \
               lower_pivotal_ci_cohens_U2_ds, upper_pivotal_ci_cohens_U2_ds, lower_pivotal_ci_cohens_U1_ds, upper_pivotal_ci_cohens_U1_ds, \
               lower_pivotal_ci_cohens_U3_ds, upper_pivotal_ci_cohens_U3_ds, lower_pivotal_ci_POS_ds, upper_pivotal_ci_POS_ds, \
               lower_pivotal_ci_POO_ds, upper_pivotal_ci_POO_ds, lower_pivotal_ci_cohens_U2_dpop, upper_pivotal_ci_cohens_U2_dpop, \
               lower_pivotal_ci_cohens_U1_dpop, upper_pivotal_ci_cohens_U1_dpop, lower_pivotal_ci_cohens_U3_dpop, upper_pivotal_ci_cohens_U1_ds, \
               lower_pivotal_ci_POS_dpop, upper_pivotal_ci_POS_dpop, lower_pivotal_ci_POO_dpop, upper_pivotal_ci_POO_dpop
         

def Other_CLES_dependent(x, y, confidence_level):
    num1 = len(x)
    num2 = len(y)

    # Calculate bandwidth h
    h_1 = (1.2 * (np.percentile(x, 75) - np.percentile(x, 25))) / (num1 ** (1 / 5))
    h_1 = max(h_1, 0.05)

    h_2 = (1.2 * (np.percentile(y, 75) - np.percentile(y, 25))) / (num2 ** (1 / 5))
    h_2 = max(h_2, 0.05)

    # Calculate F and eta for Wilcox & Musaka Q
    eta_1 = 0
    eta_2 = 0

    for test_x in x:
        count1 = np.sum(x <= (test_x + h_1))
        count2 = np.sum(x < (test_x - h_1))
        f_x1 = (count1 - count2) / (2 * num1 * h_1)

        count1 = np.sum(y <= (test_x + h_2))
        count2 = np.sum(y < (test_x - h_2))
        f_x2 = (count1 - count2) / (2 * num2 * h_2)

        if f_x1 > f_x2:
            eta_1 += 1

    for test_x in y:
        count1 = np.sum(x <= (test_x + h_1))
        count2 = np.sum(x < (test_x - h_1))
        f_x1 = (count1 - count2) / (2 * num1 * h_1)

        count1 = np.sum(y <= (test_x + h_2))
        count2 = np.sum(y < (test_x - h_2))
        f_x2 = (count1 - count2) / (2 * num2 * h_2)

        if f_x2 > f_x1:
            eta_2 += 1

    Wilcox_Q = (eta_1 + eta_2) / (num1 + num2)

    # Calculate CLES
    med_x = np.median(x)
    med_y = np.median(y)

    # Compare each value in x to the median of y
    greater_x = sum(1 for val in x if val > med_y)
    smaller_x = sum(1 for val in x if val < med_y)
    equal_x = len(x) - greater_x - smaller_x

    # Compare each value in y to the median of x
    greater_y = sum(1 for val in y if val > med_x)
    smaller_y = sum(1 for val in y if val < med_x)
    equal_y = len(y) - greater_y - smaller_y

    # Avoid division by zero by dividing it in the total cases instead
    if greater_x == 0:
        greater_x = 1

    totaln = greater_x + smaller_x + equal_x

    q1_ka = greater_x / totaln
    q1_HS = (greater_x + equal_x * 0.5) / totaln

    Kramers_Andrew_Gamma = norm.ppf(q1_ka)
    Hentschke_Stüttgen_Gamma = norm.ppf(q1_HS)

    return Wilcox_Q, Hentschke_Stüttgen_Gamma, Kramers_Andrew_Gamma

    
# Example usage:
x = np.array([1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
y = np.array([1,1,1,1,1,1,1,2,2,2,3,3,3,3,4,4,4,5,5])

def Confidence_Interval_Central_Dependent (sample_mean_1, sample_mean_2, sample_sd_1,sample_sd_2, sample_size, pearson_correlation, population_mean_diff, confidence_level):
    df1 = sample_size - 1 #degrees of freedom for within subject design
    df2 = sample_size*2 - 2 #degrees of freedom for between subject design
    convertor = (2 * (1 - pearson_correlation) / sample_size) #this one converts between effect sizes
    reversed_convertor = sample_size / (2 * (1 - pearson_correlation)) 
    correction = correction_factor(df1)    

    standardizer_dz = np.sqrt(sample_sd_1**2 + sample_sd_2**2 - 2*pearson_correlation * sample_sd_1 * sample_sd_2)
    standardizer_dav = np.sqrt((sample_sd_1**2 + sample_sd_2**2)/2)
    standardizer_drm = np.sqrt(sample_sd_1**2 + sample_sd_2**2 - 2*pearson_correlation * sample_sd_1 * sample_sd_2) / np.sqrt(2*(1-pearson_correlation))
    standardizer_ds = np.sqrt(((((sample_size-1)*sample_sd_1**2)) + ((sample_size-1)*sample_sd_2**2)) / (2*sample_size-2))
        
    cohens_dz = (((sample_mean_1-sample_mean_2) - population_mean_diff )/standardizer_dz) #This is the effect size for one sample t-test Cohen's d
    correction = correction_factor(df1)
    hedges_gz = cohens_dz*correction #This is the actual corrected effect size
    cohens_dav = abs(((sample_mean_1-sample_mean_2) - population_mean_diff )/standardizer_dav)
    hedges_gav = cohens_dav*correction
    cohens_drm = (sample_mean_1 - sample_mean_2 - population_mean_diff) / standardizer_drm
    hedges_grm = cohens_drm * correction
    cohens_ds = (((sample_mean_1-sample_mean_2) - population_mean_diff )/standardizer_ds)
    tcrit = scipy.stats.t.ppf(1-(1-confidence_level)/2, df1) #this is the critical value for calculating the confidence interval for t score

    #True Formula (Hedges 1981a):
    True_SE_dz =  np.sqrt ((df1/(df1-2)) * (convertor) *     (1 + cohens_ds**2 * reversed_convertor) - (cohens_ds**2 / (correction_factor(df1))**2))
    True_SE_dav = np.sqrt ((df2/(df2-2)) * (2/sample_size) * (1 + cohens_ds**2 * (sample_size/2))    - (cohens_ds**2 / (correction_factor(df2))**2)) 
    
    lower_central_true_ci_cohens_dz = cohens_dz - tcrit * True_SE_dz
    upper_central_true_ci_cohens_dz = cohens_dz + tcrit * True_SE_dz
    lower_central_true_ci_cohens_drm = cohens_drm - tcrit * True_SE_dz * (np.sqrt(2*(1-pearson_correlation)))
    upper_central_true_ci_cohens_drm = cohens_drm + tcrit * True_SE_dz * (np.sqrt(2*(1-pearson_correlation)))
    lower_central_true_ci_cohens_dav = cohens_dav - tcrit * True_SE_dav  
    upper_central_true_ci_cohens_dav = cohens_dav + tcrit * True_SE_dav
    
    lower_central_true_ci_cohens_gz =  hedges_gz - tcrit * True_SE_dz * correction
    upper_central_true_ci_cohens_gz =  hedges_gz + tcrit * True_SE_dz  * correction
    lower_central_true_ci_cohens_grm = hedges_grm - tcrit * True_SE_dz * (np.sqrt(2*(1-pearson_correlation)))  * correction
    upper_central_true_ci_cohens_grm = hedges_grm + tcrit * True_SE_dz * (np.sqrt(2*(1-pearson_correlation))) * correction
    lower_central_true_ci_cohens_gav = hedges_gav - tcrit * True_SE_dav   * correction
    upper_central_true_ci_cohens_gav = hedges_gav + tcrit * True_SE_dav * correction

    # Hedges Approximation (Hedges 1981b)
    Hedges_SE_dz  = np.sqrt(convertor        *  (1 + cohens_ds**2 * reversed_convertor / (2 * df1))) 
    Hedges_SE_dav = np.sqrt((2/sample_size)  *  (1 + cohens_ds**2 * (sample_size/2)    / (2 * df2))) 
    
    lower_central_Hedges_ci_cohens_dz =  cohens_dz - tcrit * Hedges_SE_dz 
    upper_central_Hedges_ci_cohens_dz =  cohens_dz + tcrit * Hedges_SE_dz
    lower_central_Hedges_ci_cohens_drm = cohens_drm - tcrit * Hedges_SE_dz * (np.sqrt(2*(1-pearson_correlation)))
    upper_central_Hedges_ci_cohens_drm = cohens_drm + tcrit * Hedges_SE_dz * (np.sqrt(2*(1-pearson_correlation)))
    lower_central_Hedges_ci_cohens_dav = cohens_dav - tcrit * Hedges_SE_dav  
    upper_central_Hedges_ci_cohens_dav = cohens_dav + tcrit * Hedges_SE_dav
    
    lower_central_Hedges_ci_cohens_gz =  hedges_gz - tcrit * Hedges_SE_dz * correction
    upper_central_Hedges_ci_cohens_gz =  hedges_gz + tcrit * Hedges_SE_dz * correction
    lower_central_Hedges_ci_cohens_grm = hedges_grm - tcrit * Hedges_SE_dz * (np.sqrt(2*(1-pearson_correlation))) * correction
    upper_central_Hedges_ci_cohens_grm = hedges_grm + tcrit * Hedges_SE_dz * (np.sqrt(2*(1-pearson_correlation))) * correction
    lower_central_Hedges_ci_cohens_gav = hedges_gav - tcrit * Hedges_SE_dav   * correction
    upper_central_Hedges_ci_cohens_gav = hedges_gav + tcrit * Hedges_SE_dav * correction

    #Hedges & Olkin Approximation
    HO_Approximate_SE_dz = np.sqrt(convertor *      (1 + cohens_ds**2 * reversed_convertor / (2 * sample_size)))   
    HO_Approximate_SE_dav = np.sqrt(2/sample_size * (1 + cohens_ds**2 * (sample_size/2)    / (2 * sample_size*2))) 
    
    lower_central_HO_approx_ci_cohens_dz =  cohens_dz - tcrit * HO_Approximate_SE_dz
    upper_central_HO_approx_ci_cohens_dz =  cohens_dz + tcrit * HO_Approximate_SE_dz
    lower_central_HO_approx_ci_cohens_drm = cohens_drm - tcrit * HO_Approximate_SE_dz * (np.sqrt(2*(1-pearson_correlation)))
    upper_central_HO_approx_ci_cohens_drm = cohens_drm + tcrit * HO_Approximate_SE_dz * (np.sqrt(2*(1-pearson_correlation)))
    lower_central_HO_approx_ci_cohens_dav = cohens_dav - tcrit * HO_Approximate_SE_dav  
    upper_central_HO_approx_ci_cohens_dav = cohens_dav + tcrit * HO_Approximate_SE_dav
    
    lower_central_HO_approx_ci_cohens_gz =  hedges_gz - tcrit * HO_Approximate_SE_dz  * correction
    upper_central_HO_approx_ci_cohens_gz =  hedges_gz + tcrit * HO_Approximate_SE_dz  * correction
    lower_central_HO_approx_ci_cohens_grm = hedges_grm - tcrit * HO_Approximate_SE_dz * (np.sqrt(2*(1-pearson_correlation)))  * correction
    upper_central_HO_approx_ci_cohens_grm = hedges_grm + tcrit * HO_Approximate_SE_dz * (np.sqrt(2*(1-pearson_correlation)))  * correction
    lower_central_HO_approx_ci_cohens_gav = hedges_gav - tcrit * HO_Approximate_SE_dav   * correction
    upper_central_HO_approx_ci_cohens_gav = hedges_gav + tcrit * HO_Approximate_SE_dav  * correction

    #MLE Approximation
    MLE_SE_dz =  np.sqrt(((df1 + 2) / df1) * convertor *       (1 + cohens_ds**2 * reversed_convertor / (2*df1))) 
    MLE_SE_dav = np.sqrt(((df2 + 2) / df2) * (2/sample_size) * (1 + cohens_ds**2 * (sample_size/2)   /  (2*df2))) 
    
    lower_central_MLE_ci_cohens_dz =  cohens_dz - tcrit * MLE_SE_dz
    upper_central_MLE_ci_cohens_dz =  cohens_dz + tcrit * MLE_SE_dz
    lower_central_MLE_ci_cohens_drm = cohens_drm - tcrit * MLE_SE_dz * (np.sqrt(2*(1-pearson_correlation)))
    upper_central_MLE_ci_cohens_drm = cohens_drm + tcrit * MLE_SE_dz * (np.sqrt(2*(1-pearson_correlation)))
    lower_central_MLE_ci_cohens_dav = cohens_dav - tcrit * MLE_SE_dav  
    upper_central_MLE_ci_cohens_dav = cohens_dav + tcrit * MLE_SE_dav
    
    lower_central_MLE_ci_cohens_gz =  hedges_gz - tcrit * MLE_SE_dz  * correction
    upper_central_MLE_ci_cohens_gz =  hedges_gz + tcrit * MLE_SE_dz * correction
    lower_central_MLE_ci_cohens_grm = hedges_grm - tcrit * MLE_SE_dz * (np.sqrt(2*(1-pearson_correlation))) * correction
    upper_central_MLE_ci_cohens_grm = hedges_grm + tcrit * MLE_SE_dz * (np.sqrt(2*(1-pearson_correlation))) * correction
    lower_central_MLE_ci_cohens_gav = hedges_gav - tcrit * MLE_SE_dav   * correction
    upper_central_MLE_ci_cohens_gav = hedges_gav + tcrit * MLE_SE_dav * correction

    #Large_n
    Large_n_SE_dz  = np.sqrt( convertor *        (1 + cohens_ds**2 / 8 ))
    Large_n_SE_dav = np.sqrt((2/sample_size) *   (1 + cohens_ds**2 / 8 )) 
    
    lower_central_Large_n_ci_cohens_dz = cohens_dz - tcrit * Large_n_SE_dz
    upper_central_Large_n_ci_cohens_dz = cohens_dz + tcrit * Large_n_SE_dz
    lower_central_Large_n_ci_cohens_drm = cohens_drm - tcrit * Large_n_SE_dz * (np.sqrt(2*(1-pearson_correlation)))
    upper_central_Large_n_ci_cohens_drm = cohens_drm + tcrit * Large_n_SE_dz * (np.sqrt(2*(1-pearson_correlation)))
    lower_central_Large_n_ci_cohens_dav = cohens_dav - tcrit * Large_n_SE_dav  
    upper_central_Large_n_ci_cohens_dav = cohens_dav + tcrit * Large_n_SE_dav
    
    lower_central_Large_n_ci_cohens_gz =  hedges_gz - tcrit * Large_n_SE_dz * correction
    upper_central_Large_n_ci_cohens_gz =  hedges_gz + tcrit * Large_n_SE_dz * correction
    lower_central_Large_n_ci_cohens_grm = hedges_grm - tcrit * Large_n_SE_dz * (np.sqrt(2*(1-pearson_correlation))) * correction
    upper_central_Large_n_ci_cohens_grm = hedges_grm + tcrit * Large_n_SE_dz * (np.sqrt(2*(1-pearson_correlation))) * correction
    lower_central_Large_n_ci_cohens_gav = hedges_gav - tcrit * Large_n_SE_dav   * correction
    upper_central_Large_n_ci_cohens_gav = hedges_gav + tcrit * Large_n_SE_dav * correction


    #Correction for small n
    Small_n_SE_dz =  np.sqrt(((df1+1)/(df1-1)) * convertor *       (1 + cohens_ds**2/8)) 
    Small_n_SE_dav = np.sqrt(((df2+1)/(df2-1)) * (2/sample_size) * (1 + cohens_ds**2/8))
    
    lower_central_Small_n_ci_cohens_dz = cohens_dz - tcrit *   Small_n_SE_dz
    upper_central_Small_n_ci_cohens_dz = cohens_dz + tcrit *   Small_n_SE_dz
    lower_central_Small_n_ci_cohens_drm = cohens_drm - tcrit * Small_n_SE_dz * (np.sqrt(2*(1-pearson_correlation)))
    upper_central_Small_n_ci_cohens_drm = cohens_drm + tcrit * Small_n_SE_dz * (np.sqrt(2*(1-pearson_correlation)))
    lower_central_Small_n_ci_cohens_dav = cohens_dav - tcrit * Small_n_SE_dav  
    upper_central_Small_n_ci_cohens_dav = cohens_dav + tcrit * Small_n_SE_dav
    
    lower_central_Small_n_ci_cohens_gz =  hedges_gz - tcrit *   Small_n_SE_dz * correction
    upper_central_Small_n_ci_cohens_gz =  hedges_gz + tcrit *   Small_n_SE_dz * correction
    lower_central_Small_n_ci_cohens_grm = hedges_grm - tcrit * Small_n_SE_dz * (np.sqrt(2*(1-pearson_correlation))) * correction
    upper_central_Small_n_ci_cohens_grm = hedges_grm + tcrit * Small_n_SE_dz * (np.sqrt(2*(1-pearson_correlation))) * correction
    lower_central_Small_n_ci_cohens_gav = hedges_gav - tcrit * Small_n_SE_dav   * correction
    upper_central_Small_n_ci_cohens_gav = hedges_gav + tcrit * Small_n_SE_dav * correction



    return cohens_dz,cohens_dav,cohens_drm,                                     \
            lower_central_true_ci_cohens_dz,    upper_central_true_ci_cohens_dz, \
            lower_central_true_ci_cohens_drm,   upper_central_true_ci_cohens_drm, \
            lower_central_true_ci_cohens_dav,   upper_central_true_ci_cohens_dav,  \
            lower_central_true_ci_cohens_gz,    upper_central_true_ci_cohens_gz, \
            lower_central_true_ci_cohens_grm,   upper_central_true_ci_cohens_grm, \
            lower_central_true_ci_cohens_gav,   upper_central_true_ci_cohens_gav,  \
                                                                                 \
            lower_central_Hedges_ci_cohens_dz,    upper_central_Hedges_ci_cohens_dz, \
            lower_central_Hedges_ci_cohens_drm,   upper_central_Hedges_ci_cohens_drm, \
            lower_central_Hedges_ci_cohens_dav,   upper_central_Hedges_ci_cohens_dav,  \
            lower_central_Hedges_ci_cohens_gz,    upper_central_Hedges_ci_cohens_gz, \
            lower_central_Hedges_ci_cohens_grm,   upper_central_Hedges_ci_cohens_grm, \
            lower_central_Hedges_ci_cohens_gav,   upper_central_Hedges_ci_cohens_gav,  \
                                                                                 \
            lower_central_HO_approx_ci_cohens_dz,    upper_central_HO_approx_ci_cohens_dz, \
            lower_central_HO_approx_ci_cohens_drm,   upper_central_HO_approx_ci_cohens_drm, \
            lower_central_HO_approx_ci_cohens_dav,   upper_central_HO_approx_ci_cohens_dav,  \
            lower_central_HO_approx_ci_cohens_gz,    upper_central_HO_approx_ci_cohens_gz, \
            lower_central_HO_approx_ci_cohens_grm,   upper_central_HO_approx_ci_cohens_grm, \
            lower_central_HO_approx_ci_cohens_gav,   upper_central_HO_approx_ci_cohens_gav,  \
                                                                                 \
            lower_central_MLE_ci_cohens_dz,    upper_central_MLE_ci_cohens_dz, \
            lower_central_MLE_ci_cohens_drm,   upper_central_MLE_ci_cohens_drm, \
            lower_central_MLE_ci_cohens_dav,   upper_central_MLE_ci_cohens_dav,  \
            lower_central_MLE_ci_cohens_gz,    upper_central_MLE_ci_cohens_gz, \
            lower_central_MLE_ci_cohens_grm,   upper_central_MLE_ci_cohens_grm, \
            lower_central_MLE_ci_cohens_gav,   upper_central_MLE_ci_cohens_gav,  \
                                                                                 \
            lower_central_Large_n_ci_cohens_dz,    upper_central_Large_n_ci_cohens_dz, \
            lower_central_Large_n_ci_cohens_drm,   upper_central_Large_n_ci_cohens_drm, \
            lower_central_Large_n_ci_cohens_dav,   upper_central_Large_n_ci_cohens_dav,  \
            lower_central_Large_n_ci_cohens_gz,    upper_central_Large_n_ci_cohens_gz, \
            lower_central_Large_n_ci_cohens_grm,   upper_central_Large_n_ci_cohens_grm, \
            lower_central_Large_n_ci_cohens_gav,   upper_central_Large_n_ci_cohens_gav,  \
                                                                                 \
            lower_central_Small_n_ci_cohens_dz,    upper_central_Small_n_ci_cohens_dz, \
            lower_central_Small_n_ci_cohens_drm,   upper_central_Small_n_ci_cohens_drm, \
            lower_central_Small_n_ci_cohens_dav,   upper_central_Small_n_ci_cohens_dav,  \
            lower_central_Small_n_ci_cohens_gz,    upper_central_Small_n_ci_cohens_gz, \
            lower_central_Small_n_ci_cohens_grm,   upper_central_Small_n_ci_cohens_grm, \
            lower_central_Small_n_ci_cohens_gav,   upper_central_Small_n_ci_cohens_gav
            
def Confidence_Interval_Central_Independent (sample_mean_1, sample_mean_2, sample_sd_1,sample_sd_2, sample_size_1, sample_size_2, pearson_correlation, population_mean_diff, confidence_level):
    sample_size = sample_size_1 + sample_size_2
    df = sample_size - 2 #degrees of freedom for between subject design
    correction = correction_factor(df)
    standardizer_ds = np.sqrt(((((sample_size_1-1)*sample_sd_1**2)) + ((sample_size_2-1)*sample_sd_2**2)) / (sample_size-2))
    h1 = (sample_size_1 + sample_size_2) / (sample_size_1 * sample_size_2) #2/(harmonic_mean of n1,n2)
    h2 = (sample_size_1 + sample_size_2) / (sample_size_1 * sample_size_2) # (harmonic_mean of n1,n2)/2
   
    cohens_ds = (((sample_mean_1-sample_mean_2) - population_mean_diff )/standardizer_ds)
    cohens_dpop = cohens_ds / np.sqrt((sample_size-2)/sample_size)
    hedges_gs = cohens_ds*correction
    tcrit = scipy.stats.t.ppf(1-(1-confidence_level)/2, df) #this is the critical value for calculating the confidence interval for t score

    #True Formula (Hedges 1981a):
    True_SE_ds =  np.sqrt ((df/(df-2)) * (h1) *     (1 + cohens_ds**2 * h2) - (cohens_ds**2 / (correction_factor(df))**2))
    
    lower_central_true_ci_cohens_ds =  cohens_ds - tcrit * True_SE_ds
    upper_central_true_ci_cohens_ds =  cohens_ds + tcrit * True_SE_ds
    lower_central_true_ci_cohens_dpop = cohens_dpop - tcrit * True_SE_ds / np.sqrt((sample_size-2)/sample_size)
    upper_central_true_ci_cohens_dpop = cohens_dpop + tcrit * True_SE_ds / np.sqrt((sample_size-2)/sample_size)
    lower_central_true_ci_cohens_gs =  hedges_gs - tcrit * True_SE_ds * correction
    upper_central_true_ci_cohens_gs =  hedges_gs + tcrit * True_SE_ds  * correction
    
    # Hedges Approximation (Hedges 1981b)
    Hedges_SE_ds  = np.sqrt(h1        *  (1 + cohens_ds**2 * h2 / (2 * df))) 
    
    lower_central_Hedges_ci_cohens_ds =  cohens_ds - tcrit * Hedges_SE_ds 
    upper_central_Hedges_ci_cohens_ds =  cohens_ds + tcrit * Hedges_SE_ds
    lower_central_Hedges_ci_cohens_dpop =  cohens_dpop - tcrit * Hedges_SE_ds / np.sqrt((sample_size-2)/sample_size)
    upper_central_Hedges_ci_cohens_dpop =  cohens_dpop + tcrit * Hedges_SE_ds / np.sqrt((sample_size-2)/sample_size)
    lower_central_Hedges_ci_cohens_gs =  hedges_gs - tcrit * Hedges_SE_ds * correction
    upper_central_Hedges_ci_cohens_gs =  hedges_gs + tcrit * Hedges_SE_ds * correction
   
    #Hedges & Olkin Approximation
    HO_Approximate_SE_ds = np.sqrt(h1 *      (1 + cohens_ds**2 * h2 / (2 * sample_size)))   
    
    lower_central_HO_approx_ci_cohens_ds =  cohens_ds - tcrit *  HO_Approximate_SE_ds
    upper_central_HO_approx_ci_cohens_ds =  cohens_ds + tcrit *  HO_Approximate_SE_ds
    lower_central_HO_approx_ci_cohens_dpop = cohens_dpop  - tcrit * HO_Approximate_SE_ds / np.sqrt((sample_size-2)/sample_size)
    upper_central_HO_approx_ci_cohens_dpop = cohens_dpop  + tcrit * HO_Approximate_SE_ds / np.sqrt((sample_size-2)/sample_size)    
    lower_central_HO_approx_ci_cohens_gs =  hedges_gs - tcrit *  HO_Approximate_SE_ds * correction
    upper_central_HO_approx_ci_cohens_gs =  hedges_gs + tcrit *  HO_Approximate_SE_ds * correction

    #MLE Approximation
    MLE_SE_ds =  np.sqrt(((df + 2) / df) * h1 *       (1 + cohens_ds**2 * h2 / (2*df))) 
    
    lower_central_MLE_ci_cohens_ds =  cohens_ds - tcrit * MLE_SE_ds
    upper_central_MLE_ci_cohens_ds =  cohens_ds + tcrit * MLE_SE_ds
    lower_central_MLE_ci_cohens_dpop = cohens_dpop  - tcrit * MLE_SE_ds / np.sqrt((sample_size-2)/sample_size)
    upper_central_MLE_ci_cohens_dpop = cohens_dpop  + tcrit * MLE_SE_ds / np.sqrt((sample_size-2)/sample_size)    
    lower_central_MLE_ci_cohens_gs =  hedges_gs - tcrit * MLE_SE_ds  * correction
    upper_central_MLE_ci_cohens_gs =  hedges_gs + tcrit * MLE_SE_ds * correction

    #Large_n
    Large_n_SE_ds  = np.sqrt( h1 *        (1 + cohens_ds**2 / 8 ))
    
    lower_central_Large_n_ci_cohens_ds =  cohens_ds - tcrit * Large_n_SE_ds
    upper_central_Large_n_ci_cohens_ds =  cohens_ds + tcrit * Large_n_SE_ds
    lower_central_Large_n_ci_cohens_dpop = cohens_dpop  - tcrit * Large_n_SE_ds / np.sqrt((sample_size-2)/sample_size)
    upper_central_Large_n_ci_cohens_dpop = cohens_dpop  + tcrit * Large_n_SE_ds / np.sqrt((sample_size-2)/sample_size) 
    lower_central_Large_n_ci_cohens_gs =  hedges_gs - tcrit * Large_n_SE_ds * correction
    upper_central_Large_n_ci_cohens_g =  hedges_gs + tcrit * Large_n_SE_ds * correction


    #Correction for small n
    Small_n_SE_ds =  np.sqrt(((df+1)/(df-1)) * h1 *       (1 + cohens_ds**2/8)) 
    
    lower_central_Small_n_ci_cohens_ds =  cohens_ds - tcrit *   Small_n_SE_ds
    upper_central_Small_n_ci_cohens_ds =  cohens_ds + tcrit *   Small_n_SE_ds
    lower_central_Small_n_ci_cohens_dpop = cohens_dpop  - tcrit * Small_n_SE_ds / np.sqrt((sample_size-2)/sample_size)
    upper_central_Small_n_ci_cohens_dpop = cohens_dpop  + tcrit * Small_n_SE_ds / np.sqrt((sample_size-2)/sample_size)    
    lower_central_Small_n_ci_cohens_gs =  hedges_gs - tcrit *   Small_n_SE_ds * correction
    upper_central_Small_n_ci_cohens_gs =  hedges_gs + tcrit *   Small_n_SE_ds * correction


    return cohens_ds,cohens_dpop, hedges_gs,                                     \
            lower_central_true_ci_cohens_ds,    upper_central_true_ci_cohens_ds, \
            lower_central_true_ci_cohens_dpop,   upper_central_true_ci_cohens_dpop, \
            lower_central_true_ci_cohens_gs,   upper_central_true_ci_cohens_gs,  \
                                                                       \
            lower_central_Hedges_ci_cohens_ds,    upper_central_Hedges_ci_cohens_ds, \
            lower_central_Hedges_ci_cohens_dpop,   upper_central_Hedges_ci_cohens_dpop, \
            lower_central_Hedges_ci_cohens_gs,   upper_central_Hedges_ci_cohens_gs,  \
                                                                                 \
            lower_central_HO_approx_ci_cohens_ds,    upper_central_HO_approx_ci_cohens_ds, \
            lower_central_HO_approx_ci_cohens_dpop,   upper_central_HO_approx_ci_cohens_dpop, \
            lower_central_HO_approx_ci_cohens_gs,   upper_central_HO_approx_ci_cohens_gs,  \
                                                                  \
            lower_central_MLE_ci_cohens_ds,    upper_central_MLE_ci_cohens_ds, \
            lower_central_MLE_ci_cohens_dpop,   upper_central_MLE_ci_cohens_dpop, \
            lower_central_MLE_ci_cohens_gs,   upper_central_MLE_ci_cohens_gs,  \
                                                                    \
            lower_central_Large_n_ci_cohens_ds,    upper_central_Large_n_ci_cohens_ds, \
            lower_central_Large_n_ci_cohens_dpop,   upper_central_Large_n_ci_cohens_dpop, \
            lower_central_Large_n_ci_cohens_gs,   upper_central_Large_n_ci_cohens_g,  \
                                                                     \
            lower_central_Small_n_ci_cohens_ds,    upper_central_Small_n_ci_cohens_ds, \
            lower_central_Small_n_ci_cohens_dpop,   upper_central_Small_n_ci_cohens_dpop, \
            lower_central_Small_n_ci_cohens_gs,   upper_central_Small_n_ci_cohens_gs


###############################################
##### Relevant Functions For Correlations!!####
###############################################

### 1. Goodman and Kruskal Lambda Coefficeint
def goodman_kruskal_lamda_correlation(matrix, confidence_level):
    
        # Calcluation of the Matrix follows Hartwig, 1976
        #################################################

        # Please Note that SPSS and DescTools have a different method for calculating the SRK in the symmetric method - they are both yields the same results 


        # Marginal total Frequencies
        csum = np.sum(matrix, axis=0)
        rsum = np.sum(matrix, axis=1)
        
        n = np.sum(matrix) # Sample Size
        Nrc = np.sum(np.max(matrix, axis=1)) # Sum of the largest values in each Row
        Nkc = np.sum(np.max(matrix, axis=0)) # Sum of the largest values in each Coloumn
        Nrm = np.max(rsum) # This is the largest value in rows
        Nkm = np.max(csum) # This is the largest value in coloumns
        Um = Nrm + Nkm #For later calculations shortcuts
        Uc = Nrc + Nkc #for later calculations shortcuts
        
        # Calculation of Nr_tag and Nk_tag - In case of duplicates one need to store all the combinations of Nr_tag and Nktag

        # Find row(s) with the largest row sum and return a vector for the Nr_tag
        row_sums = np.sum(matrix, axis=1)
        rows_with_largest_rsum = np.where(row_sums == Nrm)[0]
        largest_row_values_vector = [] # Create a vector to store the largest values from the rows with the largest rsum

        # Iterate through the rows with the largest rsum and find the largest value in each row
        for row_idx in rows_with_largest_rsum:
            largest_value_in_row = np.max(matrix[row_idx, :])
            largest_row_values_vector.append(largest_value_in_row)
        largest_rows_vector = np.array(largest_row_values_vector)

        # Find row(s) with the largest coloumn sum and return a vector for the Nk_tag
        column_sums = np.sum(matrix, axis=0)
        columns_with_largest_csum = np.where(column_sums == Nkm)[0]
        largest_col_values_vector = []

        # Iterate through the columns with the largest csum and find the largest value in each column
        for col_idx in columns_with_largest_csum:
            largest_value_in_column = np.max(matrix[:, col_idx])
            largest_col_values_vector.append(largest_value_in_column)
        largest_col_vector = np.array(largest_col_values_vector)

        #Now calculate all the possible combination of Nks_tag and Nr_tag and return them as seperate vectors
        combinations_nks_nrs = list(product(largest_rows_vector, largest_col_vector))
        Nk_tag = [item[0] for item in combinations_nks_nrs]
        Nr_tag = [item[1] for item in combinations_nks_nrs]

        # Calculation of Ntag that can also have multiple values in case of marginal totals frequencies ties
        #Locate the row
        rows_with_most_frequencies = np.where(rsum == Nrm)[0]  
        cols_with_most_frequencies = np.where(csum == Nkm)[0]

        N_tags_combinations = list(product(rows_with_most_frequencies, cols_with_most_frequencies))
        N_tags = []

        #Ntags values - test which rows\coloumns in the table have 
        for row_idx, col_idx in N_tags_combinations:
            N_tags.append(matrix[row_idx, col_idx])

        #Skcr and Srck - we need to search for the rows and coloums with the largest marginal rows and 
        
        #Skcr
        # Create a vector to store the largest values from the rows with the largest rsum
        row_sums = np.sum(matrix, axis=1)
        rows_with_largest_rsum = np.where(row_sums == Nrm)[0]
        largest_row_values_vector = []  # Create a vector to store the largest values from the rows with the largest rsum

        # Iterate through the rows with the largest rsum and find the largest value in each row
        for row_idx in rows_with_largest_rsum:
            largest_value_in_row = np.max(matrix[row_idx, :])
            largest_row_values_vector.append(largest_value_in_row)

        # Sum the values in each row that are the maximum in their respective columns
        sum_of_highest_values_rows = []
        for row_idx in rows_with_largest_rsum:
            max_values_in_cols = [matrix[row_idx, col_idx] for col_idx in range(matrix.shape[1]) if matrix[row_idx, col_idx] == np.max(matrix[:, col_idx])]
            row_sum_of_highest_values = sum(max_values_in_cols)
            sum_of_highest_values_rows.append(row_sum_of_highest_values)

        sum_of_highest_values_row_vector = np.array(sum_of_highest_values_rows)

        #Srck 
        col_sums = np.sum(matrix, axis=0)
        cols_with_largest_csum = np.where(col_sums == Nkm)[0]
        largest_col_values_vector = []  # Create a vector to store the largest values from the rows with the largest rsum

        # Iterate through the rows with the largest rsum and find the largest value in each row
        for col_idx in cols_with_largest_csum:
            largest_value_in_col = np.max(matrix[:, col_idx])
            largest_col_values_vector.append(largest_value_in_col)

        # Sum the values in each col that are the maximum in their respective row
        sum_of_highest_values_col = []
        for col_idx in cols_with_largest_csum:
            max_values_in_rows = [matrix[row_idx, col_idx] for row_idx in range(matrix.shape[0]) if matrix[row_idx,col_idx] == np.max(matrix[row_idx,: ])]
            col_sum_of_highest_values = sum(max_values_in_rows)
            sum_of_highest_values_col.append(col_sum_of_highest_values)

        sum_of_highest_values_col_vector = np.array(sum_of_highest_values_col)

        #Now calculate all the possible combination of Nks_tag and Nr_tag and return them as a seperate vectors
        combinations_Skcr_Srck = list(product(sum_of_highest_values_row_vector, sum_of_highest_values_col_vector))
        Skcr = [item[0] for item in combinations_Skcr_Srck] #This is basically the sum of all largest values in each coloumn seperatly for each row
        Srck = [item[1] for item in combinations_Skcr_Srck] #This is basically the sum of all largest values in each row seperatly for each coloumn

        #Calculate Srk - sum of all  largest values in both their row and coloumn
        largest_both_values = []
        
        for row_idx in range(matrix.shape[0]):
            for col_idx in range(matrix.shape[1]):
                value = matrix[row_idx, col_idx]
                if value == np.max(matrix[row_idx, :]) and value == np.max(matrix[:, col_idx]):
                    largest_both_values.append(value)
        
        Srk = np.sum(largest_both_values)

        #Now the Utag for shortening calcualtions
        Utag = []
        for i in range(len(Skcr)):
            sum_value = Skcr[i] + Srck[i] + Nr_tag[i] + Nk_tag[i]
            Utag.append(sum_value)


        # Final Parameters - Note that SAS has a wrong calcualation

    
        #Standard Errors
        Standard_Error_Rows = []
        for srck_value in Srck:
            standard_error_row = 1 / np.sqrt(((n - Nkm) ** 3) / ((n - Nrc) * (Nrc + Nkm - 2 * srck_value)))
            Standard_Error_Rows.append(standard_error_row)

        # Calculate Standard Errors for Columns
        Standard_Error_Coloumns = []
        for skcr_value in Skcr:
            standard_error_col = 1 / np.sqrt(((n - Nrm) ** 3) / ((n - Nkc) * (Nkc + Nrm - 2 * skcr_value)))
            Standard_Error_Coloumns.append(standard_error_col)
    
    # Calculate Symmetric Standard Errors 
        Standard_Error_Symmetric = []
        for Utag_value, Ntags_Value in zip(Utag, N_tags):
            standard_error_sym  = 1/ ((2*n - Um)**4 / ((2*n - Um) * (2*n - Uc) * (Um + Uc + 4*n - 2 *Utag_value) - 2*(2*n - Um)**2 * (n-Srk) - 2*(2*n - Uc)**2 * (n-Ntags_Value)))**0.5
            Standard_Error_Symmetric.append(standard_error_sym)

        #Final Parameters for Output
        
        # Lambda Values
        Lambda_row = (Nrc - Nkm) / (n - Nkm)
        Lambda_col = (Nkc - Nrm) / (n - Nrm)
        Lambda_symmetric = (Nrc + Nkc - Nrm - Nkm) / (2 * n - Nrm - Nkm)

        # Standard Errors Methods CI's and inferential statistics
        z_critical_value = st.norm.ppf(confidence_level + ((1 - confidence_level) / 2))

        # 1. Random Method - other software use the first values in matrix - here is the random method
        Standard_Error_Rows_random = random.choice(Standard_Error_Rows)
        Standard_Error_Coloumns_random = random.choice(Standard_Error_Coloumns)
        Standard_Error_Symmetric_random = random.choice(Standard_Error_Symmetric)
        Z_row_rand = Lambda_row / Standard_Error_Rows_random
        Z_col_rand = Lambda_col / Standard_Error_Coloumns_random
        Z_symmetric_rand = Lambda_symmetric / Standard_Error_Symmetric_random
        pval_row_rand = calculate_p_value_from_z_score(Z_row_rand)
        pval_col_rand = calculate_p_value_from_z_score(Z_col_rand)
        pval_symmetric_rand = calculate_p_value_from_z_score(Z_symmetric_rand)
        lower_ci_lambda_rows_rand = max(Lambda_row - z_critical_value * Standard_Error_Rows_random,0)
        upper_ci_lambda_rows_rand = min(Lambda_row + z_critical_value * Standard_Error_Rows_random,1)
        lower_ci_lambda_cols_rand = max(Lambda_col - z_critical_value * Standard_Error_Coloumns_random,0)
        upper_ci_lambda_cols_rand = min(Lambda_col + z_critical_value * Standard_Error_Coloumns_random,1)
        lower_ci_lambda_symmetric_rand = max(Lambda_symmetric - z_critical_value * Standard_Error_Symmetric_random,0)
        upper_ci_lambda_symmetric_rand = min(Lambda_symmetric + z_critical_value * Standard_Error_Symmetric_random,1)
        
        # 2. Maximum Method - Strict Method - Uses the Largest value
        Standard_Error_Rows_max = np.max(Standard_Error_Rows)
        Standard_Error_Coloumns_max = np.max(Standard_Error_Coloumns)
        Standard_Error_Symmetric_max = np.max(Standard_Error_Symmetric)
        Z_row_max = Lambda_row / Standard_Error_Rows_max
        Z_col_max = Lambda_col / Standard_Error_Coloumns_max
        Z_symmetric_max = Lambda_symmetric / Standard_Error_Symmetric_max
        pval_row_max = calculate_p_value_from_z_score(Z_row_max)
        pval_col_max = calculate_p_value_from_z_score(Z_col_max)
        pval_symmetric_max = calculate_p_value_from_z_score(Z_symmetric_max)
        lower_ci_lambda_rows_max = max(Lambda_row - z_critical_value * Standard_Error_Rows_max,0)
        upper_ci_lambda_rows_max = min(Lambda_row + z_critical_value * Standard_Error_Rows_max,1)
        lower_ci_lambda_cols_max = max(Lambda_col - z_critical_value * Standard_Error_Coloumns_max,0)
        upper_ci_lambda_cols_max = min(Lambda_col + z_critical_value * Standard_Error_Coloumns_max,1)
        lower_ci_lambda_symmetric_max = max(Lambda_symmetric - z_critical_value * Standard_Error_Symmetric_max,0)
        upper_ci_lambda_symmetric_max = min(Lambda_symmetric + z_critical_value * Standard_Error_Symmetric_max,1)

        # 3. Mean Method - Average of all standard Error
        Standard_Error_Rows_mean = np.mean(Standard_Error_Rows)
        Standard_Error_Coloumns_mean = np.mean(Standard_Error_Coloumns)
        Standard_Error_Symmetric_mean = np.mean(Standard_Error_Symmetric)
        Z_row_mean = Lambda_row / Standard_Error_Rows_mean
        Z_col_mean = Lambda_col / Standard_Error_Coloumns_mean
        Z_symmetric_mean = Lambda_symmetric / Standard_Error_Symmetric_mean
        pval_row_mean = calculate_p_value_from_z_score(Z_row_mean)
        pval_col_mean = calculate_p_value_from_z_score(Z_col_mean)
        pval_symmetric_mean = calculate_p_value_from_z_score(Z_symmetric_mean)
        lower_ci_lambda_rows_mean = max(Lambda_row - z_critical_value * Standard_Error_Rows_mean,0)
        upper_ci_lambda_rows_mean = min(Lambda_row + z_critical_value * Standard_Error_Rows_mean,1)
        lower_ci_lambda_cols_mean = max(Lambda_col - z_critical_value * Standard_Error_Coloumns_mean,0)
        upper_ci_lambda_cols_mean = min(Lambda_col + z_critical_value * Standard_Error_Coloumns_mean,1)
        lower_ci_lambda_symmetric_mean = max(Lambda_symmetric - z_critical_value * Standard_Error_Symmetric_mean,0)
        upper_ci_lambda_symmetric_mean = min(Lambda_symmetric + z_critical_value * Standard_Error_Symmetric_mean,1)

        results = {}

        # Lambda Values
        results["Lambda Rows"] = round(Lambda_row, 7)
        results["Lambda Columns"] = round(Lambda_col, 7)
        results["Lambda Symmetric"] = round(Lambda_symmetric, 7)

        # Random Method
        results["Random Method - P-Value Rows"] = pval_row_rand
        results["Random Method - P-Value Columns"] = pval_col_rand
        results["Random Method - P-Value Symmetric"] = pval_symmetric_rand
        results["Random Method - Confidence Interval Rows"] = [round(lower_ci_lambda_rows_rand, 7), round(upper_ci_lambda_rows_rand, 7)]
        results["Random Method - Confidence Interval Columns"] = [round(lower_ci_lambda_cols_rand, 7), round(upper_ci_lambda_cols_rand, 7)]
        results["Random Method - Confidence Interval Symmetric"] = [round(lower_ci_lambda_symmetric_rand, 7), round(upper_ci_lambda_symmetric_rand, 4)]
        results["Random Method - Standard Error Rows"] = round(Standard_Error_Rows_random, 7)
        results["Random Method - Standard Error Columns"] = round(Standard_Error_Coloumns_random, 7)
        results["Random Method - Standard Error Symmetric"] = round(Standard_Error_Symmetric_random, 7)

        # Maximum Method
        results["Maximum Method - P-Value Rows"] = pval_row_max
        results["Maximum Method - P-Value Columns"] = pval_col_max
        results["Maximum Method - P-Value Symmetric"] = pval_symmetric_max
        results["Maximum Method - Confidence Interval Rows"] = [round(lower_ci_lambda_rows_max, 7), round(upper_ci_lambda_rows_max, 7)]
        results["Maximum Method - Confidence Interval Columns"] = [round(lower_ci_lambda_cols_max, 7), round(upper_ci_lambda_cols_max, 7)]
        results["Maximum Method - Confidence Interval Symmetric"] = [round(lower_ci_lambda_symmetric_max, 7), round(upper_ci_lambda_symmetric_max, 4)]
        results["Maximum Method - Standard Error Rows"] = round(Standard_Error_Rows_max, 7)
        results["Maximum Method - Standard Error Columns"] = round(Standard_Error_Coloumns_max, 7)
        results["Maximum Method - Standard Error Symmetric"] = round(Standard_Error_Symmetric_max, 7)

        # Mean Method
        results["Mean Method - P-Value Rows"] = pval_row_mean
        results["Mean Method - P-Value Columns"] = pval_col_mean
        results["Mean Method - P-Value Symmetric"] = pval_symmetric_mean
        results["Mean Method - Confidence Interval Rows"] = [round(lower_ci_lambda_rows_mean, 7), round(upper_ci_lambda_rows_mean, 7)]
        results["Mean Method - Confidence Interval Columns"] = [round(lower_ci_lambda_cols_mean, 7), round(upper_ci_lambda_cols_mean, 7)]
        results["Mean Method - Confidence Interval Symmetric"] = [round(lower_ci_lambda_symmetric_mean, 7), round(upper_ci_lambda_symmetric_mean, 7)]
        results["Mean Method - Standard Error Rows"] = round(Standard_Error_Rows_mean, 7)
        results["Mean Method - Standard Error Columns"] = round(Standard_Error_Coloumns_mean, 7)
        results["Mean Method - Standard Error Symmetric"] = round(Standard_Error_Symmetric_mean, 7)

        result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
        return result_str

###2. Goodman and Kruskal Tau (Data (x variable) needs to be in a matrix that rperesents contingency Table)

def Goodman_Kruskal_Tau(x, confidence_level=0.95):
    #This function uses the Method presented in Libertau 1983 to calcualte the Standard Error

    # Global Variables
    sample_size = np.sum(x)
    Sum_Of_The_Rows = np.sum(x, axis=1)
    Sum_Of_The_Columns = np.sum(x, axis=0)
    Conditional_Errors_Coloumns = sample_size ** 2 - np.sum(Sum_Of_The_Columns ** 2)
    Conditional_Errors_Rows = sample_size ** 2 - np.sum(Sum_Of_The_Rows ** 2)
    Mean_Rows = Conditional_Errors_Rows / (sample_size ** 2)
    Mean_Coloumns = Conditional_Errors_Coloumns / (sample_size ** 2)

    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)

    # Calculate Tau_r for the Rows
    Unconditonal_Error_Rows = (sample_size ** 2) - sample_size * np.sum((x[:, np.newaxis] ** 2) / Sum_Of_The_Columns[np.newaxis])
    Tau_Rows = 1 - (Unconditonal_Error_Rows / Conditional_Errors_Rows)
    v = Unconditonal_Error_Rows / (sample_size ** 2)
    ASE_Rows = np.sqrt(np.sum((x * (-2 * v * (Sum_Of_The_Rows[:, np.newaxis] / sample_size) + Mean_Rows * ((2 * x / Sum_Of_The_Columns) - np.sum((x / Sum_Of_The_Columns) ** 2, axis=0)) - (Mean_Rows * (v + 1) - 2 * v)) ** 2) / (sample_size ** 2 * Mean_Rows ** 4)))
    Confidence_Intervals_Rows = zcrit * ASE_Rows * np.array([-1, 1]) + Tau_Rows

    # Calculate Tau_c for the Coloumns
    Unconditional_Error_Coloumns = (sample_size ** 2) - sample_size * np.sum((x ** 2) / Sum_Of_The_Rows[:, np.newaxis])
    Tau_Coloumns = 1 - (Unconditional_Error_Coloumns / Conditional_Errors_Coloumns)
    v2 = Unconditional_Error_Coloumns / (sample_size ** 2)

    ASE_Columns = 0
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            term = x[i, j] * (-2 * v2 * (Sum_Of_The_Columns[j] / sample_size) + Mean_Coloumns * ((2 * x[i, j] / Sum_Of_The_Rows[i]) - np.sum((x[i, :] / Sum_Of_The_Rows[i]) ** 2)) - (Mean_Coloumns * (v2 + 1) - 2 * v2)) ** 2 / (sample_size ** 2 * Mean_Coloumns ** 4)
            ASE_Columns = ASE_Columns + term
    ASE_Columns = np.sqrt(ASE_Columns)
    Confidence_Intervals_Coloumns = zcrit * ASE_Columns * np.array([-1, 1]) + Tau_Coloumns

    # Symmetric Tau
    alpha = ((sample_size ** 2) - np.sum(Sum_Of_The_Rows**2)) / ((2* (sample_size ** 2)) - np.sum(Sum_Of_The_Rows**2) - np.sum(Sum_Of_The_Columns**2))
    Tau_Symmetric = (Tau_Rows*alpha) + (1-alpha)*Tau_Coloumns
    ASE_Symmetric = (ASE_Rows*alpha) + (1-alpha)*ASE_Columns
    Confidence_Intervals_Symmetric = zcrit * ASE_Symmetric * np.array([-1, 1]) + Tau_Symmetric

    results = {}

    results["Goodman Kruskal Tau (Rows):"]= Tau_Rows
    results["Standard Error Rows"] = ASE_Rows
    results["Goodman-Kruskal CI's (Rows)"] = Confidence_Intervals_Rows
    results["Goodman Kruskal Tau (Columns)"]= Tau_Coloumns
    results["Standard Error Coloumns"]= ASE_Columns
    results["Goodman-Kruskal CI's (Coloumns)"]= Confidence_Intervals_Coloumns
    results["Goodman Kruskal Tau (Symmetric)"]= Tau_Symmetric
    results["Standard Error Symmetric"]= ASE_Symmetric
    results["Goodman-Kruskal CI's (Symmetric)"]= Confidence_Intervals_Symmetric

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

###3. Uncertainty Coefficient - Needs to be in a Contingency Table

def uncertainty_coefficient(contingency_table, confidence_level = 0.95):
    Sum_Of_Rows = np.sum(contingency_table, axis=1)
    Sum_Of_Columns = np.sum(contingency_table, axis=0)
    sample_size = np.sum(contingency_table)
    HX = -np.sum((Sum_Of_Rows * np.log(Sum_Of_Rows / sample_size)) / sample_size)
    HY = -np.sum((Sum_Of_Columns * np.log(Sum_Of_Columns / sample_size)) / sample_size)
    HXY = -np.sum(contingency_table * np.log(contingency_table / sample_size)) / sample_size
    
    # Calculate Effect Size UC 
    Uncertainty_Coefficient_Symmetric = 2 * (HX + HY - HXY) / (HX + HY)
    Uncertainty_Coefficient_Rows = (HX + HY - HXY) / HX
    Uncertainty_Coefficient_Columns = (HX + HY - HXY) / HY

    # Calculate the Asymptotic Standard Errors (Standard Errors)
    Standard_Error_Symmetric = np.sqrt((4 * np.sum(contingency_table * (HXY * np.log(np.outer(Sum_Of_Rows, Sum_Of_Columns) / sample_size ** 2) - (HX + HY) * np.log(contingency_table / sample_size)) ** 2) / (sample_size ** 2 * (HX + HY) ** 4)))
    Standard_Error_Rows = np.sqrt(np.sum(contingency_table * (HX * np.log(contingency_table / Sum_Of_Columns) + (HY - HXY) * np.log(Sum_Of_Rows / sample_size)[:, np.newaxis]) ** 2) / (sample_size ** 2 * HX ** 4))
    Standard_Error_Columns = np.sqrt(np.sum(contingency_table * (HY * np.log(contingency_table / Sum_Of_Rows[:, np.newaxis]) + (HX - HXY) * np.log(Sum_Of_Columns / sample_size)) ** 2) / (sample_size ** 2 * HY ** 4))

    # Calculate p_values        
    Z_value_Symmetric = Uncertainty_Coefficient_Symmetric / Standard_Error_Symmetric
    Z_value_Rows = Uncertainty_Coefficient_Rows / Standard_Error_Rows
    Z_value_Columns = Uncertainty_Coefficient_Columns / Standard_Error_Columns

    # Confidence Intervals
    Zcrit = 1 - (1 - confidence_level) / 2
    Symmetric_CIs = Zcrit * np.sqrt(Standard_Error_Symmetric) * np.array([-1, 1]) + Uncertainty_Coefficient_Symmetric
    Rows_CIs = Zcrit * np.sqrt(Standard_Error_Rows) * np.array([-1, 1]) + Uncertainty_Coefficient_Rows
    Columns_CIs = Zcrit * np.sqrt(Standard_Error_Columns) * np.array([-1, 1]) + Uncertainty_Coefficient_Columns

    results = {}

    results["Uncertainty Coefficient (Symmetric)"] = Uncertainty_Coefficient_Symmetric
    results["Uncertainty Coefficient (Rows)"] = Uncertainty_Coefficient_Rows
    results["Uncertainty Coefficient (Columns)"] = Uncertainty_Coefficient_Columns
    results["Uncertainty Coefficient Standard Error (Symmetric)"] = Standard_Error_Symmetric
    results["Uncertainty Coefficient Standard Error (Rows)"] = Standard_Error_Rows
    results["Uncertainty Coefficient Standard Error (Columns)"] = Standard_Error_Columns
    results["Uncertainty Coefficient Z-value (Symmetric)"] = Z_value_Symmetric
    results["Uncertainty Coefficient Z-value (Rows)"] = Z_value_Rows
    results["Uncertainty Coefficient Z-value (Columns)"] = Z_value_Columns
    results["Uncertainty Coefficient Confidence Intervals (Symmetric)"] = Symmetric_CIs
    results["Uncertainty Coefficient Confidence Intervals (Rows)"] = Rows_CIs
    results["Uncertainty Coefficient Confidence Intervals (Columns)"] = Columns_CIs

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

### 4. likelihood_ratio

def likelihood_ratio(contingency_table):
        Observed = contingency_table
        Sum_Product = np.sum(Observed)
        Expected = np.outer(np.sum(Observed, axis=1), np.sum(Observed, axis=0)) / Sum_Product
        likelihood_ratio = 0
        for i in range(2):
            for j in range(2):
                likelihood_ratio += Observed[i, j] * np.log(Observed[i, j] / Expected[i, j])
        likelihood_ratio = 2 * likelihood_ratio
        p_value = 1 - chi2.cdf(likelihood_ratio, 1)
        return [likelihood_ratio, p_value]



### 5. Berry_Mielke_Maximum_Corrected_Cramer_V
def Berry_Mielke_Maximum_Corrected_Cramer_V(matrix):
    Observed_Chi_Square = chi2_contingency(matrix).statistic
    r, c = matrix.shape 
    sum_of_rows_vector = matrix.sum(axis=1)  
    sum_of_cols_vector = matrix.sum(axis=0)  
    NR = sum_of_rows_vector.sum()  
    NC = sum_of_cols_vector.sum() 

    matrix = np.zeros((r, c)) 
    x = np.where(np.isin(sum_of_cols_vector, sum_of_rows_vector), np.argmax(np.isin(sum_of_rows_vector, sum_of_cols_vector), axis=0) + 1, np.nan)
    y = np.where(np.isin(sum_of_rows_vector, sum_of_cols_vector), np.argmax(np.isin(sum_of_cols_vector, sum_of_rows_vector), axis=0) + 1, np.nan)
    x = x[~np.isnan(x)].astype(int) - 1  
    y = y[~np.isnan(y)].astype(int) - 1  

    matrix[x,y] = sum_of_rows_vector[x]
    sum_of_rows_vector[x] = sum_of_rows_vector[x]-sum_of_rows_vector[x] 
    sum_of_cols_vector[y] = sum_of_cols_vector[y]-sum_of_cols_vector[y] 

    while NR > 0 and NC > 0:
        NR = np.sum(sum_of_rows_vector)  
        NC = np.sum(sum_of_cols_vector)  
        x = np.argmax(sum_of_rows_vector)  
        y = np.argmax(sum_of_cols_vector)  
        z = min(sum_of_rows_vector[x], sum_of_cols_vector[y]) 
        matrix[x, y] = z
        sum_of_rows_vector[x] -= z
        sum_of_cols_vector[y] -= z

    Maximum_Chi_Square = chi2_contingency(matrix).statistic
    Maximum_Corrected_Cramers_v = Observed_Chi_Square/ Maximum_Chi_Square

    return Maximum_Corrected_Cramers_v


### 6. Freemans_Theta_for_Nominal_by_Ordinal correlation
def Fremmans_Theta(Contingency_Table):
    contingency_table = pd.DataFrame(Contingency_Table)
    row_names = contingency_table.index
    vectors = {}
    contrasts = {}

    for i in range(len(row_names)):
        for j in range(i + 1, len(row_names)):
            row1 = contingency_table.loc[row_names[i]].values
            row2 = contingency_table.loc[row_names[j]].values
            vector = np.multiply.outer(row1, row2)
            key = f'{row_names[i]}_{row_names[j]}'
            vectors[key] = vector

    for i in range(len(row_names)):
        for j in range(i + 1, len(row_names)):
            vector = vectors[f'{row_names[i]}_{row_names[j]}']
            contrast = np.sum(np.triu(vector)) - np.sum(np.tril(vector))
            key = f'{row_names[i]}_{row_names[j]}'
            contrasts[key] = contrast

    Delta = np.sum(np.abs(list(contrasts.values())))

    row_sums = contingency_table.sum(axis=1).values
    T2 = np.sum(np.triu(np.outer(row_sums, row_sums), k=1))
    Theta = Delta / T2

    return Theta

#### 7. Kendall's Tau versions
def Kendall_Tau_Family(contingency_table, confidence_level = 0.95): #inspired by Desctools function
    number_of_rows, number_of_columns = contingency_table.shape
    concordant_table = np.zeros((number_of_rows, number_of_columns))
    disconcordant_table = np.zeros((number_of_rows, number_of_columns))
    for i in range(number_of_rows):
        for j in range(number_of_columns):
            concordant_table[i, j] = np.sum(contingency_table[:i, :j]) + np.sum(contingency_table[i+1:, j+1:])
            disconcordant_table[i, j] = np.sum(contingency_table[:i, j+1:]) + np.sum(contingency_table[i+1:, :j])
    Concordant_pairs, Disconcordant_pairs = np.sum(concordant_table * contingency_table) / 2, np.sum(disconcordant_table * contingency_table) / 2
    sample_size = np.sum(contingency_table)
    number_of_total_pairs = sample_size * (sample_size-1)/2  
    sum_of_the_rows = contingency_table.sum(axis=1)
    sum_of_the_cols = contingency_table.sum(axis=0)
    Number_of_ties_rows = np.sum(sum_of_the_rows * (sum_of_the_rows-1)/2)
    Number_of_ties_columns = np.sum(sum_of_the_cols * (sum_of_the_cols-1)/2)
    low_rc = (min(number_of_rows, number_of_columns)) # choose whethehr there are less row or coloumns

    Kendall_tau_a = (Concordant_pairs - Disconcordant_pairs) / number_of_total_pairs
    Kendall_tau_b = (Concordant_pairs - Disconcordant_pairs) / np.sqrt((number_of_total_pairs-Number_of_ties_rows) * (number_of_total_pairs-Number_of_ties_columns))
    Kendall_tau_c = 2 * (Concordant_pairs - Disconcordant_pairs) / (sample_size**2 *  ((low_rc - 1) / low_rc)) 

    # Standard Errors of tau
    Term1 = (concordant_table - disconcordant_table) * (contingency_table !=0)
    Term2 = np.sum(Term1[Term1 != 0]) / sample_size
    Standard_Error_Tau_a = np.sqrt(2 / (sample_size * (sample_size - 1)) * ((2 * (sample_size - 2)) / (sample_size * (sample_size - 1)**2) * np.sum((Term1 - Term2)**2) + 1 - Kendall_tau_a**2))
    Proportions_Tables = contingency_table / sample_size
    Proportions_Difference = (concordant_table - disconcordant_table) / sample_size
    sum_of_proportions_rows = np.sum(Proportions_Tables, axis=1)
    sum_of_proportions_columns = np.sum(Proportions_Tables, axis=0)
    proportions_matrix_row = (np.tile(sum_of_proportions_rows, (contingency_table.shape[1], 1)).T) * (2 * (Concordant_pairs - Disconcordant_pairs) / sample_size**2)
    proportions_matrix_columns = np.tile(sum_of_proportions_columns, (contingency_table.shape[0], 1)) * (2 * (Concordant_pairs - Disconcordant_pairs) / sample_size**2)
    delta1 = np.sqrt(1 - np.sum(sum_of_proportions_rows**2))
    delta2 = np.sqrt(1 - np.sum(sum_of_proportions_columns**2))
    Term3 = (2 * Proportions_Difference + proportions_matrix_columns) * delta2 * delta1 + (proportions_matrix_row * delta2) / delta1
    Standard_Error_Tau_b = np.sqrt(((np.sum(Proportions_Tables * Term3**2) - np.sum(Proportions_Tables * Term3)**2) / (delta1 * delta2)**4) / sample_size)
    Standard_Error_Tau_c = np.sqrt(4 * low_rc**2/((low_rc - 1)**2 * sample_size**4) * (np.sum(contingency_table * (concordant_table - disconcordant_table)**2) - 4 * (Concordant_pairs - Disconcordant_pairs)**2/sample_size))

    # Approximate Confidence Intervals Kendall's Tau
    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)
    Confidence_Interval_Tau_a = (Kendall_tau_a - zcrit * Standard_Error_Tau_a,Kendall_tau_a + zcrit * Standard_Error_Tau_a)
    Confidence_Interval_Tau_b = (Kendall_tau_b - zcrit * Standard_Error_Tau_b,Kendall_tau_b + zcrit * Standard_Error_Tau_b)
    Confidence_Interval_Tau_c = (Kendall_tau_c - zcrit * Standard_Error_Tau_c,Kendall_tau_c + zcrit * Standard_Error_Tau_c)

    Z_Score_Tau_a = Kendall_tau_a / Standard_Error_Tau_a
    Z_Score_Tau_b = Kendall_tau_b / Standard_Error_Tau_b
    Z_Score_Tau_c = Kendall_tau_c / Standard_Error_Tau_c

    p_value_Tau_a = scipy.stats.t.sf((abs(Z_Score_Tau_a)), 100000) * 2
    p_value_Tau_b = scipy.stats.t.sf((abs(Z_Score_Tau_b)), 100000) * 2
    p_value_Tau_c = scipy.stats.t.sf((abs(Z_Score_Tau_c)), 100000) * 2

    results = {}

    results["Kendalls Tau-a:"]= Kendall_tau_a
    results["Kendalls Tau-a Standard Error"] = Standard_Error_Tau_a
    results["Kendalls Tau-a Z-score"] = Z_Score_Tau_a
    results["Kendalls Tau-a p-value"]= p_value_Tau_a
    results["Kendalls Tau-a Confidence Intervals"]= Confidence_Interval_Tau_a
    results["Kendalls Tau-b:"]= Kendall_tau_b
    results["Kendalls Tau-b Standard Error"] = Standard_Error_Tau_b
    results["Kendalls Tau-b Z-score"] = Z_Score_Tau_b
    results["Kendalls Tau-b p-value"]= p_value_Tau_b
    results["Kendalls Tau-b. Confidence Intervals"]= Confidence_Interval_Tau_b
    results["Kendalls Tau-c"]= Kendall_tau_c
    results["Kendalls Tau-c Standard Error"] = Standard_Error_Tau_c
    results["Kendalls Tau-c Z-score"] = Z_Score_Tau_c
    results["Kendalls Tau-c p-value"]= p_value_Tau_c
    results["Kendalls Tau-c Confidence Intervals"]= Confidence_Interval_Tau_c


    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

# 8. Rsquare Estimation
def Rsquare_Estimation(Rsquare, sample_size, Number_Of_Predictors):
    dftotal = sample_size-1
    df_residual = sample_size - Number_Of_Predictors - 1
    df = sample_size - Number_Of_Predictors

    # Adjusted R square
    term1 = (sample_size - 3) * (1 - Rsquare) / df_residual
    Smith = 1 - (sample_size / (df)) * (1 - Rsquare) # Smith, 1929
    Ezekiel = 1 - (dftotal / df_residual) * (1 - Rsquare) #Ezekiel, 1930
    Wherry = 1 - (dftotal / df) * (1 - Rsquare) #Wherry, 1931
    olkin_pratt = 1 - term1 * special.hyp2f1(1, 1, (df + 1) / 2, 1 - Rsquare) #Olkin & Pratt, 1958
    Cattin = 1 - term1 * ((1 + (2 * (1 - Rsquare)) / df_residual) + ((8 * (1 - Rsquare) ** 2) / (df_residual * (df + 3)))) # Cattin, 1980 (Approximation to Olkin and Pratt)
    Pratt = 1 - (((sample_size - 3) * (1 - Rsquare)) / df_residual) * (1 + (2 * (1 - Rsquare)) / (df - 2.3)) # Pratt, 1964
    Herzberg = 1 - (((sample_size - 3) * (1 - Rsquare)) / df_residual) * (1 + (2 * (1 - Rsquare)) / (df+1)) # Herzberg, 1969
    Claudy =   1 - (((sample_size - 4) * (1 - Rsquare)) / df_residual) * (1 + (2 * (1 - Rsquare)) / (df+1)) # Herzberg, 1978
    def Alf_Graf_MLE(Rsquare, sample_size, Number_Of_Predictors):     # Alf and Graf 2002, MLE
        return opt.minimize_scalar(lambda rho: (1 - rho) ** (sample_size / 2) * (special.hyp2f1(0.5 * sample_size, 0.5 * sample_size, 0.5 * Number_Of_Predictors, rho * Rsquare)) * -1,bounds=(0, 1),method='bounded').x
    AlfGraf = Alf_Graf_MLE(Rsquare, sample_size, Number_Of_Predictors)

    # Squared Cross-Validity Coefficient
    Lord = 1 - (sample_size+Number_Of_Predictors+1) / (sample_size-Number_Of_Predictors-1) * (1-Rsquare) #Uhl & Eisenberg, 1970, also known as the Lord formula (it is most cited by this name)
    Lord_Nicholson = 1 - ((sample_size+Number_Of_Predictors +1) / sample_size) * (dftotal/df_residual) * (1-Rsquare) 
    Darlington_Stein = 1 - ((sample_size + 1) / sample_size) * (dftotal/df_residual) * ((sample_size-2) / (df-2)) * (1-Rsquare)
    Burket = (sample_size*Rsquare - Number_Of_Predictors) / (np.sqrt(Rsquare)*df) 
    Brown_Large_Sample = (((df - 3) * Ezekiel**2 +     Ezekiel)     / ((sample_size-2*Number_Of_Predictors - 2) * Ezekiel + Number_Of_Predictors))
    Brown_small_Sample = (((df - 3) * olkin_pratt**2 + olkin_pratt) / ((sample_size-2*Number_Of_Predictors - 2) * olkin_pratt + Number_Of_Predictors))
    Rozeboom = 1 - ((sample_size+Number_Of_Predictors) / df) * (1 - Rsquare) #Rozeboom, 1978
    Rozeboom2_Large_Sample = Ezekiel * ((1 + (Number_Of_Predictors/ (df-2)) * ((1-Ezekiel)/Ezekiel))**-1) #Rozeboom, 1981 
    Rozeboom2_small_Sample = olkin_pratt * ((1 + (Number_Of_Predictors/ (df-2)) * ((1-olkin_pratt)/olkin_pratt))**-1) #Rozeboom, 1981
    Claudy1_Large_Sample = (2* Ezekiel - (Rsquare))
    Claudy1_Small_Sample = (2* olkin_pratt - (Rsquare))

    results = {
        "Smith (1928)": Smith,
        "Ezekiel (1930)": Ezekiel,
        "Wherry (1931)": Wherry,
        "Olkin & Pratt (1958)": olkin_pratt,
        "Olkin & Pratt, Pratt's Approximation (1964)": Pratt,
        "Olkin & Pratt, Herzberg's Approximation (1968)": Herzberg,
        "Olkin & Pratt, Claudy's Approximation (1978)": Claudy,
        "Olkin & Pratt, Cattin's Approximation (1980)": Cattin,
        "Alf and Graf MLE (2002)": AlfGraf,
        "Lord(1950)" 
        "Lord_Nicholson(1960)": Lord_Nicholson,
        "Darlington/Stein (1967/1960)": Darlington_Stein,
        "Burket (1964)": Burket,
        "Brown_Large_Samples (1975)": Brown_Large_Sample,
        "Brown_Small_Samples (1975)": Brown_small_Sample,
        "Rozeboom I (1978)": Rozeboom,
        "Rozeboom II Large Samples- (1978)": Rozeboom2_Large_Sample,
        "Rozeboom II Small Samples (1978)": Rozeboom2_small_Sample, 
        "Claudy-I, Large Samples (1978)": Claudy1_Large_Sample,
        "Claudy-I, Small Samples (1978)": Claudy1_Small_Sample,   
    }

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

# 9. Coefficeint Eta (Correlation Ratio)
def Eta_Correlation_Ratio(x,y,confidence_level):
    ss_between_x_independent = np.sum([(len(y[x == value]) * (np.mean(y[x == value]) - np.mean(y)) ** 2) for value in np.unique(x)])
    ss_total_x_indpednent = np.sum((y - np.mean(y)) ** 2)
    eta_x_independent = np.sqrt(ss_between_x_independent/ss_total_x_indpednent)
    ss_between_y_independent = np.sum([(len(x[y == value]) * (np.mean(x[y == value]) - np.mean(x)) ** 2) for value in np.unique(y)])
    ss_total_y_indpednent = np.sum((x - np.mean(x)) ** 2)
    eta_y_independent = np.sqrt(ss_between_y_independent/ss_total_y_indpednent)

    # Attenuated Corrected Eta by Metsamuronen (2023):
    x_sorted = x[np.argsort(x)]
    y_sorted = y[np.argsort(y)]
    ss_between_x_max = np.sum([(len(y_sorted[x_sorted == value]) * (np.mean(y_sorted[x_sorted == value]) - np.mean(y_sorted)) ** 2) for value in np.unique(x_sorted)])
    eta_max_x_indpendent = np.sqrt(ss_between_x_max/ss_total_x_indpednent)

    ss_between_y_max = np.sum([(len(x_sorted[y_sorted == value]) * (np.mean(x_sorted[y_sorted == value]) - np.mean(x_sorted)) ** 2) for value in np.unique(y_sorted)])
    eta_max_y_indpendent = np.sqrt(ss_between_y_max/ss_total_y_indpednent)

    Attenuated_Corrected_Eta_X_Independent = (eta_x_independent/eta_max_x_indpendent)
    Attenuated_Corrected_Eta_Y_Independent = (eta_y_independent/eta_max_y_indpendent)

    results = {
        "Eta (Variable X is Indpendent)": eta_x_independent,
        "Eta (Variable y is Indpendent)": eta_y_independent,
        "Attenuated Correct Eta (Variable X is Indpendent)": Attenuated_Corrected_Eta_X_Independent,
        "Attenuated Correct Eta (Variable y is Indpendent)": Attenuated_Corrected_Eta_Y_Independent
    }

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

# 10. Pearson Correlation
def pearson_correlation(x,y,confidence_level = 0.95):
    Pearson_Correlation, Pearson_pvalue =  scipy.stats.pearsonr(x,y)

    sample_size = len(x)
    Sample_1_Mean = np.mean(x)    
    Sample_2_Mean = np.mean(y)
    Sample_1_Standard_Deviation = np.std(x, ddof = 1)    
    Sample_2_Standard_Deviation = np.std(y, ddof = 1)
    Slope = (Sample_2_Standard_Deviation/Sample_1_Standard_Deviation) * Pearson_Correlation
    constant = -(Slope * Sample_1_Mean - Sample_2_Mean)
    Predicted_Values = Slope*np.array(x) + constant
    Sum_of_square_deviations_from_mean_y = np.sum((y - Sample_2_Mean)**2) ##Sstotal
    Sum_of_square_deviations_from_mean_x = np.sum((x - Sample_1_Mean)**2) ##Ssx
    Sum_of_residuals = np.sum((y-Predicted_Values)**2) #SSres
    sum_of_regression = Sum_of_square_deviations_from_mean_y - Sum_of_residuals #SSreg
    Standard_error_slope = np.sqrt((1/(sample_size-2)) * (Sum_of_residuals/Sum_of_square_deviations_from_mean_x)) 
    t_score_of_the_slope = Slope / Standard_error_slope
    Rsquare = (sum_of_regression/Sum_of_square_deviations_from_mean_y)
    Approximated_r = Pearson_Correlation + (Pearson_Correlation*(1-Rsquare)) / (2*(sample_size-3))

    # Approximated Standrd Errors - Gnambs, 2023
    Fisher_SE = (1-Rsquare) / np.sqrt(sample_size*(1+Rsquare))  #Fisher, 1896
    Fisher_Filon_SE = (1-Rsquare) / np.sqrt(sample_size)  #Fisher & Filon, 1898
    Soper_1_SE = ((1-Rsquare) / np.sqrt(sample_size-1))  # Soper, 1913, Large Sample
    Soper_2_SE = ((1-Rsquare)/np.sqrt(sample_size)) * ( 1 + (1+5.5*Rsquare) / (2*sample_size))  # Soper, 1913
    Soper_3_SE = ((1-Rsquare)/np.sqrt(sample_size-1)) * (1 + (11*Rsquare) / (4*(sample_size-1)))  # Soper, 1913
    Hoteling = ((1-Rsquare)/np.sqrt(sample_size-1)) * (1 + ((11*Rsquare) / (4*(sample_size-1))) + ((-192*Rsquare+479*Rsquare**2)/ (32*(sample_size-1)**2)))  # Hoteling, 1953
    Ghosh_SE = np.sqrt((1 - ((sample_size - 2) * (1 - Rsquare) / (sample_size - 1) * hyp2f1(1, 1, (sample_size + 1) / 2, Rsquare)) - ((2 / (sample_size - 1) * (math.gamma(sample_size / 2) / math.gamma((sample_size - 1) / 2))**2) * np.sqrt(Rsquare) * hyp2f1(0.5, 0.5, (sample_size + 1) / 2, Rsquare))**2))
    Hedges_SE = np.sqrt((Pearson_Correlation * scipy.special.hyp2f1(0.5, 0.5, (sample_size - 2) / 2, 1 - Rsquare))**2 - (1 - (sample_size - 3) * (1 - Rsquare) * scipy.special.hyp2f1(1, 1, sample_size / 2, 1 - Rsquare) / (sample_size - 2)))
    Bonett_SE = (1-Rsquare) / np.sqrt(sample_size - 3)
    Regression_SE = np.sqrt((1-Rsquare) / (sample_size - 2))
 
    # Confidence Intervals:
    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)
    tcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, sample_size-2)

    # 1. Fisher (1921)
    Zr = 0.5 * np.log((1+Pearson_Correlation) / (1-Pearson_Correlation)) # Fisher's Transformation
    Standard_Error_ZR =  1 / (np.sqrt(sample_size-3)) #Fisher (1921) Approximation of the Variance
    Z_Lower = Zr - zcrit * Standard_Error_ZR
    Z_Upper = Zr + zcrit * Standard_Error_ZR
    Pearosn_Fisher_CI_lower = (np.exp(2*Z_Lower) - 1) / (np.exp(2*Z_Lower) + 1)
    Pearosn_Fisher_CI_upper = (np.exp(2*Z_Upper) - 1) / (np.exp(2*Z_Upper) + 1)

    # 2. Olivoto et al., 2018 - Half Widthe Confidecne Interval
    Half_Width = (0.45304**abs(Pearson_Correlation)) * 2.25152 * (sample_size**-0.50089)
    Lower_Olivoto = Pearson_Correlation - Half_Width
    Upperr_Olivoto = Pearson_Correlation + Half_Width

    # 3. Olkin & Fin, 1995
    Standard_Error_R_Sqaure = np.sqrt((4 * Rsquare * (1- Rsquare)**2*sample_size**2) / ((sample_size**2 -1)*(sample_size+3)))# Olkin & Finn, 1995
    Pearson_CI_Olkin_Fin_Lower = Rsquare - zcrit * Standard_Error_R_Sqaure
    Pearson_CI_Olkin_Fin_Upper = Rsquare + zcrit * Standard_Error_R_Sqaure

    # Common Language Effect Sizes and more
    Common_Language_Effect_Size_Dunlap = np.arcsin(Pearson_Correlation) / math.pi + 0.5
    Counter_Null_EffectSize = np.sqrt(4*Rsquare/(1+3*Rsquare))    
    Absolute_R_Square = 1 - (sum_of_regression / (np.sum((y - np.median(y))**2))) # Bertsimas et al., 2008
 
    results = {}
    results["Pearson Correlation"] = (Pearson_Correlation)
    results["t score"] = round(t_score_of_the_slope, 4)
    results["Degrees of Freedom"] = round(sample_size-2, 4)
    results["Pearson Correlation P-value"] = round(Pearson_pvalue, 4)
    results["Standrd Error of the Slope"] = round(Standard_error_slope, 4)
    results["Constant"] = round(constant, 4)
    results["Slope"] = round(Slope, 4)

    results["Standard Error Fisher "] = round(Fisher_SE, 4)
    results["Standard Error Fisher & Filon"] = round(Fisher_Filon_SE, 4)
    results["Standard Error Soper-I"] = round(Soper_1_SE, 4)
    results["Standard Error Soper-II"] = round(Soper_2_SE, 4)
    results["Standard Error Soper-III"] = round(Soper_3_SE, 4)
    results["Standard Error Hoteling"] = round(Hoteling, 4)
    results["Standard Error Ghosh"] = round(Ghosh_SE, 4)
    results["Standard Error Hedges"] = round(Hedges_SE, 4)
    results["Standard Error Bonett"] = round(Bonett_SE, 4)    
    results["Standard Error From Regression"] = round(Regression_SE, 4)

    results["Fisher's Zr"] = (Zr)
    results["Standard Error of Zr"] = (Standard_Error_ZR)
    results["Confidence Intervals Fisher (1921)"] = f"({round(Pearosn_Fisher_CI_lower, 4)}, {round(Pearosn_Fisher_CI_upper, 4)})"
    results["Confidence Intervals Olivoto"] = f"({round(Lower_Olivoto, 4)}, {round(Upperr_Olivoto, 4)})"
    results["Confidence Intervals for R-square (Olkin & Fin)"] = f"({round(Pearson_CI_Olkin_Fin_Lower, 4)}, {round(Pearson_CI_Olkin_Fin_Upper, 4)})"


    results["Common Language Effect Size (Dunlap, 1994)"] = (Common_Language_Effect_Size_Dunlap)
    results["Approximated r (Hedges & Olkin, 1985)"] = (Approximated_r)
    results["Counter_Null_EffectSize"] = (Counter_Null_EffectSize)
    results["Absolute R Square"] = (Absolute_R_Square)


    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str
    
# 11. Robust Alternatives to Pearson
def Robust_Measures_Interval(x,y, confidence_level = 0.95): 
    
    # A. Percentage Bend Correlation
    n = len(x)
    Omega_Hat_X= sorted((abs(np.array(x) - np.median(x))))[math.floor((1 - 0.2**2) * n) - 1]
    psix = (x-np.array(np.median(x)))/Omega_Hat_X
    i1 = sum((x_i - np.median(x)) / Omega_Hat_X < -1 for x_i in x)
    i2 = sum((x_i - np.median(np.median(x))) / Omega_Hat_X > 1 for x_i in x)
    Sx = np.sum(np.where(np.logical_or(psix < -1, psix > 1), 0, x))
    Phi_x = (Omega_Hat_X * (i2 - i1) + Sx) / (n - i1 - i2)
    Ai = np.clip((x - Phi_x) / Omega_Hat_X, -1, 1)

    Omega_Hat_Y = sorted((abs(np.array(y) - np.median(y))))[math.floor((1 - 0.2**2) * n) - 1]
    psiy = (y - np.array(np.median(y))) / Omega_Hat_Y
    i1_y = sum((y_i - np.median(y)) / Omega_Hat_Y < -1 for y_i in y)
    i2_y = sum((y_i - np.median(np.median(y))) / Omega_Hat_Y > 1 for y_i in y)
    Sy = np.sum(np.where(np.logical_or(psiy < -1, psiy > 1), 0, y))
    Phi_y = (Omega_Hat_Y * (i2_y - i1_y) + Sy) / (n - i1_y - i2_y)
    Bi = np.clip((y - Phi_y) / Omega_Hat_Y, -1, 1)

    Percentage_Bend_Correlation = np.sum(Ai * Bi)/np.sqrt(np.sum(Ai**2) * np.sum(Bi**2))
    T_Statistic_PB = Percentage_Bend_Correlation * np.sqrt((n - 2) / (1 - Percentage_Bend_Correlation**2))
    P_value_PB  = 2 * (1 - stats.t.cdf(np.abs(T_Statistic_PB), n - 2))

    # B. Winsorized Correlation
    lower_items = int(np.floor(0.2 * n)) + 1
    upper_items = len(x) - lower_items + 1
    sorted_x = np.sort(x)
    sorted_y = np.sort(y)

    Winzorized_X = np.where((x <= sorted_x[lower_items - 1]) | (x >= sorted_x[upper_items - 1]),np.where(x <= sorted_x[lower_items - 1], sorted_x[lower_items - 1], sorted_x[upper_items - 1]),x)
    Winzorized_Y = np.where((y <= sorted_y[lower_items - 1]) | (y >= sorted_y[upper_items - 1]),np.where(y <= sorted_y[lower_items - 1], sorted_y[lower_items - 1], sorted_y[upper_items - 1]),y)
    Winzorized_Correlation = np.corrcoef(Winzorized_X, Winzorized_Y)[0, 1]
    T_Statistic_WC = Winzorized_Correlation * np.sqrt((n - 2) / (1 - Winzorized_Correlation**2))
    degrees_of_freedom_WC = n - 2 * int(np.floor(0.2 * n)) - 2
    P_Value_Winzoried = 2 * (1 - t.cdf(np.abs(T_Statistic_WC), df=degrees_of_freedom_WC))

    # C. Biweight Midcorrelation (Verified with asbio for r)
    Ui = (x - np.median(x)) / (9 * t.ppf(0.75, 100000) * 1.4826*np.median(np.abs(x- np.median(x))))
    Vi = (y - np.median(y)) / (9 * t.ppf(0.75, 100000) * 1.4826*np.median(np.abs(y - np.median(y))))
    Ai = np.where((Ui <= -1) | (Ui >= 1), 0, 1)
    Bi = np.where((Vi <= -1) | (Vi >= 1), 0, 1)

    Sxx = (np.sqrt(n) * np.sqrt(sum((Ai * ((x - np.median(x))**2)) * ((1 - Ui**2)**4))) /  abs(sum(Ai * (1 - Ui**2) * (1 - 5 * Ui**2))))**2
    Syy = (np.sqrt(n) * np.sqrt(sum((Bi * ((y - np.median(y))**2)) * ((1 - Vi**2)**4))) / (abs(sum(Bi * (1 - Vi**2) * (1 - 5 * Vi**2)))))**2
    Sxy = n * sum((Ai *(x - np.median(x))) * ((1-Ui**2)**2) * (Bi* (y-np.median(y))) * ((1-Vi**2)**2)) / (sum((Ai* (1-Ui**2)) * (1-5*Ui**2)) * sum((Bi* (1-Vi**2)) * (1 - 5 * Vi**2)))
    Biweight_midcorrelation = Sxy / (np.sqrt(Sxx * Syy))
    T_statistic_BM = np.sqrt(n - 2) * np.abs(Biweight_midcorrelation) / np.sqrt(1 - Biweight_midcorrelation**2)
    degrees_of_freedom_bm = n-2
    P_value_BM = 2 * (1 - t.cdf(T_statistic_BM, degrees_of_freedom_bm))

    # D. Gausian Rank Correlation 
    Normalized_X = norm.ppf((np.argsort(x) + 1) / (len(x) + 1))
    Normalized_Y = norm.ppf((np.argsort(y) + 1) / (len(y) + 1))
    Gaussian_Rank_Correlation = pearsonr(Normalized_X, Normalized_Y)
    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)
    Normalized_X = norm.ppf((rankdata(x) / (len(x) + 1)))
    Normalized_Y = norm.ppf((rankdata(y) / (len(y) + 1)))
    Gaussian_Rank_Correlation, GRS_pvalue = pearsonr(Normalized_X, Normalized_Y)
    Zr = 0.5 * np.log((1+Gaussian_Rank_Correlation) / (1-Gaussian_Rank_Correlation)) # Fisher's Transformation
    Standard_Error_ZR =  1 / (np.sqrt(n-3)) #Fisher (1921) Approximation of the Variance
    Z_Lower = Zr - zcrit * Standard_Error_ZR
    Z_Upper = Zr + zcrit * Standard_Error_ZR
    Gaussian_Rank_Correlation_Fisher_CI_lower = (np.exp(2*Z_Lower) - 1) / (np.exp(2*Z_Lower) + 1)
    Gaussian_Rank_Correlation_Fisher_CI_upper = (np.exp(2*Z_Upper) - 1) / (np.exp(2*Z_Upper) + 1)

    results = {
    "Percentage Bend Correlation": Percentage_Bend_Correlation,
    "Percentage Bend Statistic": T_Statistic_PB,
    "Percentage Bend p-value)": P_value_PB,
    "Winsorized Correlation": Winzorized_Correlation}

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

# 12. Point Biserial Correlation
# For equal and unequal sample sizes

# 13. Skipped Correalation (from Pinguine)
def skipped_Correlation(x, y, corr_type="spearman"):
    from pingouin.utils import _is_sklearn_installed

    _is_sklearn_installed(raise_error=True)
    from scipy.stats import chi2
    from sklearn.covariance import MinCovDet

    X = np.column_stack((x, y))
    nrows, ncols = X.shape
    gval = np.sqrt(chi2.ppf(0.975, 2))
    center = MinCovDet(random_state=42).fit(X).location_
    B = X - center
    bot = (B**2).sum(axis=1)
    dis = np.zeros(shape=(nrows, nrows))
    for i in np.arange(nrows):
        if bot[i] != 0:  # Avoid division by zero error
            dis[i, :] = np.linalg.norm(B.dot(B[i, :, None]) * B[i, :] / bot[i], axis=1)

    def idealf(x):
        n = len(x)
        j = int(np.floor(n / 4 + 5 / 12))
        y = np.sort(x)
        g = (n / 4) - j + (5 / 12)
        low = (1 - g) * y[j - 1] + g * y[j]
        k = n - j + 1
        up = (1 - g) * y[k - 1] + g * y[k - 2]
        return up - low

    # One can either use the MAD or the IQR (see Wilcox 2012)
    # MAD = mad(dis, axis=1)
    iqr = np.apply_along_axis(idealf, 1, dis)
    thresh = np.median(dis, axis=1) + gval * iqr
    outliers = np.apply_along_axis(np.greater, 0, dis, thresh).any(axis=0)
    # Compute correlation on remaining data
    if corr_type == "spearman":
        r, pval = spearmanr(X[~outliers, 0], X[~outliers, 1])
    else:
        r, pval = pearsonr(X[~outliers, 0], X[~outliers, 1])
    return r, pval

# 14. Robust Measures Ordinal correlation
def Robust_Measures_Ordinal(x,y, confidence_level = 0.95): 
    # A. Sheperds Pi
    def bsmahal(a, b, n_boot=200):
        n, m = b.shape
        MD = np.zeros((n, n_boot))
        nr = np.arange(n)
        xB = np.random.choice(nr, size=(n_boot, n), replace=True)
        for i in np.arange(n_boot):
            s1 = b[xB[i, :], 0]
            s2 = b[xB[i, :], 1]
            X = np.column_stack((s1, s2))
            mu = X.mean(0)
            _, R = np.linalg.qr(X - mu)
            sol = np.linalg.solve(R.T, (a - mu).T)
            MD[:, i] = np.sum(sol**2, 0) * (n - 1)
        return MD.mean(1)

    def shepherd(x, y, n_boot=200):
        X = np.column_stack((x, y))
        m = bsmahal(X, X, n_boot)
        outliers = m >= 6
        r, pval = spearmanr(x[~outliers], y[~outliers])
        return r, pval
    
    #B. Gassuan Rank Correlation
    Normalized_X = norm.ppf((np.argsort(x) + 1) / (len(x) + 1))
    Normalized_Y = norm.ppf((np.argsort(y) + 1) / (len(y) + 1))
    Gaussian_Rank_Correlation = pearsonr(Normalized_X, Normalized_Y)
    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)
    Normalized_X = norm.ppf((rankdata(x) / (len(x) + 1)))
    Normalized_Y = norm.ppf((rankdata(y) / (len(y) + 1)))
    Gaussian_Rank_Correlation, GRC_pvalue = spearmanr(Normalized_X, Normalized_Y)


# 15. Spearman Correlation
def Spearman_Correlation(x,y,confidence_level): 

    spearman_rho_p_value = spearmanr(x,y)[1]

    ranked_x = rankdata(x)
    ranked_y = rankdata(y)

    mean_sample_1 = np.mean(ranked_x)
    mean_sample_2 = np.mean(ranked_y)
    sample_size = len(x)

    Spearman = (np.sum ((ranked_x- mean_sample_1) * (ranked_y - mean_sample_2))) / (np.sqrt(np.sum((ranked_x- mean_sample_1)**2)) * np.sqrt(np.sum((ranked_y- mean_sample_2)**2)))
    fisher_Zrho = math.atanh(Spearman)

    Bonett_Wright_Standard_Error = np.sqrt((1 + Spearman**2 / 2) / (sample_size - 3)) # 2000
    Fieller_Standard_Error = 1.06 / (sample_size-3) # 1957
    Caruso_and_Cliff = (1 / (sample_size - 2)) +  (abs(fisher_Zrho)/(6*sample_size + 4 *np.sqrt(sample_size)))# 1997

    # Confidence Interval
    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)

    Lower_ci_BW = math.tanh(fisher_Zrho - zcrit * Bonett_Wright_Standard_Error)
    Upper_ci_BW = math.tanh(fisher_Zrho + zcrit * Bonett_Wright_Standard_Error)

    Lower_ci_Fieller = math.tanh(fisher_Zrho - zcrit * Fieller_Standard_Error)
    Upper_ci_Fieller = math.tanh(fisher_Zrho + zcrit * Fieller_Standard_Error)

    Lower_ci_CC = math.tanh(fisher_Zrho - zcrit * Caruso_and_Cliff)
    Upper_ci_CC = math.tanh(fisher_Zrho + zcrit * Caruso_and_Cliff)

    # Spearman Corrected for ties Oyeka and Nwankwo Chike 2014 
    Expected_ranks_x = x.argsort().argsort() + 1
    Expected_ranks_y = y.argsort().argsort() + 1
    Difference_Rank_x = Expected_ranks_x - ranked_x
    Difference_Rank_y = Expected_ranks_y - ranked_y
    Product_x = Expected_ranks_x * Difference_Rank_x
    Product_y = Expected_ranks_y * Difference_Rank_y
    Di_x = Difference_Rank_x**2 /2
    Di_y = Difference_Rank_y**2 /2
    pi_x = non_zero_count = sum(1 for element in Difference_Rank_x if element != 0) / sample_size
    pi_y = non_zero_count = sum(1 for element in Difference_Rank_y if element != 0) / sample_size
    Multi_Ranked = ranked_x * ranked_y
    term_x = (((sample_size*(sample_size**2-1))/12) - (2*pi_x * (sum(Product_x)-sum(Di_x))))
    term_y = (((sample_size*(sample_size**2-1))/12) - (2*pi_y * (sum(Product_y)-sum(Di_y))))
    Numerator = np.sum(Multi_Ranked) - ((sample_size*(sample_size+1)**2) / 4)  
    Denomirator = np.sqrt(term_x*term_y)
    Corrected_Spearman = Numerator / Denomirator

    # Spearman Corrected for ties Taylor (1964)
    d_square = np.sum((ranked_x - ranked_y)**2)
    frequency_vector_x = np.array([count for count in Counter(x).values() if count > 1]) # return only repeating frequencies for ties correction 
    frequency_vector_y = np.array([count for count in Counter(y).values() if count > 1]) # return only repeating frequencies for ties correction 
    Tx = sum(frequency_vector_x**3-frequency_vector_x) / 12
    Ty = sum(frequency_vector_y**3-frequency_vector_y) / 12
    Spearman_Corrected_Taylor = 1 - (6 * (np.sum(d_square)+Tx+Ty)) / (sample_size*(sample_size**2-1))


    results = {}
    results["Spearman"] = Spearman
    results["Spearman p-value"] = spearman_rho_p_value
    results["Ties Corrected Spearman (Oyeka and Nwankwo Chike, 2014) "] = Corrected_Spearman
    results["Ties Corrected Spearman (Taylor, 1964)"] = Spearman_Corrected_Taylor

    results["Standard Error (Bonett & Wright)"] = Bonett_Wright_Standard_Error
    results["Standard Error (Fieller)"] = Fieller_Standard_Error
    results["Standard Error (Caruso and_Cliff)"] = Caruso_and_Cliff

    results["Confidence Intervals (Bonett Wright)"] = f"({round(Lower_ci_BW, 4)}, {round(Upper_ci_BW, 4)})"
    results["Confidence Intervals (Fieller)"] = f"({round(Lower_ci_Fieller, 4)}, {round(Upper_ci_Fieller, 4)})"
    results["Confidence Intervals (and_Cliff)"] = f"({round(Lower_ci_CC, 4)}, {round(Upper_ci_CC, 4)})"

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

# 16. Blomqvist_beta
def Blomqvist_Beta(x,y,confidence_level):
    sample_size = len(x)
    Sorted_X_Values = rankdata(x) / (len(x) + 1)
    Sorted_Y_Values = rankdata(y) / (len(y) + 1)
    Sorted_Data = np.column_stack((Sorted_X_Values, Sorted_Y_Values))
    Blomqvist_Beta = 2 * np.mean(np.all(Sorted_Data <= 0.5, axis=1) + np.all(Sorted_Data > 0.5, axis=1)) - 1
    Statistic = math.atanh(Blomqvist_Beta) * np.sqrt(len(x))
    p_value = 2 * norm.cdf(-abs(Statistic))
    
    results = {
    "Blomqvist_Beta": Blomqvist_Beta,
    "Statistic": Statistic,
    "p_value": p_value}

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

# 17. Gini's Gama
def ginis_gamma(x, y, confidence_level=0.95):
    ranked_x = rankdata(x)
    ranked_y = rankdata(y)
    sample_size = len(x)

    term1 = np.sum(np.abs((sample_size+1 - ranked_x) - ranked_y) - np.abs(ranked_x - ranked_y))
    zcrit = t.ppf(1 - (1 - confidence_level) / 2, 100000)

    if sample_size % 2 == 0:
        index1 = 1 / (sample_size**2 / 2)
        gamma = term1 * index1
        ASE = np.sqrt((2 * (sample_size**2 + 2)) / (3 * (sample_size - 1) * (sample_size**2)))
        Z = gamma / ASE
        p_value = 2 * (1 - t.cdf(np.abs(Z), df=100000))  # Two-tailed test
        result_ASE = ASE
        lower_ci = gamma - ASE*zcrit
        upper_ci = gamma + ASE*zcrit
    else:
        index2 = 1 / ((sample_size**2 - 1) / 2)
        gamma = term1 * index2
        ASE = np.sqrt((2 * (sample_size**2 + 3)) / (3 * (sample_size - 1) * (sample_size**2 - 1)))
        Z = gamma / ASE
        p_value = 2 * (1 - t.cdf(np.abs(Z), df=100000))  
        result_ASE = ASE
        lower_ci = gamma - ASE*zcrit
        upper_ci = gamma + ASE*zcrit

    return {
        'Ginis Gamma': gamma,
        'Standard Error': result_ASE,
        'lower_ci': max(lower_ci, -1),  
        'upper_ci': min(upper_ci, 1),   
        'p_value': p_value
    }


# 18. Polychoric Correlation

# 19. Distance Correlation 
     
# 20. Wilson e


# Extra Hoffedings D   
# Lin's concordance correlation coefficient


########################################################
########################################################

# Functions to measure the levels of agreement

# 1. Cohens Kappa

def calculate_p_value_from_z_score(score):
    p_value = st.t.sf((abs(score)), 100000) * 2
    return min(float(p_value), 0.99999)

#Cohens Kappa fpr two raters
contingency_table = np.array([[106,10,4],[22,28,10],[2,12,6]])
#contingency_table = np.array([[4,4,2],[2,6,0],[0,2,0]])
contingency_table = np.array([[10,4,1],[6,16,2],[0,3,8]])
contingency_table2 = np.array([[20,5],[10,15]])

confidence_level = 0.95

def Cohens_Kappa_2_raters(contingency_table, confidence_level):
    Sample_Size = np.sum(contingency_table)
    Diagonal_of_Agreement = np.diag(contingency_table)
    Sum_of_coloumns = np.sum(contingency_table, axis=0) / Sample_Size
    Sum_of_rows = np.sum(contingency_table, axis=1) / Sample_Size
    Disagreement_Table = np.outer(Sum_of_coloumns, Sum_of_rows)
    Percentages_of_DissAgreement = np.trace(Disagreement_Table)
    Percentages_of_Agreement = np.sum((Diagonal_of_Agreement)/ Sample_Size)
    Cohens_Kappa = (Percentages_of_Agreement - Percentages_of_DissAgreement) / (1 - Percentages_of_DissAgreement)

    # Calculate the Variance (Fleiss, Cohen & Everrit, 1969)
    number_of_levels = contingency_table.shape[1]
    Agreement_Matrix = np.eye(contingency_table.shape[1])
    Probabilty_Matrix = (contingency_table/Sample_Size)
    Probability_Coloumns = np.repeat(np.sum(Probabilty_Matrix, axis=1) * (1-Cohens_Kappa), repeats=number_of_levels)
    Probability_Rows = np.repeat((np.sum(Probabilty_Matrix, axis=0) * (1 - Cohens_Kappa))[np.newaxis, :], repeats=number_of_levels, axis=0)
    Final_Probabilities = np.transpose(Probabilty_Matrix).flatten() * (Agreement_Matrix.flatten() -  Probability_Coloumns - Probability_Rows.flatten())**2
    Standart_Error_Kappa_Fleisse = np.sqrt((np.sum(Final_Probabilities) - (Cohens_Kappa - Percentages_of_DissAgreement * (1 - Cohens_Kappa))**2) / (1-Percentages_of_DissAgreement)**2 / Sample_Size)

    # Calculate the Variance for H0 (Fleiss, Cohen & Everrit, 1969)
    outer_mulitiplication_matrix = np.transpose(np.outer(Sum_of_rows, Sum_of_coloumns))
    outer_addition_matrix = np.add.outer(Sum_of_rows, Sum_of_coloumns)
    Variance_matrix = outer_mulitiplication_matrix * (Agreement_Matrix - outer_addition_matrix)**2
    Standard_Error_H0 = np.sqrt((np.sum(Variance_matrix) - Percentages_of_DissAgreement**2) / (Sample_Size * (1-Percentages_of_DissAgreement)**2))

    # Signficance
    StatisticH0 = Cohens_Kappa / Standard_Error_H0
    p_valueH0 = calculate_p_value_from_z_score(StatisticH0)
    Statistic_Fleiss = Cohens_Kappa / Standart_Error_Kappa_Fleisse
    p_value_Fleiss = calculate_p_value_from_z_score(Statistic_Fleiss)

    # Confidence Interval
    zcrit = st.t.ppf(1 - (1 - confidence_level) / 2, 100000)
    Lower_Confidence_Interval_Kappa = Cohens_Kappa - Standart_Error_Kappa_Fleisse*zcrit
    Upper_Confidence_Interval_Kappa = Cohens_Kappa + Standart_Error_Kappa_Fleisse*zcrit
    Lower_Confidence_Interval_Kappa_H0 = Cohens_Kappa - Standard_Error_H0*zcrit
    Upper_Confidence_Interval_Kappa_H0 = Cohens_Kappa + Standard_Error_H0*zcrit


    results = {}

    results["Cohens Kappa"]= Cohens_Kappa
    results["p_value"] = p_valueH0
    results["Statistic"]= StatisticH0
    results["Standard_Error_H0"]= Standard_Error_H0
    results["Standart_Error_Fleisse, Cohen & Everett"] = Standart_Error_Kappa_Fleisse
    results["p_value (Fleisse, Cohen & Everett)"] = p_value_Fleiss
    results["Statistic (Fleisse, Cohen & Everett)"]= Statistic_Fleiss
    results["Confidence Intervals Kappa"] = f"({round(Lower_Confidence_Interval_Kappa, 4)}, {round(Upper_Confidence_Interval_Kappa, 4)})"
    results["Confidence Intervals Kappa_H0"] = f"({round(Lower_Confidence_Interval_Kappa_H0, 4)}, {round(Upper_Confidence_Interval_Kappa_H0, 4)})"

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str


# 2. Scott's Agreement Index...# Note that we used the CI and significance with Z.  irrCAC (GPL) used the t distribution while we used the Z distribution as most packages do
def scotts_pi(Contingency_Table, confidence_level):
    Sample_Size = np.sum(Contingency_Table)
    Number_Of_Levels = Contingency_Table.shape[1]

    Weights = np.eye(Number_Of_Levels)
    Percentages_of_Agreement = np.sum(Weights * Contingency_Table / Sample_Size)
    Sum_of_Coloumns = np.sum(Contingency_Table, axis=0) / Sample_Size
    Sum_of_Rows = np.transpose(np.sum(Contingency_Table, axis=1) / Sample_Size)
    Joint_Distribution = (Sum_of_Coloumns + Sum_of_Rows) / 2
    Percentage_Of_Disagreement = np.sum(Weights * np.outer(Joint_Distribution, Joint_Distribution))
    Scotts_Pi = (Percentages_of_Agreement - Percentage_Of_Disagreement) / (1 - Percentage_Of_Disagreement)

    # Calcaulate the Varaince
    Probability_Table = Contingency_Table / Sample_Size
    Probability_Rows = np.dot(Weights, Sum_of_Rows)
    Probability_Coloumns = np.dot(np.transpose(Weights), Sum_of_Coloumns)
    Joint_Probabilities = (Probability_Rows + Probability_Coloumns) / 2
    Term1 = sum(Probability_Table[k, l] * (Weights[k, l] - (1 - Scotts_Pi) * (Joint_Probabilities[k] + Joint_Probabilities[l]))**2 for k in range(Number_Of_Levels) for l in range(Number_Of_Levels))
    Standart_Error_Scotts_Pi = np.sqrt((1 / (Sample_Size * (1 - Percentage_Of_Disagreement)**2)) * (Term1 - (Percentages_of_Agreement - 2 * (1 - Scotts_Pi) * Percentage_Of_Disagreement)**2))

    # Significance and confidence Intervals
    Statistic = Scotts_Pi / Standart_Error_Scotts_Pi
    p_value = calculate_p_value_from_z_score(Statistic)

    # Confidence Interval
    zcrit = st.t.ppf(1 - (1 - confidence_level) / 2, 100000)
    Lower_Confidence_Interval_Scotts_Pi = Scotts_Pi - Standart_Error_Scotts_Pi*zcrit
    Upper_Confidence_Interval_Kappa_Scotts_Pi = Scotts_Pi + Standart_Error_Scotts_Pi*zcrit

    results = {}

    results["Scott's Phi"]= Scotts_Pi
    results["Z-Statistic"]= Statistic
    results["p_value"] = p_value
    results["Standart Error Scott's Phi"] = Standart_Error_Scotts_Pi
    results["Confidence Intervals Kappa"] = f"({round(Lower_Confidence_Interval_Scotts_Pi, 4)}, {round(Upper_Confidence_Interval_Kappa_Scotts_Pi, 4)})"

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return results


# 3. Krippendorff alpha

# 4. Spearman Foot rule (Check Berry)


# 5. Bhapkar
def BhapkarTest(contingency_table):
    sample_size = np.sum(contingency_table)
    Sum_Of_The_Rows = np.sum(contingency_table, axis=1)[:-1]
    Sums_Of_The_Coloumns = np.sum(contingency_table, axis=0)[:-1]
    Matrix_of_d_values = np.array([Sum_Of_The_Rows - Sums_Of_The_Coloumns] * len(Sums_Of_The_Coloumns)).T  # Adjusted here
    Agreement_Matrix = np.zeros((contingency_table.shape[0] - 1, contingency_table.shape[1] - 1))
    np.fill_diagonal(Agreement_Matrix, Sum_Of_The_Rows + Sums_Of_The_Coloumns)
    Weights = contingency_table[:-1, :-1]
    Variance_Covaraince_Matrix = Agreement_Matrix - Weights.T - Weights - (Matrix_of_d_values * Matrix_of_d_values.T) / sample_size  # Adjusted here
    Inverse_Variance_Matrix = np.linalg.inv(Variance_Covaraince_Matrix)
    Chisquare_Value = abs((Matrix_of_d_values @ Matrix_of_d_values.T @ Inverse_Variance_Matrix)[0,0])
    Degrees_Of_Freedom = contingency_table.shape[0] - 1
    p_value = 1 - chi2.cdf(Chisquare_Value, Degrees_Of_Freedom)


    results = {}
    results["Chisquare_Value"]= Chisquare_Value
    results["Degrees_Of_Freedom"]= Degrees_Of_Freedom
    results["p_value"] = p_value

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return results

# 6. Fleiss Kappa


# 7. Weighted Kappa
def Cohens_Kappa_2_raters(contingency_table, confidence_level, weights_type='equal-spacing'):
    number_of_levels = contingency_table.shape[1]
    Sample_Size = np.sum(contingency_table)
    Sum_of_coloumns = np.sum(contingency_table, axis=0) / Sample_Size
    Sum_of_rows = np.sum(contingency_table, axis=1) / Sample_Size
    if weights_type == 'equal-spacing':
        Weights_Matrix = 1 - abs(np.subtract.outer(np.arange(1, number_of_levels+1), np.arange(1, number_of_levels+1))) / (number_of_levels-1)
    elif weights_type == 'fleiss':
        Weights_Matrix = 1 - (abs(np.subtract.outer(np.arange(1, number_of_levels+1), np.arange(1, number_of_levels+1))) / (number_of_levels-1))**2


    Percentages_of_Agreement = np.sum(Weights_Matrix * contingency_table)/ Sample_Size
    Percentages_Of_Disagreement = np.sum(Weights_Matrix * np.outer(Sum_of_rows, Sum_of_coloumns))
    Cohens_Kappa = (Percentages_of_Agreement - Percentages_Of_Disagreement) / (1 - Percentages_Of_Disagreement)

    # Calculate the Variance (Fleiss, Cohen & Everrit, 1969)
    Agreement_Matrix = np.eye(contingency_table.shape[1])
    Probabilty_Matrix = (contingency_table)/Sample_Size
    Variance_Matrix = np.subtract((np.subtract(Weights_Matrix.T, (np.dot(Weights_Matrix, np.sum(Probabilty_Matrix, axis=1)) * (1 - Cohens_Kappa))).T), np.dot(Weights_Matrix, np.sum(Probabilty_Matrix, axis=0)) * (1 - Cohens_Kappa)).T
    Standart_Error_Kappa_Fleisse = np.sqrt((np.sum(Probabilty_Matrix * Variance_Matrix**2) - (Cohens_Kappa - Percentages_Of_Disagreement * (1 - Cohens_Kappa))**2) / np.inner(1 - Percentages_Of_Disagreement, 1 - Percentages_Of_Disagreement) / Sample_Size)
    
    # Calculate the Variance of H0
    Outer_Probability_Matrix = np.multiply.outer(Sum_of_coloumns, Sum_of_rows)
    Sum_of_rows = np.sum(np.sum(Probabilty_Matrix,axis = 0) * Weights_Matrix,axis = 1)
    Sum_of_coloumns = np.sum(np.sum(Probabilty_Matrix,axis = 1) * Weights_Matrix,axis = 1)
    Weighted_Variance_Matrix = (Weights_Matrix - np.add.outer(Sum_of_rows,Sum_of_coloumns))**2
    term = np.sum(Outer_Probability_Matrix * Weighted_Variance_Matrix)
    Standard_Error_H0 = np.sqrt((term - Percentages_Of_Disagreement**2) / (Sample_Size* (1-Percentages_Of_Disagreement)**2))

    # Signficance
    StatisticH0 = Cohens_Kappa / Standart_Error_Kappa_Fleisse
    p_valueH0 = calculate_p_value_from_z_score(StatisticH0)
    Statistic_Fleiss = Cohens_Kappa / Standard_Error_H0
    p_value_Fleiss = calculate_p_value_from_z_score(Statistic_Fleiss)

    # Confidence Interval
    zcrit = st.t.ppf(1 - (1 - confidence_level) / 2, 100000)
    Lower_Confidence_Interval_Kappa = Cohens_Kappa - Standart_Error_Kappa_Fleisse*zcrit
    Upper_Confidence_Interval_Kappa = Cohens_Kappa + Standart_Error_Kappa_Fleisse*zcrit
    Lower_Confidence_Interval_Kappa_H0 = Cohens_Kappa - Standard_Error_H0*zcrit
    Upper_Confidence_Interval_Kappa_H0 = Cohens_Kappa + Standard_Error_H0*zcrit

    results = {}

    results["Cohens Kappa"]= Cohens_Kappa
    results["p_value"] = p_valueH0
    results["Statistic"]= StatisticH0
    results["Standard_Error_H0"]= Standard_Error_H0
    results["Standart_Error_Fleisse, Cohen & Everett"] = Standart_Error_Kappa_Fleisse
    results["p_value (Fleisse, Cohen & Everett)"] = p_value_Fleiss
    results["Statistic (Fleisse, Cohen & Everett)"]= Statistic_Fleiss
    results["Confidence Intervals Kappa"] = f"({round(Lower_Confidence_Interval_Kappa, 4)}, {round(Upper_Confidence_Interval_Kappa, 4)})"
    results["Confidence Intervals Kappa_H0"] = f"({round(Lower_Confidence_Interval_Kappa_H0, 4)}, {round(Upper_Confidence_Interval_Kappa_H0, 4)})"

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str


# 8. Bennette Alpert and Goldstein S

# 9. Brennan-Prediger

# 10. More Binary Measures

# 11. Aickin's Alpha

# 12. Kendall's W

# 13. Gwet's Family

# 14. ICC

# 16. Lin's CCC


####################################################
##### Relevant functions for proportion effect sizes
####################################################

def one_sample_proportion_effect_size(sample_size, proportion_sample, population_proportion = 0.5, confidence_level = 0.95):
    cohens_g = abs(proportion_sample - population_proportion)
    phi_sample = 2 *(np.arcsin(np.sqrt(proportion_sample)))
    phi_population = 2 *(np.arcsin(np.sqrt(population_proportion)))
    cohens_h = phi_sample - phi_population
    Standard_Error_Wald = np.sqrt((proportion_sample * (1 - proportion_sample)) / sample_size)
    Standard_Error_score = np.sqrt((population_proportion * (1 - population_proportion)) / sample_size)
    z_score_wald = (proportion_sample - population_proportion) / Standard_Error_Wald

    Number_of_Succeses_sample = proportion_sample*sample_size
    Number_Of_Failures_sample = sample_size - Number_of_Succeses_sample
    Number_of_Succeses_Population = population_proportion*sample_size
    Number_Of_Failures_Population = sample_size - Number_of_Succeses_Population

    z_score = (proportion_sample - population_proportion) / Standard_Error_score
    correction = sample_size * population_proportion + 0.5
    Z_score_wald_corrected = ((proportion_sample * sample_size) - correction) / np.sqrt((proportion_sample * (1 - proportion_sample) * sample_size))
    Z_score_corrected = ((proportion_sample * sample_size) - correction) / np.sqrt((population_proportion * (1 - population_proportion) * sample_size))

    p_value_wald = calculate_p_value_from_z_score(z_score_wald)
    p_value_wald_corrected = calculate_p_value_from_z_score(Z_score_wald_corrected)
    p_value_score = calculate_p_value_from_z_score(z_score)
    p_value_score_corrected = calculate_p_value_from_z_score(Z_score_corrected)

    # Risk Measures
    Relative_Risk = proportion_sample/population_proportion
    Odds_Ratio = (proportion_sample/1-proportion_sample) / (population_proportion/1-population_proportion)


    results = {}
    results["Sample's Proportion"] = proportion_sample
    results["Population's Proportion"] = population_proportion
    results["Sample Size"] = sample_size
    results["Confidence Level"] = confidence_level
    results["Number of Successes (Sample)"] = Number_of_Succeses_sample
    results["Number of Failures (Sample)"] = Number_Of_Failures_sample
    results["Expected Number of Successes (Population)"] = Number_of_Succeses_Population
    results["Expected Number of Failures (Population)"] = Number_Of_Failures_Population
    results["Standard Error (Wald)"] = round(Standard_Error_Wald, 4)
    results["Standard Error (Score)"] = round(Standard_Error_score, 4)
    results["Z-score (Wald)"] = round(z_score_wald, 4)
    results["Z-score (Score)"] = round(z_score, 4)
    results["Z-score (Wald Corrected)"] = round(Z_score_wald_corrected, 4)
    results["Z-score (Score Corrected)"] = round(Z_score_corrected, 4)
    results["P-value (Wald)"] = round(p_value_wald, 4)
    results["P-value (Wald) Corrected"] = round(p_value_wald_corrected, 4)
    results["P-value (Score)"] = round(p_value_score, 4)
    results["P-value (Score) Corrected"] = round(p_value_score_corrected, 4)
    results["Cohen's g"] = round(cohens_g, 4)
    results["cohens_h"] = round(cohens_h, 4)
    results["Relative Risk"] = round(Relative_Risk, 4)
    results["Odds_Ratio"] = round(Odds_Ratio, 4)



    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

def proportion_of_hits(number_correct,number_of_trials, Number_of_Choices, confidence_level = 0.95): 
    Proportion_Correct = number_correct/number_of_trials
    pi = (Proportion_Correct*(Number_of_Choices-1)) / (1+Proportion_Correct*(Number_of_Choices-2))
    Standard_Error_pi = (1/np.sqrt(number_of_trials)) * (  (pi*(1-pi)) / (np.sqrt(Proportion_Correct*(1-Proportion_Correct))) ) 
    Z_score = (pi-0.5)/Standard_Error_pi
    p_value = calculate_p_value_from_z_score(Z_score)
    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)
    lower_CI = pi - zcrit * Standard_Error_pi
    upper_CI = pi + zcrit * Standard_Error_pi
    
    results = {}
    
    results["Proportion of Hits [π]"] = round(pi, 7)
    results["Standard Error of π"] = round(Standard_Error_pi,7)
    results["Z-score"] = round(Z_score, 7)
    results["p vlaue of π"] = round(p_value,7)
    results["Confidence Intervals for π"] = f"({round(lower_CI, 4)}, {round(upper_CI, 4)})"

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

def one_sample_proportion_conf_int(p1, sample_size, confidence_level):
    alpha = 1 - confidence_level
    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)

    # 1. Agresti-Coull
    AC_CI = proportion_confint(p1, sample_size, alpha, method = "agresti_coull")
    
    # 2. Wald CI's
    Wald_CI = proportion_confint(p1, sample_size, alpha, method = "normal")

    # 3. Wald_Corrected 
    correction = 0.05/sample_size
    Wald_Corrected = np.array(Wald_CI) + np.array([-correction, correction])
    
    # 4. Wilson
    wilson_CI = proportion_confint(p1, sample_size, alpha, method = "wilson")
    
    # 5. Wilson Corrected
    LowerCi_Wilson_Corrected=(2*p1 + zcrit**2-1-zcrit*np.sqrt(zcrit**2-2-1/sample_size + 4*(p1/sample_size)*(sample_size*(1-p1/sample_size)+1)))/(2*(sample_size +zcrit**2)) 
    UpperCi_Wilson_Corrected=(2*p1 + zcrit**2+1+zcrit*np.sqrt(zcrit**2+2-1/sample_size + 4*(p1/sample_size)*(sample_size*(1-p1/sample_size)-1)))/(2*(sample_size +zcrit**2))

    # 6. Logit
    lambdahat = math.log(p1/(sample_size-p1))
    term1 = sample_size/(p1*(sample_size-p1))
    lambdalow = lambdahat - zcrit*np.sqrt(term1)
    lambdaupper = lambdahat + zcrit*np.sqrt(term1)
    logitlower = math.exp(lambdalow)/(1 + math.exp(lambdalow))
    logitupper = math.exp(lambdaupper)/(1 + math.exp(lambdaupper))

    # 7. Jeffereys
    lowerjeffreys = beta.ppf(alpha/2, p1+0.5, sample_size-p1+0.5)
    upperjeffreys = beta.ppf(1-alpha/2, p1+0.5, sample_size-p1+0.5)

    # 8. Clopper-Pearson CI's
    lowerCP = beta.ppf(alpha/2, p1, sample_size-p1+1)
    upperCP= beta.ppf(1-alpha/2, p1+1, sample_size-p1)

    # 9. arcsine CI's 1 (Kulynskaya)
    ptilde = (p1 + 0.375)/(sample_size + 0.75)
    arcsinelower = math.sin(math.asin(np.sqrt(ptilde)) - 0.5*zcrit/np.sqrt(sample_size))**2
    arcsineupper = math.sin(math.asin(np.sqrt(ptilde)) + 0.5*zcrit/np.sqrt(sample_size))**2

    # 10. Pratt
    A = ((p1 + 1)/(sample_size-p1))**2
    B = 81 * (p1+1) * (sample_size-p1) - 9*sample_size - 8
    C = -3 * zcrit * np.sqrt(9*(p1+1)*(sample_size - p1) * (9*sample_size + 5 - zcrit**2) + sample_size + 1)
    D = 81 * (p1 + 1)**2 - 9 *(p1+1)* (2+zcrit**2) + 1
    E = 1 + A * ((B+C)/D)**3
    A2 = (p1/ (sample_size-p1-1)) **2
    B2 = 81 * (p1) * (sample_size-p1-1) - 9*sample_size - 8
    C2 = 3 * zcrit * np.sqrt(9*p1*(sample_size-p1-1) * (9*sample_size + 5 - zcrit**2) + sample_size + 1)
    D2 = 81 * p1**2 - (9 *p1) * (2+zcrit**2) + 1
    E2 = 1 + A2 * ((B2+C2)/D2)**3

    upperPratt = 1/E    
    lowerPratt = 1/E2

    # 11. Blaker
    def blakersCI(x, n, conf_level=0.95, tol=0.00001):
    
        def acceptance_probability(x, n, p):
            probabilty1 = 1 - stats.binom.cdf(x - 1, n, p)
            probabilty2 = stats.binom.cdf(x, n, p)
            a1 = probabilty1 + stats.binom.cdf(stats.binom.ppf(probabilty1, n, p) - 1, n, p)
            a2 = probabilty2 + 1 - stats.binom.cdf(stats.binom.ppf(1 - probabilty2, n, p), n, p)
            return min(a1, a2)

        CI_lower_blaker = stats.beta.ppf((1 - conf_level) / 2, x, n - x + 1)
        CI_upper_blaker = stats.beta.ppf(1 - (1 - conf_level) / 2, x + 1, n - x)

        while x != 0 and acceptance_probability(x, n, CI_lower_blaker + tol) < (1 - conf_level):
            CI_lower_blaker += tol
        while x != n and acceptance_probability(x, n, CI_upper_blaker - tol) < (1 - conf_level):
            CI_upper_blaker -= tol

        ci = [CI_lower_blaker, CI_upper_blaker]
        
        return ci
    
    CI_blakers = blakersCI(p1, sample_size, confidence_level)

    
    # 12. Mid-p
    def calculate_midp(x, n, conf_level):
        def f_low(pi):return 0.5 * binom.pmf(x, n, pi) + binom.cdf(x - 1, n, pi) - (1 + conf_level) / 2
        def f_up(pi):return 0.5 * binom.pmf(x, n, pi) + binom.cdf(x - 1, n, pi) - (1 - conf_level) / 2
        CI_lower_midp = newton(f_low, x/n)
        CI_upper_midp = newton(f_up, x/n)
        return CI_lower_midp, CI_upper_midp

    midp_cis = calculate_midp(p1, sample_size, confidence_level)


    results= {}
    results["Agresti Coull CI's"] = AC_CI
    results["Wald CI"] = Wald_CI
    results["Wald CI Corrected"] = Wald_Corrected
    results["Wilson"] = wilson_CI
    results["Wilson Corrected"] = LowerCi_Wilson_Corrected,UpperCi_Wilson_Corrected
    results["logit"] = logitlower,logitupper
    results["Jeffereys"] = lowerjeffreys, upperjeffreys
    results["Clopper-Pearson"] = lowerCP, upperCP
    results["Arcsine"] = arcsinelower, arcsineupper
    results["Pratt"] = lowerPratt, upperPratt
    results["Blaker"] = CI_blakers
    results["Mid-p"] = midp_cis

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str


def effect_sizes_difference_two_indpendednt_proportions(sample_size_1, sample_size_2, proportion_sample_1, proportion_sample_2,difference_in_population):
    #todo - add population difference to the functions and inferntial statistics options
    # Effect Sizes to be added
    # 1. Point Biserial Correlation
    # 2. Chamber's R (see Kirk 2005)
    # 3. rBESD (see Rosenthal and Rosnow, 2003) 

    # Initial Definitions 
    p1 = proportion_sample_1 * sample_size_1
    p2 = proportion_sample_2 * sample_size_2
    q1 = sample_size_1 - p1
    q2 = sample_size_2 - p2
    sample_size = sample_size_1 + sample_size_2 

    #Effect_Sizes
    
    #1. Cohens h
    phi1 = 2*np.arcsin(np.sqrt(proportion_sample_1))
    phi2 = 2*np.arcsin(np.sqrt(proportion_sample_2))
    cohens_h =  abs(phi1 - phi2)
    samples_difference = proportion_sample_1-proportion_sample_2
     
    #2. Calculate Probit_d and its Variance (see Wilson, 2017)
    Z_score_Proportion_1 =  scipy.stats.t.ppf(proportion_sample_1, 100000)
    Z_score_Proportion_2 =  scipy.stats.t.ppf(proportion_sample_2, 100000)
    Probit_d = Z_score_Proportion_1 - Z_score_Proportion_2
    #Add variance calculation for CI's

    #3. Calculate Logit d
    logit_d = np.log((proportion_sample_1/(1-proportion_sample_1))) - np.log((proportion_sample_2/(1-proportion_sample_2)))
    # Add the variance for this

    # 4. Cohens W
    table = np.array([[p1,p2], [q1,q2]])
    chi_square = scipy.stats.chi2_contingency(table).statistic
    Cohens_w = np.sqrt(chi_square/sample_size)


    #Inferntial_Statistics
    continuiety_correction = (1/sample_size_1 + 1/sample_size_2)/2

    #1. Wald Type Z-Statistic
    Standard_Error_Wald_sample_1 = (proportion_sample_1*(1-proportion_sample_1)) / sample_size_1
    Standard_Error_Wald_sample_2 = (proportion_sample_2*(1-proportion_sample_2)) / sample_size_2
    Standard_Error_Wald = np.sqrt(Standard_Error_Wald_sample_1 + Standard_Error_Wald_sample_2)
    Z_wald = (samples_difference - difference_in_population) / Standard_Error_Wald
    p_value_wald = calculate_p_value_from_z_score(Z_wald)

    Z_Wald_corrected = (samples_difference - difference_in_population - continuiety_correction) / Standard_Error_Wald
    p_value_Wald_corrected = calculate_p_value_from_z_score(Z_Wald_corrected)

    #2. Wald_H0 Type Z-Statistic
    Estimitated_Population_Proportion =  (proportion_sample_1*sample_size_1 + proportion_sample_2*sample_size_2) / (sample_size_1 + sample_size_2)
    Standard_error_Wald_H0 = np.sqrt(Estimitated_Population_Proportion * (1-Estimitated_Population_Proportion) * (1/sample_size_1 + 1/sample_size_2))
    Z_Wald_H0 = (samples_difference - difference_in_population) / Standard_error_Wald_H0
    p_value_Wald_H0 = calculate_p_value_from_z_score(Z_Wald_H0)

    Z_Wald_H0_corrected = (samples_difference - difference_in_population - continuiety_correction) / Standard_error_Wald_H0
    p_value_Wald_H0_corrected = calculate_p_value_from_z_score(Z_Wald_H0_corrected)

    # 3. Hauck and Anderson's Z Statistic
    correction_HA = 1/(2 * min(sample_size_1, sample_size_2))# Hauck and Anderson's correction
    Standard_Error_sample_1_HA = (proportion_sample_1*(1-proportion_sample_1)) / (sample_size_1 - 1)
    Standard_Error_sample_2_HA = (proportion_sample_2*(1-proportion_sample_2)) / (sample_size_2 - 1)
    Standard_error_HA = np.sqrt(Standard_Error_sample_1_HA + Standard_Error_sample_2_HA)
    z_score_HA = (samples_difference -  difference_in_population - correction_HA) / Standard_error_HA
    p_value_HA = calculate_p_value_from_z_score(z_score_HA)

    # 4. Conditional Mantel Haenszel Test
    Standart_Error_MH = np.sqrt((sample_size_1*sample_size_2*(p1+p2)* (q1+q2)) / (sample_size**2 *(sample_size-1)))
    Mean_MH = (sample_size_1*(p1+p2)) / sample_size
    Z_MH = (p1 - Mean_MH) / Standart_Error_MH
    pval_MH = calculate_p_value_from_z_score(Z_MH)

    # 5. Barnard Exact
    data_2x2 = [[p1, p2] , [q1,q2]]
    Barnard_Exact = barnard_exact(data_2x2)
    pvalue_barnard = Barnard_Exact.pvalue
    Barnard_D_Statistic = Barnard_Exact.statistic

    # 6. Fisher Exact
    data_2x2 = [[p1, p2] , [q1,q2]]
    Fisher_Exact_p_value = fisher_exact(data_2x2).pvalue

   
    results = {}

    # Descreptive Statistics
    results["Sample 1's Proportion"] = round(proportion_sample_1,4)
    results["Sample 2's Proportion"] = round(proportion_sample_2,4)
    results["Difference Between Proportions"] = round(proportion_sample_1 - proportion_sample_2,4)
    results["Number Of Sucesses 1"] = p1
    results["Number Of Sucesses 2"] = p2
    results["Sample 1 Size"] = sample_size_1
    results["Sample 2 Size"] = sample_size_2
    results["Difference in Population"] = difference_in_population
    
    # Inferential Statistics
    results["Standard Error (Wald/Wald Corrected)"] = round(Standard_Error_Wald, 4)
    results["Z-score (Wald)"] = round(Z_wald, 4)
    results["P-value (Wald)"] = round(p_value_wald, 4)
    results["Z-score (Wald Corrected)"] = round(Z_Wald_corrected, 4)
    results["P-value (Wald Corrected)"] = round(p_value_Wald_corrected, 4)
    results["Standard Error (Wald_H0/Wald Corrected_H0)"] = round(Standard_error_Wald_H0, 4)
    results["Z-score (Wald_H0)"] = round(Z_Wald_H0, 4)
    results["P-value (Wald_H0)"] = round(p_value_Wald_H0, 4)
    results["Z-score (Wald_H0 Corrected)"] = round(Z_Wald_H0_corrected, 4)
    results["P-value (Wald_H0 Corrected)"] = round(p_value_Wald_H0_corrected, 4)
    results["Standart Error Hauck & Anderson"] = round(Standard_error_HA, 4)
    results["Z-score Hauck & Anderson"] = round(z_score_HA, 4)
    results["p-value Hauck & Anderson"] = round(p_value_HA, 4)
    results["Standard Error Mantel & Haenszel"] = round(Standart_Error_MH, 4)
    results["Z-score Mantel & Haenszel"] = round(Z_MH, 4)
    results["p-value Mantel & Haenszel"] = round(pval_MH, 4)
    results["D statistic Barnard Exact"] = round(Barnard_D_Statistic,4)
    results["p-value Barnard Exact"] = round(pvalue_barnard,4)
    results["p-value Fisher Exact"] = round(Fisher_Exact_p_value,4)

    # Effect Sizes and confidence Intervals
    results["Effect Size - Cohen's h"] = round(cohens_h, 4)
    results["Effect Size - Probit d"] = round(Probit_d, 4)
    results["Effect Size - Logit d"] = round(logit_d, 4)
    
    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

def confidence_intervals_for_difference_proporiton(sample_size_1, sample_size_2, proportion_sample_1, proportion_sample_2, Population_Difference, confidence_level):
    
    #Preperations: 
    p1 = proportion_sample_1 * sample_size_1
    p2 = proportion_sample_2 * sample_size_2
    q1 = sample_size_1 - p1
    q2 = sample_size_2 - p2
    sample_size = sample_size_1 + sample_size_2 
    
    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)
    samples_difference = proportion_sample_1 - proportion_sample_2
    Standard_Error_Wald_sample_1 = (proportion_sample_1*(1-proportion_sample_1)) / sample_size_1
    Standard_Error_Wald_sample_2 = (proportion_sample_2*(1-proportion_sample_2)) / sample_size_2
    Standard_Error_Wald = np.sqrt(Standard_Error_Wald_sample_1 + Standard_Error_Wald_sample_2)

    # Method 1 - Wald Method
    lower_ci_wald = samples_difference - zcrit*(Standard_Error_Wald)
    upper_ci_wald = samples_difference + zcrit*(Standard_Error_Wald)

    # Method 2 - Wald corrected Method
    lower_ci_wald_corrected = samples_difference - (0.5*(1/sample_size_1 + 1/sample_size_2) + zcrit*(Standard_Error_Wald))
    upper_ci_wald_corrected = samples_difference + (0.5*(1/sample_size_1 + 1/sample_size_2) + zcrit*(Standard_Error_Wald))

    # Method 3 - Haldane
    psi_Haldane = (proportion_sample_1 + proportion_sample_2) / 2
    v = (1/sample_size_1 - 1/sample_size_2) / 4
    mu = (1/sample_size_1 + 1/sample_size_2) / 4
    theta_haldane = ((proportion_sample_1 - proportion_sample_2) + zcrit**2*v*(1-2*psi_Haldane)) / (1 + zcrit**2*mu)
    w_haldane = (zcrit/ (1+zcrit**2*mu)) * np.sqrt(mu*(4*psi_Haldane*(1-psi_Haldane) - (proportion_sample_1 - proportion_sample_2)**2) + 2*v*(1-2*psi_Haldane) * (proportion_sample_1 - proportion_sample_2) + 4*zcrit**2*mu**2*(1-psi_Haldane)*psi_Haldane  + zcrit**2*v**2*(1-2*psi_Haldane)**2  )
    lower_ci_haldane = theta_haldane - w_haldane
    upper_ci_haldane = theta_haldane + w_haldane

    # Method 4 - Jeffreys-Perks
    psi_JP = 0.5*(((p1 + 0.5) / (sample_size_1+1)) + ((p2+ 0.5) / (sample_size_2+1))) 
    theta_JP = ((proportion_sample_1 - proportion_sample_2) + zcrit**2*v*(1-2*psi_JP)) / (1 + zcrit**2*mu)
    w_JP = (zcrit/ (1+zcrit**2*mu)) * np.sqrt(mu*(4*psi_JP*(1-psi_JP) - (proportion_sample_1 - proportion_sample_2)**2) + 2*v*(1-2*psi_JP) * (proportion_sample_1 - proportion_sample_2) + 4*zcrit**2*mu**2*(1-psi_JP)*psi_JP  + zcrit**2*v**2*(1-2*psi_JP)**2  )
    lower_ci_JP= theta_JP - w_JP
    upper_ci_JP = theta_JP + w_JP

    # Method 5+6 - MEE and Miettinen-Nurminen
    def Standart_Error_Calcualte(proportion_sample_1, sample_size_1, proportion_sample_2, sample_size_2, difference_in_population):
        sample_size_ratio = sample_size_2 / sample_size_1
        a = 1 + sample_size_ratio
        b = -(1 + sample_size_ratio + proportion_sample_1 + sample_size_ratio * proportion_sample_2 + difference_in_population * (sample_size_ratio + 2))
        c = difference_in_population * difference_in_population + difference_in_population * (2 * proportion_sample_1 + sample_size_ratio + 1) + proportion_sample_1 + sample_size_ratio * proportion_sample_2
        d = -proportion_sample_1 * difference_in_population * (1 + difference_in_population)
        v = (b / a / 3) ** 3 - b * c / (6 * a * a) + d / a / 2
        v = np.where(np.abs(v) < np.finfo(float).eps, 0, v)
        s = np.sqrt((b / a / 3) ** 2 - c / a / 3)
        u = np.where(v > 0, 1, -1) * s
        w = (np.pi + np.arccos(v / u ** 3)) / 3
        proportion_sample_hat_1 = 2 * u * np.cos(w) - b / a / 3
        proportion_sample_hat_2 = proportion_sample_hat_1 - difference_in_population
        n = sample_size_1 + sample_size_2
        Variance_Miettinen_Nurminen = (proportion_sample_hat_1 * (1 - proportion_sample_hat_1) / sample_size_1 + proportion_sample_hat_2 * (1 - proportion_sample_hat_2) / sample_size_2) * (n / (n - 1)) 
        Standart_Error_Miettinen_Nurminen = np.sqrt(Variance_Miettinen_Nurminen)
        Variance_Miettinen_MEE = (proportion_sample_hat_1 * (1 - proportion_sample_hat_1) / sample_size_1 + proportion_sample_hat_2 * (1 - proportion_sample_hat_2) / sample_size_2)
        Standart_Error_MEE = np.sqrt(Variance_Miettinen_MEE)


        return Standart_Error_Miettinen_Nurminen, Standart_Error_MEE

    def pval(proportion_sample_1, sample_size_1, proportion_sample_2, sample_size_2, difference_in_population):
        samples_difference = proportion_sample_1 - proportion_sample_2
        se_mn, se_mee = Standart_Error_Calcualte(proportion_sample_1, sample_size_1, proportion_sample_2, sample_size_2, difference_in_population)
        z_mn = (samples_difference - difference_in_population) / se_mn
        z_mee = (samples_difference - difference_in_population) / se_mee
        p_mn = 2 * np.minimum(stats.norm.cdf(z_mn), 1 - stats.norm.cdf(z_mn))
        p_mee = 2 * np.minimum(stats.norm.cdf(z_mee), 1 - stats.norm.cdf(z_mee))

        return p_mn, p_mee

    def confidence_interval(proportion_sample_1, sample_size_1, proportion_sample_2, sample_size_2, confidence_level):
        lower_bracket = [-1, proportion_sample_1 - proportion_sample_2]
        upper_bracket = [proportion_sample_1 - proportion_sample_2, 0.999999]

        def root_func(difference_in_population):
            return pval(proportion_sample_1, sample_size_1, proportion_sample_2, sample_size_2, difference_in_population)[0] - confidence_level

        CI_mn_lower = optimize.root_scalar(root_func, bracket=lower_bracket).root
        CI_mn_upper = optimize.root_scalar(root_func, bracket=upper_bracket).root

        def root_func_mee(difference_in_population):
            return pval(proportion_sample_1, sample_size_1, proportion_sample_2, sample_size_2, difference_in_population)[1] - confidence_level

        CI_mee_lower = optimize.root_scalar(root_func_mee, bracket=lower_bracket).root
        CI_mee_upper = optimize.root_scalar(root_func_mee, bracket=upper_bracket).root


        return (max(-1, CI_mn_lower), min(1, CI_mn_upper)), (max(-1, CI_mee_lower), min(1, CI_mee_upper))

    def z_score(proportion_sample_1, sample_size_1, proportion_sample_2, sample_size_2, Population_Difference):
        samples_difference = proportion_sample_1 - proportion_sample_2
        se_mn, se_mee = Standart_Error_Calcualte(proportion_sample_1, sample_size_1, proportion_sample_2, sample_size_2, Population_Difference)
        z_score_mn = (samples_difference - Population_Difference) / se_mn
        z_score_mee = (samples_difference - Population_Difference) / se_mee
        return z_score_mn, z_score_mee

    CI_mn, CI_mee = confidence_interval(proportion_sample_1, sample_size_1, proportion_sample_2, sample_size_2, 1 - confidence_level)
    z_score_mn, z_score_mee = z_score(proportion_sample_1, sample_size_1, proportion_sample_2, sample_size_2, Population_Difference)
    se_mn, se_mee = Standart_Error_Calcualte(proportion_sample_1, sample_size_1, proportion_sample_2, sample_size_2, Population_Difference)
    
    # Method 7 - Agresti_Caffo
    p1_agresti_caffo = (p1 +1) / (sample_size_1 +2)
    p2_agresti_caffo = (p2 +1) / (sample_size_2 +2)
    Standard_Error_sample_1_AC = (p1_agresti_caffo*(1-p1_agresti_caffo)) / (sample_size_1 + 2)
    Standard_Error_sample_2_AC = (p2_agresti_caffo*(1-p2_agresti_caffo)) / (sample_size_2 + 2)
    Standard_error_AC = np.sqrt(Standard_Error_sample_1_AC + Standard_Error_sample_2_AC)
    lower_ci_AC= (p1_agresti_caffo-p2_agresti_caffo) - zcrit* Standard_error_AC
    upper_ci_AC = (p1_agresti_caffo-p2_agresti_caffo) + zcrit* Standard_error_AC

    #Method 8 - Wilson
    epsilon_1, constant1 = (p1 + zcrit ** 2 / 2) / ( sample_size_1 + zcrit ** 2), zcrit * math.sqrt(sample_size_1) / (sample_size_1 + zcrit ** 2) * math.sqrt(proportion_sample_1 * (1 - proportion_sample_1) + zcrit ** 2 / (4 * sample_size_1))
    CI_wilson1_lower, CI_wilson1_upper = max(0, epsilon_1 - constant1), min(1, epsilon_1 + constant1)
    epsilon_2, constant2 = (p2 + zcrit ** 2 / 2) / ( sample_size_2 + zcrit ** 2), zcrit * math.sqrt(sample_size_2) / (sample_size_2 + zcrit ** 2) * math.sqrt(proportion_sample_2 * (1 - proportion_sample_2) + zcrit ** 2 / (4 * sample_size_2))
    CI_wilson2_lower, CI_wilson2_upper = max(0, epsilon_2 - constant2), min(1, epsilon_2 + constant2)

    CI_wilson_lower = samples_difference - zcrit * np.sqrt(CI_wilson1_lower*(1-CI_wilson1_lower)/sample_size_1 + CI_wilson2_upper*(1-CI_wilson2_upper)/sample_size_2)
    CI_wilson_upper = samples_difference + zcrit * np.sqrt(CI_wilson1_upper*(1-CI_wilson1_upper)/sample_size_1 + CI_wilson2_lower*(1-CI_wilson2_lower)/sample_size_2)

    #Method 9 - Wilson Corrected
    CI_wilsonc1_lower = (2 * p1 + zcrit**2 - 1 - zcrit * math.sqrt(zcrit**2 - 2 - 1/sample_size_1 + 4 * proportion_sample_1 * (sample_size_1 * (1 - proportion_sample_1) + 1))) / (2 * (sample_size_1 + zcrit**2))
    CI_wilsonc1_upper = (2 * p1 + zcrit**2 + 1 + zcrit * math.sqrt(zcrit**2 + 2 - 1/sample_size_1 + 4 * proportion_sample_1 * (sample_size_1 * (1 - proportion_sample_1) - 1))) / (2 * (sample_size_1 + zcrit**2))
    CI_wilsonc2_lower = (2 * p2 + zcrit**2 - 1 - zcrit * math.sqrt(zcrit**2 - 2 - 1/sample_size_2 + 4 * proportion_sample_2 * (sample_size_2 * (1 - proportion_sample_2) + 1))) / (2 * (sample_size_2 + zcrit**2))
    CI_wilsonc2_upper = (2 * p2 + zcrit**2 + 1 + zcrit * math.sqrt(zcrit**2 + 2 - 1/sample_size_2 + 4 * proportion_sample_2 * (sample_size_2 * (1 - proportion_sample_2) - 1))) / (2 * (sample_size_2 + zcrit**2))

    CI_wilsonc_lower = max(-1, samples_difference - np.sqrt((proportion_sample_1 - CI_wilsonc1_lower)**2 + (CI_wilsonc2_upper-proportion_sample_2)**2) )
    CI_wilsonc_upper = min( 1, samples_difference + np.sqrt((CI_wilsonc1_upper-proportion_sample_1)**2 + (proportion_sample_2-CI_wilsonc2_lower)**2) )

    #Method 10 - Hauck-Anderson
    correction_HA = 1/(2 * min(sample_size_1, sample_size_2))# Hauck and Anderson's correction
    Standard_Error_sample_1_HA = (proportion_sample_1*(1-proportion_sample_1)) / (sample_size_1 - 1)
    Standard_Error_sample_2_HA = (proportion_sample_2*(1-proportion_sample_2)) / (sample_size_2 - 1)
    Standard_error_HA = np.sqrt(Standard_Error_sample_1_HA + Standard_Error_sample_2_HA)
   
    CI_lower_HA = max(samples_difference - 1/(2 * min(sample_size_1,sample_size_2)) - zcrit * Standard_error_HA, -1)
    CI_upper_HA = min(samples_difference + 1/(2 * min(sample_size_1,sample_size_2)) + zcrit * Standard_error_HA, 1)

    #Method 11 - Brown, Li’s Jeffreys
    p1_BLJ = (p1 + 0.5) / (sample_size_1+1)
    p2_BLJ = (p2 + 0.5) / (sample_size_2+1)
    Standard_Error_BLJ = np.sqrt(p1_BLJ*(1-p1_BLJ)/sample_size_1 + p2_BLJ*(1-p2_BLJ)/sample_size_2)
    CI_BLJ_lower = (p1_BLJ - p2_BLJ) - zcrit * Standard_Error_BLJ
    CI_BLJ_upper = (p1_BLJ - p2_BLJ) + zcrit * Standard_Error_BLJ

    #method 12 - Gart Nam
    robjects.r('''scoretheta <- function(x1, n1, x2, n2, theta, level = 0.95) {Prop_Diff <- ((x1 / n1) - (x2 / n2)) - theta
            N <- n1 + n2
            a <- (n1 + 2 * n2) * theta - N - (x1 + x2)
            b <- (a / N / 3)^3 - a * ((n2 * theta - N - 2 * x2) * theta + (x1 + x2)) / (6 * N * N) + (x2 * theta * (1 - theta)) / N / 2
            c <- ifelse(b > 0, 1, -1) * sqrt(pmax(0, (a / N / 3)^2 - ((n2 * theta - N - 2 * x2) * theta + (x1 + x2)) / N / 3))
            p2d <- pmin(1, pmax(0, round(2 * c * cos(((pi + acos(pmax(-1, pmin(1, ifelse(c == 0 & b == 0, 0, b / c^3))))) / 3)) - a / N / 3, 10)))
            p1d <- pmin(1, pmax(0, p2d + theta))
            Variance <- pmax(0, (p1d * (1 - p1d) / n1 + p2d * (1 - p2d) / n2))
            scterm <- (p1d * (1 - p1d) * (1 - 2 * p1d) / (n1^2) - p2d * (1 - p2d) * (1 - 2 * p2d) / (n2^2)) / (6 * Variance^(3 / 2))
            score <- ifelse(scterm == 0, (Prop_Diff / sqrt(Variance)), (-1 + sqrt(pmax(0, 1^2 - 4 * scterm * -(Prop_Diff / sqrt(Variance) + scterm))) ) / (2 * scterm))
            return(score)}

            Binary_Search <- function(score_function, max.iter = 100, tail = "lower") {
            nstrat <- length(eval(score_function(1)))
            hi <- rep(1, nstrat)
            lo <- rep(-1, nstrat)
            niter <- 1
            while (niter <= max.iter && any(2 > 0.0000005 | is.na(hi))) {
            mid <- pmax(-1, pmin(1, round((hi + lo) / 2, 10)))
            scor <- score_function(mid)
            check <- (scor <= 0) | is.na(scor)
            hi[check] <- mid[check]
            lo[!check] <- mid[!check]
            niter <- niter + 1}
            ci <- if (tail == "lower") lo else hi
            return(ci)}

            gart_nam = function(x1, n1, x2, n2, level = 0.95) {
            zcrit <- qnorm(1 - (1 - level)/2)
            lower <- Binary_Search(score_function = function(theta) scoretheta(x1, n1, x2, n2,theta) - zcrit, tail = "lower")
            upper <- Binary_Search(score_function = function(theta) scoretheta(x1, n1, x2, n2,theta) + zcrit, tail = "upper")
            return(c(lower, upper))
            }''')

    Gart_Nam_CI = robjects.r['gart_nam'](p1, sample_size_1, p2, sample_size_2, confidence_level)  # type: ignore


    results = {}    
    
    results["Confidence Intervals Wald"] = f"({round(lower_ci_wald, 4)}, {round(upper_ci_wald, 4)})"
    results["Confidence Intervals Wald Corrected"] = f"({round(lower_ci_wald_corrected, 4)}, {round(upper_ci_wald_corrected, 4)})"
    results["Confidence Intervals Haldane"] = f"({round(lower_ci_haldane, 4)}, {round(upper_ci_haldane, 4)})"
    results["Confidence Intervals Jeffreys-Perks"] = f"({round(lower_ci_JP, 4)}, {round(upper_ci_JP, 4)})"
    results["Confidence Intervals Mee"] = around(CI_mee,4)
    results["Confidence Intervals Miettinen-Nurminen"] = around(CI_mn,4)
    results["Confidence Intervals Gart-Nam"] = around(Gart_Nam_CI,4)
    results["Confidence Intervals Agrest-Caffo"] = f"({round(lower_ci_AC, 4)}, {round(upper_ci_AC, 4)})"
    results["Confidence Intervals Wilson"] = f"({round(CI_wilson_lower, 4)}, {round(CI_wilson_upper, 4)})"
    results["Confidence Intervals Wilson Corrected"] = f"({round(CI_wilsonc_lower, 4)}, {round(CI_wilsonc_upper, 4)})"
    results["Confidence Intervals Hauck-Anderson"] = f"({round(CI_lower_HA, 4)}, {round(CI_upper_HA, 4)})"
    results["Confidence Intervals Brown-Lee-Jeffereys"] = f"({round(CI_BLJ_lower, 4)}, {round(CI_BLJ_upper, 4)})"

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str


def odds_Ratio_two_independent_samples(sample_size_1, sample_size_2, proportion_sample_1, proportion_sample_2, confidence_level):
    
    #Initial Definitions
    p1 = proportion_sample_1 * sample_size_1
    p2 = proportion_sample_2 * sample_size_2
    q1 = sample_size_1 - p1
    q2 = sample_size_2 - p2
    sample_size = sample_size_1 + sample_size_2

    
    # Odds Ratio effect sizes Calculation
    
    # 1. Conditional MLE (Corenfield)
    odds_ratio_Conditional_test = odds_ratio([[int(p1), int(q1)], [int(p2), int(q2)]])
    odds_ratio_Conditional = odds_ratio_Conditional_test.statistic
    
    # 2. Unconditional MLE (Wald)
    odds_ratio_wald = (proportion_sample_1*(1-proportion_sample_2)) / (proportion_sample_2* (1-proportion_sample_1))

    # 3. Wald_Adjusted
    odds_ratio_wald_adjusted = ((p1 + 0.5) * (q2 + 0.5)) / ((p2 + 0.5) * (q1 + 0.5))

    # 4. Wald Small Samples Correction
    odds_ratio_small_samples_adjusted = (p1 * q2) / ((p2 + 1) * (q1 + 1))

    # 5. Median unbaised odds ratio
    robjects.r('''odds_ratio_median_unbaised = function(p1, q1, p2, q2, conf.level = 0.95, interval = c(0, 1000)) {
        alpha = 1 - conf.level
        x = matrix(c(p1, q1, p2, q2), 2, 2)
        oddsratio = uniroot(function(or_val) {fisher.test(x, or = or_val, alternative = "less")$p.value - fisher.test(x, or = or_val, alternative = "greater")$p.value},interval = interval)$root
        return(oddsratio)}''')
    
    odds_ratio_median_unbaised = robjects.r['odds_ratio_median_unbaised'](p1, p2, q1, q2)  # type: ignore


    # Standard Error of the Odds Ratio
    standard_error_odds_ratio =np.sqrt((1 / (p1)) + (1 / (p2)) + (1 / (q1)) + (1 / (q2)))
    standard_error_odds_ratio_adjusted =np.sqrt((1 / (p1+0.5)) + (1 / (p2+0.5)) + (1 / (q1 + 0.5)) + (1 / (q2 + 0.5)))


    # Inferential Statistics
    
    # 1. chi_square_p values
    chi2_odds_ratio = (abs(p1 - ((p1+q1)*(p1+p2)/sample_size)))**2 / ((p1+q1)*(p1+p2)/sample_size) + \
    (abs(q1 - ((p1+q1)*(q1+q2)/sample_size)))**2 / ((p1+q1)*(q1+q2)/sample_size) + \
    (abs(p2 - ((p2+q2)*(p1+p2)/sample_size)))**2 / ((p2+q2)*(p1+p2)/sample_size) + \
    (abs(q2 - ((p2+q2)*(q1+q2)/sample_size)))**2 / ((p2+q2)*(q1+q2)/sample_size) 

    p_value_odds_ratio  = calculate_p_value_from_chi_score(chi2_odds_ratio, 1)

    # 2. chi_square_adjusted_p values
    chi2_odds_ratio_adjusted = (abs(p1 - ((p1+q1)*(p1+p2)/sample_size)) - 0.5)**2 / ((p1+q1)*(p1+p2)/sample_size) + \
    (abs(q1 - ((p1+q1)*(q1+q2)/sample_size)) - 0.5)**2 / ((p1+q1)*(q1+q2)/sample_size) + \
    (abs(p2 - ((p2+q2)*(p1+p2)/sample_size)) - 0.5)**2 / ((p2+q2)*(p1+p2)/sample_size) + \
    (abs(q2 - ((p2+q2)*(q1+q2)/sample_size)) - 0.5)**2 / ((p2+q2)*(q1+q2)/sample_size)

    p_value_odds_ratio_adjusted = calculate_p_value_from_chi_score(chi2_odds_ratio_adjusted, 1)

    # 3. Fisher Exact p.value
    contingency_table = [[p1, p2], [q1, q2]]
    p_value_fisher = fisher_exact(contingency_table)[1]

    # 4. Mid_p_value
    def odds_ratio_mid_p_value_function(p1, q1, p2, q2):
        x = [[p1, q1], [p2, q2]]
        p_value_less = fisher_exact(x, alternative='less')[1]
        p_value_greater = fisher_exact(x, alternative='greater')[1]
        mid_p = 0.5 * (p_value_less - p_value_greater + 1)
        one_sided_mid_p = min(mid_p, 1 - mid_p)
        two_sided_mid_p = 2 * one_sided_mid_p
        return two_sided_mid_p

    mid_p_value = odds_ratio_mid_p_value_function(p1, p2, q1, q2)

    # Yules Q
    Yules_Q = (p1*q2 - q1*p2) / (p1*q2 + q1*p2)
    Yules_Y = ((p1*q2)**0.5 - (q1*p2)**0.5) / ((p1*q2)**0.5 + (q1*p2)**0.5)
    Digbys_H = ((p1*q2)**0.75 - (q1*p2)**0.75) / ((p1*q2)**0.75 + (q1*p2)**0.75)

    #Yules Y* (Bonett and Price, 2007) 
    Marginal_Proportion_1 = (p1+p2)/sample_size
    Marginal_Proportion_2 = (p1+q1)/sample_size
    Marginal_Proportion_3 = (p2+q2)/sample_size
    Marginal_Proportion_4 = (q1+q2)/sample_size

    marginal_proportions = np.array([Marginal_Proportion_1, Marginal_Proportion_2, Marginal_Proportion_3, Marginal_Proportion_4])
    min_marginal_prooportion = min(marginal_proportions)
    c = 0.5 - (0.5 - min_marginal_prooportion) **2
    corrected_odds_ratio = ((p1+0.1) * (q2+0.1)) / ((p2+0.1) * (q1+0.1))
    Yules_Y_Star = (corrected_odds_ratio**c - 1) / (corrected_odds_ratio**c + 1)

    # Standard Errors by Bishop, Fienberg, Holland - 2007 (Discrete Mulyivariete analysis). For the Y star its from Bonett and Star
    Standard_Error_Yules_Q = 0.5*(1-Yules_Q**2) * standard_error_odds_ratio
    Standard_Error_Yules_Y = 0.25*(1-Yules_Y**2) * standard_error_odds_ratio
    Standard_Error_Digbys_H = 0.5*0.75*(1-Digbys_H**2) * standard_error_odds_ratio
    Standard_Error_Yules_Y_Star = np.sqrt( (c**2/4) * ((1 /(p1+0.1)) + (1/(q2+0.1)) + (1/(p2+0.1)) +( 1/(q1+0.1))))
    
    #Confidence Intervals for the Yules Family
    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)

    Yules_Q_CI_lower = (Yules_Q - zcrit*Standard_Error_Yules_Q)
    Yules_Q_CI_upper = (Yules_Q + zcrit*Standard_Error_Yules_Q)

    Digbys_H_Lower = (Yules_Q - zcrit*Standard_Error_Digbys_H)
    Digbys_H_Upper = (Digbys_H + zcrit*Standard_Error_Digbys_H)

    Yules_Y_CI_lower  = (Yules_Y - zcrit*Standard_Error_Yules_Y)
    Yules_Y_CI_upper  = (Yules_Y + zcrit*Standard_Error_Yules_Y)

    Yules_Y_Star_CI_lower = np.tanh(np.arctanh(Yules_Y_Star) - zcrit * Standard_Error_Yules_Y_Star)
    Yules_Y_Star_CI_upper = np.tanh(np.arctanh(Yules_Y_Star) + zcrit * Standard_Error_Yules_Y_Star)


    # add the log odds ratio next to the odds ratio

    results = {}

    results["Odds Ratio (Wald Unconditional MLE)"] = round(odds_ratio_wald, 4)
    results["Odds Ratio (Wald Adjusted)"] = round(odds_ratio_wald_adjusted, 4)
    results["Odds Ratio (Fisher Conditional MLE)"] = round(odds_ratio_Conditional, 4)
    results["Odds Ratio Small Samples Adaptation"] = round(odds_ratio_small_samples_adjusted, 4)
    results["Odds Ratio Median Unbaised (Mid-P)"] = around(odds_ratio_median_unbaised, 4)
    results["Yule's Q"] = around(Yules_Q, 4)
    results["Yule's Y"] = around(Yules_Y, 4)
    results["Digby's H"] = around(Digbys_H, 4)
    results["Yule's Y* (Bonett & Price)"] = around(Yules_Y_Star, 4)

    results["Standard Error of the Odds Ratio"] = round(standard_error_odds_ratio, 4)
    results["Standard Error of the Odds Ratio (adjusted)"] = round(standard_error_odds_ratio_adjusted, 4)
    results["Standard Error of Yules Q"] = around(Standard_Error_Yules_Q, 4)
    results["Standard Error of Yules Y"] = around(Standard_Error_Yules_Y, 4)
    results["Standard Error of Digby's H"] = around(Standard_Error_Digbys_H, 4)
    results["Standard Error of Yule's Y*"] = around(Standard_Error_Yules_Y_Star, 4)

    results["Odds Ratio p-value (Chi square)"] = round(p_value_odds_ratio, 7)
    results["Odds Ratio p-value (Adjusted Chi square)"] = round(p_value_odds_ratio_adjusted, 7)
    results["Odds Ratio p-value (Fisher Exact)"] = round(p_value_fisher, 7)
    results["Odds Ratio p-value (Mid-p value)"] = round(mid_p_value, 7)
    
    results["Confidence Intervals Yules Q"] = f"({round(Yules_Q_CI_lower, 4)}, {round(Yules_Q_CI_upper, 4)})"
    results["Confidence Intervals Yules Y"] = f"({round(Yules_Y_CI_lower, 4)}, {round(Yules_Y_CI_upper, 4)})"
    results["Confidence Intervals Digby's H "] = f"({round(Digbys_H_Lower, 4)}, {round(Digbys_H_Upper, 4)})"
    results["Confidence Intervals Yules Y*"] = f"({round(Yules_Y_Star_CI_lower, 4)}, {round(Yules_Y_Star_CI_upper, 4)})"

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str
         

def Indpendent_Odds_Ratio_CI(sample_size_1, sample_size_2, proportion_sample_1, proportion_sample_2, confidence_level):
    
    p1 = proportion_sample_1 * sample_size_1
    p2 = proportion_sample_2 * sample_size_2
    q1 = sample_size_1 - p1
    q2 = sample_size_2 - p2

    odds_ratio_wald = (proportion_sample_1*(1-proportion_sample_2)) / (proportion_sample_2* (1-proportion_sample_1))
    standard_error_odds_ratio =np.sqrt((1 / (p1)) + (1 / (p2)) + (1 / (q1)) + (1 / (q2)))
    standard_error_odds_ratio_adjusted =np.sqrt((1 / (p1+0.5)) + (1 / (p2+0.5)) + (1 / (q1 + 0.5)) + (1 / (q2 + 0.5)))

    #Confidence Intervals:
    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)

    # Method 1 - Woolf (Aka Wald) - Logarithmic asymptotyc CI's
    upper_ci_woolf = np.exp(np.log(odds_ratio_wald) + zcrit* standard_error_odds_ratio)
    lower_ci_woolf = np.exp(np.log(odds_ratio_wald) - zcrit* standard_error_odds_ratio)
    
    # Method 2 - Woolf Corrected  (Wald adjusted) - Logarithmic asymptotyc CI's with + 0.5 correction
    upper_or_woolf_adjusted = np.exp(np.log(odds_ratio_wald) + zcrit* standard_error_odds_ratio_adjusted)
    lower_or_woolf_adjusted = np.exp(np.log(odds_ratio_wald) - zcrit* standard_error_odds_ratio_adjusted)

    # Method 3 - Cornfield (Fisher exact CI's)
    odds_ratio_Conditional_test = odds_ratio([[int(p1), int(q1)], [int(p2), int(q2)]])
    odds_ratio_fisher_ci = odds_ratio_Conditional_test.confidence_interval(confidence_level)
    formatted_ci_lower = format(odds_ratio_fisher_ci[0], ".6f")
    formatted_ci_upper = format(odds_ratio_fisher_ci[1], ".6f")

    # Method 4 - Mid-p Confidence Intervals
    robjects.r('''odds_ratio_mid_p_value = function(p1, q1, p2, q2, conf.level = 0.95, interval = c(0, 1000)) {
        mid_p_function = function(or_val = 1) {
            less_p_value = fisher.test(matrix(c(p1, q1, p2, q2), 2, 2), or = or_val, alternative = "less")$p.value
            greater_p_value = fisher.test(matrix(c(p1, q1, p2, q2), 2, 2), or = or_val, alternative = "greater")$p.value
            0.5 * (less_p_value - greater_p_value + 1)
            }

        alpha = 1 - conf.level
        x = matrix(c(p1, q1, p2, q2), 2, 2)
            
        oddsratio = uniroot(function(or_val) {fisher.test(x, or = or_val, alternative = "less")$p.value - fisher.test(x, or = or_val, alternative = "greater")$p.value},interval = interval)$root
        lower_ci_small = uniroot(function(or_val) {1 - mid_p_function(or_val) - alpha/2}, interval = interval)$root
        upper_ci_small = 1/uniroot(function(or_val) {mid_p_function(1/or_val) - alpha/2}, interval = interval)$root

        return(c(oddsratio, lower_ci_small, upper_ci_small))}''')

    lower_ci_mid_p  = robjects.r['odds_ratio_mid_p_value'](p1, p2, q1, q2)[1] # type: ignore
    upper_ci_mid_p  = robjects.r['odds_ratio_mid_p_value'](p1, p2, q1, q2)[2] # type: ignore
   
    # Method 5 - the score method of Miettinen and Nurminen (1985)
    CI_MN = confint_proportions_2indep(nobs1=sample_size_1,count1 = p1,nobs2=sample_size_2,count2 = p2,method = "score", compare = "odds-ratio", alpha = 1 - confidence_level)


    # Method 6 - Baptista and Pike 1977
    robjects.r('''Baptista_Pike = function(p1,q1,p2,q2, conf.level = 0.95, orRange = c(10^-10, 10^10)) {
        x = matrix(c(p1,q1,p2,q2),2,2)
        alpha <- 1 - conf.level; 
        n1 <- sum(x[1, ]); n2 <- sum(x[2, ])
        sum_of_ps <- sum(x[, 1]); x <- x[1, 1]
        support <- max(0, sum_of_ps - n2):min(n1, sum_of_ps)
        dnhyper <- function(OR) {
            d <- dhyper(support, n1, n2, sum_of_ps, log = TRUE) + log(OR) * support
            exp(d - max(d)) / sum(exp(d - max(d)))}
        pnhyper <- function(x, OR, lower.tail = TRUE) {
            f <- dnhyper(OR)
            X <- if (lower.tail) support <= x else support >= x
            sum(f[X])}
        intercept <- function(xlo, xhi, ORRange = orRange) {
            X <- support <= xlo; Y <- support >= xhi
            uniroot(function(beta) sum(dnhyper(beta)[X]) - sum(dnhyper(beta)[Y]), ORRange)$root}
        ints_greater <- intercept(x, x + 1)
        ints_less <- intercept(x - 1, x)
        CINT_upper <-uniroot(function(or) alpha - pnhyper(x, or, lower.tail = TRUE), c(ints_greater, orRange[2]))$root
        CINT_lower <-uniroot(function(or) alpha - pnhyper(x, or, lower.tail = FALSE), c(orRange[1], ints_less))$root
        c(CINT_lower, CINT_upper)}''')

    lower_bp  = robjects.r['Baptista_Pike'](p1,q1,p2,q2)[0] # type: ignore
    upper_ci_bp  = robjects.r['Baptista_Pike'](p1,q1,p2,q2)[1] # type: ignore

    # Method 7 - inverse hyperbolic sine method (Newcomb, 2001)
    standart_error_sinh = 2 * math.asinh(zcrit / 2 * math.sqrt(1/p1 + 1/(q1) + 1/p2 + 1/(q2)))
    lower_limit_sinh = math.exp(math.log(odds_ratio_wald) - standart_error_sinh)
    upper_limit_sinh = math.exp(math.log(odds_ratio_wald) + standart_error_sinh) 

    # Method 8 - Independent Smooth logit (Agresti, 1999)
    p1new = p1 + 2 * sample_size_1 * (p1 + p2) / (sample_size_1 + sample_size_2)**2
    q1new = sample_size_1 - p1 + 2 * sample_size_1 * (sample_size_1 - p1 + sample_size_2 - p2) / (sample_size_1 + sample_size_2)**2
    p2new = p2 + 2 * sample_size_2 * (p1 + p2) / (sample_size_1 + sample_size_2)**2
    q2new = sample_size_2 - p2 + 2 * sample_size_2 * (sample_size_1 - p1 + sample_size_2 - p2) / (sample_size_1 + sample_size_2)**2
    log_theta = np.log(p1new * q2new / (p2new * q1new))
    ci_half_len = norm.ppf(1 - (1 - confidence_level) / 2) * np.sqrt(1 / p1new + 1 / q1new + 1 / p2new + 1 / q2new)
    ci_lower_agresti_ind = np.exp(log_theta - ci_half_len)
    ci_upper_agresti_ind = np.exp(log_theta + ci_half_len)

    # Method 9 - Farrington-Manning
    robjects.r('''score_test_statistic.Uncorrected <- function(theta0, n11, n21, n1p, n2p) {
        p2hat <- (-(n1p * theta0 + n2p - (n11 + n21) * (theta0 - 1)) + sqrt((n1p * theta0 +
        n2p - (n11 + n21) * (theta0 - 1))^2 - 4 * n2p * (theta0 - 1) * -(n11 + n21))) / (2 * n2p * (theta0 - 1))
        p1hat <- p2hat * theta0 / (1 + p2hat * (theta0 - 1))
        T0 <- ((n1p * (n11 / n1p - p1hat)) * sqrt(1 / (n1p * p1hat * (1 - p1hat)) + 1 / (n2p * p2hat * (1 - p2hat))))}

        lower_limit <- function(theta0, n11, n21, n1p, n2p, alpha) {
        T0 <- score_test_statistic.Uncorrected(theta0, n11, n21, n1p, n2p)
        f <- T0 - qnorm(1 - alpha / 2, 0, 1)}

        upper_limit <- function(theta0, n11, n21, n1p, n2p, alpha) {
        T0 <- score_test_statistic.Uncorrected(theta0, n11, n21, n1p, n2p)
        f <- T0 + qnorm(1 - alpha / 2, 0, 1)}

        FM_CI <- function(p1,p2,q1,q2, alpha = 0.05) {
        L <- uniroot(lower_limit, c(0.000001, 100000),p1, q1, (p1+p2), (q1+q2), alpha)$root
        U <- uniroot(upper_limit, c(0.000001, 100000),p1, q1, (p1+p2), (q1+q2), alpha )$root
        c(L, U)}''')
    
    lower_FM  = robjects.r['FM_CI'](p1,q1,p2,q2)[0]  #type: ignore
    upper_FM  = robjects.r['FM_CI'](p1,q1,p2,q2)[1]  #type: ignore

    # Method 10 - Unconditional Score Method (Agresti & Min, 2002) #see exact method (parmtype = oddsration and method = scores)



    results = {}

    results["Confidence Intervals (Woolf)"] = f"({round(lower_ci_woolf, 4)}, {round(upper_ci_woolf, 4)})"
    results["Confidence Intervals (Woolf Adjusted)"] = f"({round(lower_or_woolf_adjusted, 4)}, {round(upper_or_woolf_adjusted, 4)})"
    results["Confidence Intervals Fisher Exact (Cornfield)"] = formatted_ci_lower, formatted_ci_upper
    results["Confidence Intervals Mid-p"] = f"({round(lower_ci_mid_p, 4)}, {round(upper_ci_mid_p, 4)})"
    results["Confidence Intervals Miettinen and Nurminen"] = around(CI_MN,4)
    results["Confidence Intervals Baptista-Pike"] = f"({round(lower_bp, 4)}, {round(upper_ci_bp, 4)})"
    results["Confidence Intervals Inverse Sine (Newcomb)"] = f"({round(lower_limit_sinh, 4)}, {round(upper_limit_sinh, 4)})"
    results["Confidence Intervals Independent Smooth (Agresti)"] = f"({round(ci_lower_agresti_ind, 4)}, {round(ci_upper_agresti_ind, 4)})"
    results["Confidence Intervals Uncorrected Score (Farringot and Manning)"] = f"({round(lower_FM, 4)}, {round(upper_FM, 4)})"

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

def risk_Ratio_two_independent_samples(sample_size_1, sample_size_2, proportion_sample_1, proportion_sample_2,confidence_level):
    p1 = proportion_sample_1 * sample_size_1
    p2 = proportion_sample_2 * sample_size_2
    q1 = sample_size_1 - p1
    q2 = sample_size_2 - p2
    samples_difference = (proportion_sample_1 - proportion_sample_2)
    
    sample_size = sample_size_1 + sample_size_2 

    Realtive_risk = proportion_sample_1/proportion_sample_2
    Realtive_risk_unbiased = (p1*(q1+q2+1)) / (((p1+p2) * (q1 + 1)))
    Realtive_risk_adjusted_Walters = np.exp(np.log((p1+0.5) / (sample_size_1+0.5)) - np.log((p2+0.5) / (sample_size_2+0.5)) )

    
    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)


    #Standard Error of the relative risk
    standart_error_Relative_Risk_katz = np.sqrt((((1 - proportion_sample_1)) / p1) + ((1-proportion_sample_2)/p2))
    standart_error_Relative_Risk_Walters = np.sqrt((1/(p1+0.5)) - (1/(sample_size_1+0.5)) + (1/(p2+0.5)) - (1/(sample_size_2+0.5)))

    # 1. Walters
    upper_rr_walters = np.exp(np.log(Realtive_risk_adjusted_Walters) + zcrit* standart_error_Relative_Risk_Walters)
    lower_rr_walters = np.exp(np.log(Realtive_risk_adjusted_Walters) - zcrit* standart_error_Relative_Risk_Walters)

    # 2. Katz
    upper_rr_kats = np.exp(np.log(Realtive_risk) + zcrit* standart_error_Relative_Risk_katz)
    lower_rr_kats = np.exp(np.log(Realtive_risk) - zcrit* standart_error_Relative_Risk_katz)

    # 3. Miettinen and Nurminen
    def RRci(x1, n1, x2, n2, conf_level):
        z = abs(stats.norm.ppf((1 - conf_level) / 2))
        a1 = n2 * (n2 * (n2 + n1) * x1 + n1 * (n2 + x1) * (z ** 2))
        a2 = -n2 * (n2 * n1 * (x2 + x1) + 2 * (n2 + n1) * x2 * x1 + n1 * (n2 + x2 + 2 * x1) * (z ** 2))
        a3 = 2 * n2 * n1 * x2 * (x2 + x1) + (n2 + n1) * (x2 ** 2) * x1 + n2 * n1 * (x2 + x1) * (z ** 2)
        a4 = -n1 * (x2 ** 2) * (x2 + x1)
        b1 = a2 / a1
        b2 = a3 / a1
        b3 = a4 / a1
        c1 = b2 - (b1 ** 2) / 3
        c2 = b3 - b1 * b2 / 3 + 2 * (b1 ** 3) / 27
        ceta = math.acos(math.sqrt(27) * c2 / (2 * c1 * math.sqrt(-c1)))
        t1 = -2 * math.sqrt(-c1 / 3) * math.cos(math.pi / 3 - ceta / 3)
        t2 = -2 * math.sqrt(-c1 / 3) * math.cos(math.pi / 3 + ceta / 3)
        t3 = 2 * math.sqrt(-c1 / 3) * math.cos(ceta / 3)
        p01 = t1 - b1 / 3
        p02 = t2 - b1 / 3
        p03 = t3 - b1 / 3
        p0sum = p01 + p02 + p03
        p0up = min(p01, p02, p03)
        p0low = p0sum - p0up - max(p01, p02, p03)
        ul = (1 - (n1 - x1) * (1 - p0up) / (x2 + n1 - (n2 + n1) * p0up)) / p0up
        ll = (1 - (n1 - x1) * (1 - p0low) / (x2 + n1 - (n2 + n1) * p0low)) / p0low
        cint = [ll, ul]
        return cint

    CI_Koopman = RRci(p1,sample_size_1, p2, sample_size_2, confidence_level)

    # 4+5 Inverse Sine and adjusted Inverse Sine
    Adjusted_Epsilon = math.asinh(0.5 * zcrit * np.sqrt(1 / p1 + 1 / p2 - 1 / (sample_size_1+ 1) - 1 / (sample_size_2 + 1)))
    Epsilon = math.asinh(0.5 * zcrit * np.sqrt(1/p1 + 1/p2 - 1/sample_size_1 - 1/sample_size_2))
    Lower_Sinh_adjusted = np.exp(np.log(Realtive_risk) - 2 * Adjusted_Epsilon)
    Upper_Sinh_adjusted = np.exp(np.log(Realtive_risk) + 2 * Adjusted_Epsilon)
    Lower_Sinh_ = np.exp(np.log(Realtive_risk) - 2 * Epsilon)
    Upper_Sinh_ = np.exp(np.log(Realtive_risk) + 2 * Epsilon)

    results = {}

    results["Realtive risk"] = round(Realtive_risk, 7)
    results["Realtive risk unbiased (Jewell)"] = round(Realtive_risk_unbiased, 7)
    results["Realtive risk adjusted (Walters)"] = round(Realtive_risk_adjusted_Walters, 7)

    results["standart_error_Relative_Risk_Katz"] = round(standart_error_Relative_Risk_katz, 7)
    results["standart_error_Relative_Risk_WalterS"] = round(standart_error_Relative_Risk_Walters, 7)

    results["Confidence Intervals (Katz)"] = f"({round(lower_rr_kats, 4)}, {round(upper_rr_kats, 4)})"
    results["Confidence (Walters)"] = f"({round(lower_rr_walters, 4)}, {round(upper_rr_walters, 4)})"
    results["Confidence Intervals (Koopman)"] = around(CI_Koopman, 4)
    results["Confidence Intervals Inverse Sine"] = f"({round(Lower_Sinh_, 4)}, {round(Upper_Sinh_, 4)})"
    results["Confidence Intervals Inverse Sine (Adjusted)"] = f"({round(Lower_Sinh_adjusted, 4)}, {round(Upper_Sinh_adjusted, 4)})"
    
    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str


def risk_measures_of_effect_size_two_indpendent_samples (sample_size_1, sample_size_2, proportion_sample_1, proportion_sample_2,difference_in_population, confidence_level): 
    p1 = proportion_sample_1 * sample_size_1
    p2 = proportion_sample_2 * sample_size_2
 
    sample_size = sample_size_1 + sample_size_2 

    # Risk Measures and Ratios
    Risk_Difference = proportion_sample_1-proportion_sample_2 #Samples Difference / Atributional Risk / Excess Risk / Risk Reduction / Absoloute Risk Reduction
    Relative_Risk = proportion_sample_1 / proportion_sample_2
    Population_Attributional_Risk = ((p1/(p1+p2)) * ((Relative_Risk-1) /Relative_Risk)) 
    NNT = 1 / (proportion_sample_1-proportion_sample_2)
    NNH = 1 / (proportion_sample_2-proportion_sample_1)
    Incidental_Rate_Exposed = proportion_sample_1
    Incidental_Rate_UnExposed = proportion_sample_2
    Incidental_Rate_Population = (p1+p2)/sample_size

    Risk_Differnce = proportion_sample_1 - proportion_sample_2
    Risk_Differnce_percentages = Risk_Differnce * 100
    Exposed_Attributable_Fraction =  Risk_Differnce / Incidental_Rate_Exposed
    Exposed_Attributable_Fraction_percentages = Exposed_Attributable_Fraction * 100
    Population_attributable_risk_percentages = Population_Attributional_Risk 
    Population_Attributable_Fraction = (Population_Attributional_Risk / Incidental_Rate_Population) / 100
    Population_Attributable_Fraction_percentages = Population_Attributable_Fraction 

    
    results = {}
    
    results["Incidental Rate Exposed"] = round((Incidental_Rate_Exposed), 4)
    results["Incidental Rate UnExposed"] = round((Incidental_Rate_UnExposed), 4)
    results["Incidental Rate Population"] = round((Incidental_Rate_Population), 4)

    results["Risk Difference (Absolute Risk Reduction)"] = round((proportion_sample_1 - proportion_sample_2), 4)
    results["Risk Difference (%)"] = round(abs(Risk_Differnce_percentages), 4)
    results["Exposed Attributable Fraction"] = round((Exposed_Attributable_Fraction), 4)
    results["Exposed_Attributable_Fraction (%)"] = round((abs(Exposed_Attributable_Fraction_percentages)), 4)

    results["Population Atributable Risk"] = round((Population_Attributional_Risk), 4)
    results["Population Atributable Risk (%) "] = round((abs(Population_attributable_risk_percentages*100)), 4)
    results["Population Atributable Fraction (Relative Risk Reduction) "] = round((Population_Attributable_Fraction), 4)
    results["Population Atributable Fraction (%) "] = round((abs(Population_Attributable_Fraction_percentages*100)), 4)

    results["Number Needed to Treat (NNT)"] = round(NNT, 7)
    results["Number Needed to Harm (NNH)"] = round(NNH, 7)

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

# Effect Sizes Categorical_Correlations
def Nominal_Variables_Correlation(contingency_table, confidence_level = 0.95):

    # To do 
    # add the standrd error of cramers  and for phi

    #1. Phi/Cramer's v abased on chi square
    #coloumn_1, coloumn_2 = Wide_to_Long_2_coloumns(coloumn_1,coloumn_2)
    sample_size_1 = len(x)
    sample_size_2 = len(y)
    sample_size = sample_size_1+sample_size_2
    number_of_rows= np.array(contingency_table).shape[0]
    number_of_coloumns= np.array(contingency_table).shape[1]

    Chisquare_test = scipy.stats.chi2_contingency(contingency_table, correction=False)
    Chisquare_test_corrected = scipy.stats.chi2_contingency(contingency_table) #This one is with Yates correction

    chi_square_statistic = Chisquare_test[0]
    chi_square_p_value = Chisquare_test[1]
    chi_square_statistic_corrected = Chisquare_test_corrected[0]
    chi_square_p_value_corrected = Chisquare_test_corrected[1]

    Phi = np.sqrt(chi_square_statistic/sample_size)
    Phi_corrected = np.sqrt(chi_square_statistic_corrected/sample_size)

    #2. Contingency  Coefficient and adjusgted contingency coefficient
    # adjusted is for situations in which CC can't reach 1

    # in a 2x2 max CC and CC are equal  
    Contingency_Coefficient = np.sqrt((chi_square_statistic /  (chi_square_statistic+sample_size)))
    k = min(number_of_rows,number_of_coloumns)
    max_contingency_coefficaient = np.sqrt((k-1)/k)
    adjusted_Contingency_Coefficient = Contingency_Coefficient/max_contingency_coefficaient

    #3. Tetrechoric correlation
    p1 = int(contingency_table[0][0])
    p2 = int(contingency_table[0][1])
    q1 = int(contingency_table[1][0])
    q2 = int(contingency_table[1][1])
    Tetrechoric_correlation = np.cos(3.14 / (np.sqrt( (q1*p2) / (p1*q2) ) + 1))

    # 4. loglikelihood
    LR, Pval_LR = scipy.stats.chi2_contingency(contingency_table, correction=False, lambda_ = 0)[0:2]
    MOD_LR, Pval_Mod_LR = scipy.stats.chi2_contingency(contingency_table, correction=False, lambda_ = -1)[0:2]

    # 5. Tschuprov's T
    Tschuprovs_T = chi_square_statistic/ (sample_size*(number_of_rows-1)*(number_of_coloumns-1))
    Tschuprovs_T_corrected = chi_square_statistic_corrected / (sample_size*(number_of_rows-1)*(number_of_coloumns-1))


    # 1. uncertinty coefficient
    Uncertainty_Coefficient_Output = uncertainty_coefficient(contingency_table, confidence_level)
        
    # 2.lambda
    Goodman_Krushkal_Lambda_Output = goodman_kruskal_lamda_correlation(contingency_table, confidence_level)
    
    # 3. tau         
    Goodman_Krushkal_Tau_Output = Goodman_Kruskal_Tau(contingency_table, confidence_level)


    #more options:

    #4.1 modified log likelihood (G-test)
    #mod_log_likelihood = scipy.stats.chi2_contingency(contingency_table, correction=False, lambda_ = -1)[0]

    #4.2 freeman tukey 
    #tukey_freeman = scipy.stats.chi2_contingency(contingency_table, correction=False, lambda_ = -0.5)[0]

    #4.3 Cressie-Read
    #Cressie_Read = scipy.stats.chi2_contingency(contingency_table, correction=False, lambda_ = "cressie-read")[0]
    
    #4.4 Neyman 
    #Neyman = scipy.stats.chi2_contingency(contingency_table, correction=False, lambda_ = -2)[0]
    

    results = {}

    results["Chi Square"] = round(chi_square_statistic, 4)
    results["p.value"] = round(chi_square_p_value, 4)
    results["Chi Square (With Yates Correcteion)"] = round(chi_square_statistic_corrected, 4)
    results["p.value_(With Yates Correction)"] = round(chi_square_p_value_corrected, 4)
    results["Likelihood Ratio"] = round(LR, 4)
    results["p.value Likelihood Ratio"] = round(Pval_LR, 4)
    results["Phi/Cramer's V"] = round(Phi, 4)
    results["Phi/Cramer's V Corrected"] = round(Phi_corrected, 4)
    results["Maximum Correcte Cramer's V"] = round(Phi_corrected, 4)
    results["Tschuprovs T"] = round(Tschuprovs_T, 4)
    results["Tschuprovs T Corrected"] = round(Tschuprovs_T_corrected, 4)
    results["Contingency Coefficient"] = round(Contingency_Coefficient, 4)
    results["Contingency Coefficient"] = round(adjusted_Contingency_Coefficient, 4)
    results["Tetrechoric_correlation's v corrected"] = round(Tetrechoric_correlation, 4)
    results["Goodman Kruskal Lambda Table"] = Goodman_Krushkal_Lambda_Output
    results["Goodman Kruskal Tau Table"] = Goodman_Krushkal_Tau_Output
    results["Uncrtinty Coefficeint Table"] = Uncertainty_Coefficient_Output


    #results["tukey_freeman"] = round(tukey_freeman, 4)
    #results["mod_log_likelihood"] = round(mod_log_likelihood, 4)
    #results["Cressie_Read"] = round(Cressie_Read, 4)
    #results["Neyman"] = round(Neyman, 4)

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str



 

def effect_sizes_difference_two_dependent_proportions_2x2(yes_yes,yes_no,no_yes,no_no, confidence_level):

    # TO DO 
    # Consider adding the log odd ratios
    #adding +0.5 to deal with zeros
    #Check Panello 2010 for multiple Proportions

    sample_size = yes_yes + yes_no + no_yes + no_no
    Delta = 0.95     #This is just for Debugging

    # p1total = The number of peopole that said yes in Variable 1 (including yes1 and no1 and yes1 and no2)
    # p2total = The number of peopole that said yes in Variable 2 (including yes2 and no1 and yes2 and no2)
    p1total = yes_yes + yes_no
    p2total = yes_yes + no_yes
    Proportion_Sample_1 = p1total / sample_size
    Proportion_Sample_2 = p2total / sample_size
    Proportion_Matching = (yes_yes + no_no) / sample_size
    Proportion_Not_Matching = 1 - Proportion_Matching
    contingency_table = [[yes_yes, yes_no], [no_yes, no_no]]
    x = np.array(contingency_table).flatten()

    #These are the counts of the sum rows and coloumns of the 2X2 matrix
    p1 = yes_yes+yes_no; p2=yes_yes+no_yes; q1 = no_yes+no_no; q2=yes_no+no_no

    #Difference Between Proportions and confidence intervals
    difference_between_proportions = Proportion_Sample_1 - Proportion_Sample_2

    # Confidence Intervals
    z_critical_value = st.norm.ppf(confidence_level + ((1 - confidence_level) / 2))

    # 1. Wald CI's
    Standard_Error_Wald = np.sqrt((yes_no + no_yes) - ((yes_no - no_yes)**2) / sample_size) / sample_size
    LowerCi_WALD = max(difference_between_proportions - z_critical_value *Standard_Error_Wald,-1 )
    UpperCi_WALD = min(difference_between_proportions + z_critical_value *Standard_Error_Wald,1 )

    # 2. Wald with CC correction (Fleiss et al., 2003)
    Standard_Error_Wald_Corrected = np.sqrt((yes_no + no_yes) - ((yes_no - no_yes)**2) / sample_size) / sample_size
    LowerCi_WALD_Corrected = max(difference_between_proportions - z_critical_value *Standard_Error_Wald_Corrected - (1/sample_size),-1 )
    UpperCi_WALD_Corrected = min(difference_between_proportions + z_critical_value *Standard_Error_Wald_Corrected + (1/sample_size),1 )
    
    # 3. Wald with Yates correction
    Standard_Error_Wald_Corrected_yates = np.sqrt((yes_no + no_yes) - ((yes_no - no_yes - 1)**2) / sample_size) / sample_size
    LowerCi_WALD_Corrected_Yates = max(difference_between_proportions - z_critical_value *Standard_Error_Wald_Corrected_yates,-1 )
    UpperCi_WALD_Corrected_Yates = min(difference_between_proportions + z_critical_value *Standard_Error_Wald_Corrected_yates,1 )
    
    # 4. Agresti & Min (2005)
    Standard_Error_AM =np.sqrt(((yes_no+0.5) + (no_yes+0.5)) - (((yes_no+0.5) - (no_yes+0.5))**2) / (sample_size+2)) / (sample_size+2)
    LowerCi_AM = max(((yes_no+0.5) - (no_yes+0.5)) /  (sample_size + 2) - z_critical_value * Standard_Error_AM,-1 )
    UpperCi_AM = min(((yes_no+0.5) - (no_yes+0.5)) /  (sample_size + 2) + z_critical_value * Standard_Error_AM, 1 )

    # 5. Bonett & Price (2005)
    p1_adjusted = (yes_no + 1) / (sample_size + 2)
    p2_adjusted = (no_yes + 1) / (sample_size + 2)
    Standard_Error_BP = np.sqrt((p1_adjusted+p2_adjusted-(p2_adjusted-p1_adjusted)**2)/(sample_size+2))
    LowerCi_BP = max(p1_adjusted - p2_adjusted - z_critical_value * Standard_Error_BP,-1 )
    UpperCi_BP = min(p1_adjusted - p2_adjusted + z_critical_value * Standard_Error_BP,1 )

    # 6. Newcomb, Square and Add
    
    A1 = (2 * sample_size * ((p1total)/sample_size) + z_critical_value**2) / (2 * sample_size + 2 * z_critical_value**2)
    B1 = (z_critical_value * np.sqrt(z_critical_value**2 + 4 * sample_size * (p1total/sample_size) * (1 - (p1total/sample_size)))) / (2 * sample_size + 2 * z_critical_value**2)  
    A2 = (2 * sample_size * ((p2total)/sample_size) + z_critical_value**2) / (2 * sample_size + 2 * z_critical_value**2)
    B2 = (z_critical_value * np.sqrt(z_critical_value**2 + 4 * sample_size * (p2total/sample_size) * (1 - (p2total/sample_size)))) / (2 * sample_size + 2 * z_critical_value**2)
    lower_p1 = A1 - B1
    upper_p1 = A1 + B1
    lower_p2 = A2 - B2
    upper_p2 = A2 + B2

    if p1total == 0 or p2total == 0 or (sample_size-p1total) == 0 or (sample_size - p2total) == 0:
             products_correction= 0
    else:
        marginals_product = p1total*p2total*(sample_size-p1total)*(sample_size-p2total)
        cells_product = yes_yes*no_no - no_yes*no_yes
        if cells_product > sample_size / 2:
            products_correction = (cells_product - sample_size / 2) / np.sqrt(marginals_product)
        elif cells_product >= 0 and cells_product <= sample_size / 2:
            products_correction = 0
        else:
            products_correction = cells_product / np.sqrt(marginals_product)

    LowerCi_newcomb = difference_between_proportions - np.sqrt((Proportion_Sample_1 - lower_p1)**2 + (upper_p2 - Proportion_Sample_2)**2 - 2 * products_correction * (Proportion_Sample_1 - lower_p1) * (upper_p2 - Proportion_Sample_2))
    UpperCi_newcomb = difference_between_proportions + np.sqrt((Proportion_Sample_2 - lower_p2)**2 + (upper_p1 - Proportion_Sample_1)**2 - 2 * products_correction * (Proportion_Sample_2 - lower_p2) * (upper_p1 - Proportion_Sample_1))
    
    # Significance

    # Mcnemar Test (2x2)
    exact_mcnemar = mcnemar(contingency_table, exact=True)

    mcnemar_chi_square = ((abs(yes_no - no_yes))**2) / (yes_no+no_yes)
    mcnemar_chi_square_corrected = ((abs(yes_no - no_yes) - 1)**2) / (yes_no+no_yes)

    mcnemar_pvalue = calculate_p_value_from_chi_score(mcnemar_chi_square,1)
    mcnemar_pvalue_corrected = calculate_p_value_from_chi_score(mcnemar_chi_square_corrected,1)

    # Optional - Can check this built in Functions for more version of the mcnemar test
    #data = [[p1, q1], [p2, q2]]
    #a = (mcnemar(data, exact=True))
    #b = (mcnemar(data, exact=False, correction=False))
    #c = (mcnemar(data, exact=False, correction=True))


    # Matched Pairs Odds Ratio
    ##########################
    # 1+ 2 Matched Pairs Odds Ratio and adjusted matched pairs odds ratio
    Match_Pairs_Odds_Ratio = yes_no / no_yes
    Match_Pairs_Odds_Ratio_Adjusted = (yes_no / (no_yes+1))

    #More rare adjusments for the matched pairs odds ratio (Jeweel, 1984) - fits small samples
    #3. Jacknifhe odds ratio
    Odds_Ratio_Matched_Pairs_Jackknife = Match_Pairs_Odds_Ratio - ((sample_size - 1) / (no_yes-1)) * (Match_Pairs_Odds_Ratio/sample_size)
    Odds_Ratio_Bias_Corrected = Match_Pairs_Odds_Ratio * (1-no_yes**-1)

    # Confidence Intervals for the matched pairs Odds Ratio
    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)

    # 1. Rigby & Robinsone (2000)
    CI_Mcnemar_odds_ratio_lower = Match_Pairs_Odds_Ratio**(1-zcrit/np.sqrt(mcnemar_chi_square))
    CI_Mcnemar_odds_ratio_upper = Match_Pairs_Odds_Ratio**(1+zcrit/np.sqrt(mcnemar_chi_square))
    
    CI_Mcnemar_corrected_odds_ratio_lower = Match_Pairs_Odds_Ratio**(1-zcrit/np.sqrt(mcnemar_chi_square_corrected))
    CI_Mcnemar_corrected_odds_ratio_upper = Match_Pairs_Odds_Ratio**(1+zcrit/np.sqrt(mcnemar_chi_square_corrected))

    # 2. Logarithmic (find the source)
    Standrd_Error_Odds_ratio = np.sqrt(1/yes_no + 1/no_yes) 
    CI_odds_ratio_log_lower = Match_Pairs_Odds_Ratio * math.exp(-zcrit*Standrd_Error_Odds_ratio)
    CI_odds_ratio_log_upper = Match_Pairs_Odds_Ratio * math.exp(zcrit*Standrd_Error_Odds_ratio)

    # 3. Binomial Methods - basicly transfer the proportions to odds ratio by (p1/(1-p1)) 
    # Note: Delta=Logit, Score = Wilson, IM= Clopper-Pearson, Fiducial = Jeffereys - These are the names in Chen et al., 2020

    # 4. The Score Method also known as the Wilson score Method
    A = (2*yes_no*no_yes) + zcrit**2*(yes_no+no_yes)
    lower_ci_score_method = (A - np.sqrt(A**2 - ((2*yes_no*no_yes)**2))) / (2*no_yes**2)
    upper_ci_score_method = (A + np.sqrt(A**2 - ((2*yes_no*no_yes)**2))) / (2*no_yes**2)


    #Matched Pairs Relative Risk:
    Matched_Pairs_Relative_Risk = (yes_yes+yes_no)/ (yes_yes+no_yes)
    sample_size = yes_yes+yes_no+no_no+no_yes
    Realtive_risk_unbiased = (p1*(q1+q2+1)) / (((p1+p2) * (q1 + 1)))
    Realtive_risk_adjusted_Walters = np.exp(np.log((p1+0.5) / (sample_size+0.5)) - np.log((p2+0.5) / (sample_size+0.5)))

    # Statistics, Standard Errors and Confidence Intervals for the rate ratio(Relative Risk)
    #################################################################################

    #Function based on ratesci
    def Tang_T1_function(x, Delta): 
        N = np.sum(x)
        Stheta = ((x[1] + x[0]) - (x[2] + x[0]) * Delta)
        A = N * (1 + Delta)
        B = (x[0] + x[2]) * Delta**2 - (x[0] + x[1] + 2 * x[2])
        C_ = x[2] * (1 - Delta) * (x[0] + x[1] + x[2]) / N
        num = -B + np.sqrt(B**2 - 4 * A * C_)
        q21 = num / (2 * A)
        Variance = np.maximum(0, N * (1 + Delta) * q21 + (x[0] + x[1] + x[2]) * (Delta - 1))
        Z_Score_Tang = Stheta / np.sqrt(Variance)
        return Z_Score_Tang, np.sqrt(Variance)
    
    def Tang_CI_Function(Function, Maximum_Iterations=100, CI = "lower_CI"):
        hi = 1; lo = -1; niter = 1
        while niter <= Maximum_Iterations:
            mid = max(-1, min(1, (hi + lo) / 2))
            scor = Function(np.tan(np.pi * (mid + 1) / 4))
            check = (scor <= 0) or np.isnan(scor)
            hi = mid if check else hi
            lo = mid if not check else lo
            niter += 1
        Optimize = lo if CI == "lower_CI" else hi
        return np.tan((Optimize + 1) * np.pi / 4)

    def myfun(Delta):
        return Tang_T1_function(Delta=Delta, x=x)[0]
    Statistic_Tang, Standard_Deviation_Tang = Tang_T1_function(Delta=Delta, x=x)
    Standard_Error_Tang = Standard_Deviation_Tang / np.sqrt((sample_size-no_no)**2)
    P_Val_Tang = calculate_p_value_from_z_score(Statistic_Tang)
    CI_Tang_lower = Tang_CI_Function(lambda Delta: myfun(Delta) - zcrit, CI="lower_CI")
    CI_Tang_Upper = Tang_CI_Function(lambda Delta: myfun(Delta) + zcrit, CI="upper_CI")

    # 2. Delta Method
    Standard_Error_Delta = np.sqrt((yes_yes+yes_no)*(yes_no+no_yes)/(yes_yes+no_yes)**3)
    Statistic_Delta = (Matched_Pairs_Relative_Risk - Delta) / Standard_Error_Delta
    P_Val_Delta = calculate_p_value_from_z_score(Statistic_Delta)
    CI_delta_lower = Matched_Pairs_Relative_Risk - Standard_Error_Delta * zcrit
    CI_delta_Upper = Matched_Pairs_Relative_Risk + Standard_Error_Delta * zcrit

    # 3. Delta Method Modified
    Standard_Error_Delta_modified = np.sqrt(Delta * (yes_no+no_yes)/(yes_yes+no_yes)**2)
    Statistic_Delta_modified = (Matched_Pairs_Relative_Risk - Delta) / Standard_Error_Delta_modified
    P_Val_Delta_modified = calculate_p_value_from_z_score(Statistic_Delta_modified)
    CI_delta_lower_modified = Matched_Pairs_Relative_Risk - Standard_Error_Delta_modified * zcrit
    CI_delta_Upper_modified = Matched_Pairs_Relative_Risk + Standard_Error_Delta_modified * zcrit

    # 4. Katz Logarithmic
    Standard_Error_Katz = np.sqrt((no_yes+yes_no)/((yes_yes+yes_no)*(yes_yes+no_yes)))
    Statistic_Katz = (math.log(yes_no+yes_yes) - math.log(yes_yes+no_yes) - math.log(Delta)) / Standard_Error_Katz
    P_Val_Katz = calculate_p_value_from_z_score(Statistic_Katz)
    CI_katz_lower = Matched_Pairs_Relative_Risk - Standard_Error_Katz * zcrit
    CI_katz_Upper = Matched_Pairs_Relative_Risk + Standard_Error_Katz * zcrit

    # 5. Katz logarithmic Modified
    Standard_Error_Katz_Modified = np.sqrt((yes_no + no_yes) / (Delta*(yes_yes+no_yes)**2))
    Statistic_Katz_Modified = (math.log(yes_no+yes_yes) - math.log(yes_yes+no_yes) - math.log(Delta))  / Standard_Error_Katz_Modified
    P_Val_Katz_Modified = calculate_p_value_from_z_score(Statistic_Katz_Modified)
    CI_katz_modified_lower = Matched_Pairs_Relative_Risk - Standard_Error_Katz_Modified * zcrit
    CI_katz_modified_Upper = Matched_Pairs_Relative_Risk + Standard_Error_Katz_Modified * zcrit

    # 6. Lachenbruch and Lynch
    Standard_Deviation_Lachenbruch_Lynch = np.sqrt((yes_no*(sample_size-yes_no) + yes_yes*(sample_size-yes_yes)*(1-Delta)**2 + no_yes*(sample_size-no_yes)*Delta**2 + 2*yes_no*no_yes*Delta-2*yes_yes*yes_no*(1-Delta) + 2*yes_yes*no_yes*Delta*(1-Delta))/ sample_size)
    Standard_Error_Lachenbruch_Lynch = Standard_Deviation_Lachenbruch_Lynch / np.sqrt((sample_size-no_no)**2)
    Statistic_Lachenbruch_Lynch = ((yes_yes+yes_no) - Delta*(yes_yes+no_yes)) / Standard_Deviation_Lachenbruch_Lynch
    P_Val_Lachenbruch_Lynce = calculate_p_value_from_z_score(Statistic_Lachenbruch_Lynch)
    CI_Lachenbruch_Lynce_lower = Matched_Pairs_Relative_Risk - Standard_Error_Lachenbruch_Lynch * zcrit
    CI_Lachenbruch_Lynce_Upper = Matched_Pairs_Relative_Risk + Standard_Error_Lachenbruch_Lynch * zcrit

    # Other Relative Measures
    Population_Attributional_Risk = ((p1/(p1+p2)) * ((Matched_Pairs_Relative_Risk-1) /Matched_Pairs_Relative_Risk)) 
    NNT = 1 / (Proportion_Sample_1-Proportion_Sample_2)
    NNH = 1 / (Proportion_Sample_2-Proportion_Sample_1)
    Incidental_Rate_Exposed = Proportion_Sample_1
    Incidental_Rate_UnExposed = Proportion_Sample_2
    Incidental_Rate_Population = (p1+p2)/sample_size

    Risk_Differnce = Proportion_Sample_1 - Proportion_Sample_2
    Risk_Differnce_percentages = Risk_Differnce * 100
    Exposed_Attributable_Fraction =  Risk_Differnce / Incidental_Rate_Exposed
    Exposed_Attributable_Fraction_percentages = Exposed_Attributable_Fraction * 100
    Population_attributable_risk_percentages = Population_Attributional_Risk 
    Population_Attributable_Fraction = (Population_Attributional_Risk / Incidental_Rate_Population) / 100
    Population_Attributable_Fraction_percentages = Population_Attributable_Fraction 


    results = {}
    
    results["Proportion of Sucess of Variable 1"] = round(Proportion_Sample_1, 7)
    results["Proportion of Sucess of Variable 2"] = round(Proportion_Sample_2, 7)
    results["Proportion Matching"] = round(Proportion_Matching, 7)
    results["Proportion not Matching"] = round(Proportion_Not_Matching, 7)
    
    results["Difference Between Proportions"] = round(difference_between_proportions, 7)
    results["Confidence Intervals Wald"] = f"({round(LowerCi_WALD, 4)}, {round(UpperCi_WALD, 4)})"
    results["Confidence Intervals Wald Corrected (Edwards)"] = f"({round(LowerCi_WALD_Corrected, 4)}, {round(UpperCi_WALD_Corrected, 4)})"
    results["Confidence Intervals Wald Corrected (Yates)"] = f"({round(LowerCi_WALD_Corrected_Yates, 4)}, {round(UpperCi_WALD_Corrected_Yates, 4)})"
    results["Confidence Intervals adjusted (Agresti & Min, 2005)"] = f"({round(LowerCi_AM, 4)}, {round(UpperCi_AM, 4)})"
    results["Confidence Intervals adjusted (Bonett & Price, 2012)"] = f"({round(LowerCi_BP, 4)}, {round(UpperCi_BP, 4)})"
    results["Confidence Intervals (NewComb)"] = f"({round(LowerCi_newcomb, 4)}, {round(UpperCi_newcomb, 4)})"

    results["McNemar Chi Square Statistic"] = f"({round(mcnemar_chi_square, 4)})"
    results["McNemar Chi Square Statistic Corrected)"] = f"({round(mcnemar_chi_square_corrected, 4)})"
    results["McNemar pvalue"] = f"({round(mcnemar_pvalue, 4)})"
    results["McNemar pvalue corrected)"] = f"({round(mcnemar_pvalue_corrected, 4)})"
    results["McNemar Exact)"] = f"({(exact_mcnemar)})"

    results["Matched Pairs Odds Ratio)"] = f"({round(Match_Pairs_Odds_Ratio, 4)})"
    results["Matched Pairs Odds Ratio (Adjusted)"] = f"({round(Match_Pairs_Odds_Ratio_Adjusted, 4)})"
    results["Matched Pairs Odds Ratio (Jacknife Correction)"] = f"({round(Odds_Ratio_Matched_Pairs_Jackknife, 4)})"
    results["Matched Pairs Odds Ratio)"] = f"({round(Odds_Ratio_Bias_Corrected, 4)})"

    results["Matched Pairs Odds Ratio CI's (Rigby & Robinsone)"] = f"({round(CI_Mcnemar_odds_ratio_lower, 4)}, {round(CI_Mcnemar_odds_ratio_upper, 4)})"
    results["Matched Pairs Odds Ratio Corrected CI's (Rigby & Robinsone)"] = f"({round(CI_Mcnemar_corrected_odds_ratio_lower, 4)}, {round(CI_Mcnemar_corrected_odds_ratio_upper, 4)})"
    
    # All these confidence intervals are also computed for one sample proportion
    results["Matched Pairs Odds Ratio CI's (Logarithmic / Delta Method)"] = f"({round(CI_odds_ratio_log_lower, 4)}, {round(CI_odds_ratio_log_upper, 4)})"
    results["Matched Pairs Odds Ratio CI's Score Method"] = f"({round(lower_ci_score_method, 4)}, {round(upper_ci_score_method, 4)})"

    results["Matched Pairs Relative Risk"] = f"({round(Matched_Pairs_Relative_Risk, 4)})"
    results["Matched Pairs Relative Risk (Unbiased)"] = f"({round(Realtive_risk_unbiased, 4)})"
    results["Matched Pairs Relative Risk (Adjusted Walters)"] = f"({round(Realtive_risk_adjusted_Walters, 4)})"
    
    results["Rate Ratio Z-score (Tang T1 Method)"] = round(Statistic_Tang, 4)
    results["Rate Ratio Z-score (Delta Method)"] = round(Statistic_Delta, 4)
    results["Rate Ratio Z-score (Delta Method Modified)"] = round(Statistic_Delta_modified, 4)
    results["Rate Ratio Z-score (Katz Logarithmic)"] = round(Statistic_Katz, 4)
    results["Rate Ratio Z-score (Katz Logarithmic Modified)"] = round(Statistic_Katz_Modified, 4)
    results["Rate Ratio Z-score (Lachenbruch and Lynch)"] = round(Statistic_Lachenbruch_Lynch, 4)

    results["Rate Ratio P-value (Tang T1 Method)"] = round(P_Val_Tang, 4)
    results["Rate Ratio P-value (Delta Method)"] = round(P_Val_Delta, 4)
    results["Rate Ratio P-value (Delta Method Modified)"] = round(P_Val_Delta_modified, 4)
    results["Rate Ratio P-value (Katz Logarithmic)"] = round(P_Val_Katz, 4)
    results["Rate Ratio P-value (Katz Logarithmic Modified)"] = round(P_Val_Katz_Modified, 4)
    results["Rate Ratio P-value (Lachenbruch and Lynch)"] = round(P_Val_Lachenbruch_Lynce, 4)

    results["Rate Ratio CI's (Tang T1 Method)"] = f"({round(CI_Tang_lower, 4)}, {round(CI_Tang_Upper, 4)})"
    results["Rate Ratio CI's (Delta Method)"] = f"({round(CI_delta_lower, 4)}, {round(CI_delta_Upper, 4)})"
    results["Rate Ratio CI's (Delta Method Modified)"] = f"({round(CI_delta_lower_modified, 4)}, {round(CI_delta_Upper_modified, 4)})"
    results["Rate Ratio CI's (Katz Logarithmic)"] = f"({round(CI_katz_lower, 4)}, {round(CI_katz_Upper, 4)})"
    results["Rate Ratio CI's (Katz Logarithmic Modified)"] = f"({round(CI_katz_modified_lower, 4)}, {round(CI_katz_modified_Upper, 4)})"
    results["Rate Ratio CI's (Lachenbruch and Lynch)"] = f"({round(CI_Lachenbruch_Lynce_lower, 4)}, {round(CI_Lachenbruch_Lynce_Upper, 4)})"

    results["Rate Ratio Standard Error (Tang T1 Method)"] = round(Standard_Error_Tang, 4)
    results["Rate Ratio Standard Error (Delta Method)"] = round(Standard_Error_Delta, 4)
    results["Rate Ratio Standard Error (Delta Method Modified)"] = round(Standard_Error_Delta_modified, 4)
    results["Rate Ratio Standard Error (Katz Logarithmic)"] = round(Standard_Error_Katz, 4)
    results["Rate Ratio Standard Error (Katz Logarithmic Modified)"] = round(Standard_Error_Katz_Modified, 4)
    results["Rate Ratio Standard Error (Lachenbruch and Lynch)"] = round(Standard_Error_Lachenbruch_Lynch, 4)

    results["Population Attributional Risk"] = round(Population_Attributional_Risk, 4)
    results["Number Needed to Treat (NNT)"] = round(NNT, 4)
    results["Number Needed to Harm (NNH)"] = round(NNH, 4)
    results["Incidental Rate Exposed"] = round(Incidental_Rate_Exposed, 4)
    results["Incidental Rate Unexposed"] = round(Incidental_Rate_UnExposed, 4)
    results["Incidental Rate Population"] = round(Incidental_Rate_Population, 4)


    results["Risk Difference (Absolute Risk Reduction)"] = round((Proportion_Sample_1 - Proportion_Sample_2), 4)
    results["Risk Difference (%)"] = round(abs(Risk_Differnce_percentages), 4)
    results["Exposed Attributable Fraction"] = round((Exposed_Attributable_Fraction), 4)
    results["Exposed_Attributable_Fraction (%)"] = round((abs(Exposed_Attributable_Fraction_percentages)), 4)

    results["Population Atributable Risk"] = round((Population_Attributional_Risk), 4)
    results["Population Atributable Risk (%) "] = round((abs(Population_attributable_risk_percentages*100)), 4)
    results["Population Atributable Fraction (Relative Risk Reduction) "] = round((Population_Attributable_Fraction), 4)
    results["Population Atributable Fraction (%) "] = round((abs(Population_Attributable_Fraction_percentages*100)), 4)

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

#Cochran todo = it only works with 1 and 0 so change the first value in data to sucess (1) and the second failure (0)
def Cochran_Q_based_Effect_Size(Final_Data):
    sample_size = len(Final_Data)

    Variables_Number = Final_Data.shape[1]
    Degrees_Of_Freedom = Variables_Number - 1
    row_sums = np.sum(Final_Data, axis=1)
    Pis = ((1/Variables_Number)*row_sums)
    A = sum(Pis)
    B = sample_size - A
    C = Degrees_Of_Freedom / (2*sum(Pis * (1-Pis)))

    # Display the transposed DataFrame
    Q = cochrans_q(Final_Data).statistic
    pval = cochrans_q(Final_Data).pvalue
    VarianceQ = (Q/C - 2*A*B) / (-sample_size *(sample_size-1))
    MeanQ = (2 / (sample_size*(sample_size-1))) * (A * B - (sum(Pis * (1-Pis))))
    Effect_Size_R = 1 - VarianceQ/MeanQ

    results = {}
    
    results["Cochrans Q"] = round(Q, 7)
    results["Degrees of Freedom"] = round(Degrees_Of_Freedom)
    results["Chocran's Q p-value"] = round(pval, 7)
    results["Variance of Q"] = round(VarianceQ)
    results["Mean Q"] = round(MeanQ, 7)
    results["Chance Corrected Q-based Effect Size (Berry et al., 2010)"] = round(Effect_Size_R, 7)

    return results

def goodness_of_fit_from_frequency(column_1, expected_proportions=None, expected_frequencies=None, expected_ratios=None, confidence_level=None):
    data_series = pd.Series(column_1)
    Observed = data_series.value_counts()
    sample_size = sum(Observed)
    Final_Data = pd.DataFrame({'level name': Observed.index, 'frequency': Observed.values})
    levels_number = Final_Data.shape[0]


    if expected_proportions is not None:
        Expected = np.array(expected_proportions) * sample_size
    elif expected_frequencies is not None:
        Expected = np.array(expected_frequencies)
    elif expected_ratios is not None:
        ratio_sum = sum(expected_ratios)
        Expected = (np.array(expected_ratios) / ratio_sum) * sample_size
    else:
        Expected = np.array([(1 / levels_number)] * levels_number) * sample_size

    degrees_of_freedom = levels_number - 1
    
    # Pearson Chi Square Test
    Chi_square = sum((Observed - Expected)**2 / Expected)
    p_value_chi_square = calculate_p_value_from_chi_score( Chi_square,degrees_of_freedom)
    
    # Wilks_G_Square Test
    Observed_Proportions = Observed / sample_size
    Expected_Proportions = Expected / sample_size

    Wilks_G_Square = 2 *sample_size * (sum(Observed_Proportions * np.log(Observed_Proportions/Expected_Proportions)))


    # Effect Sizes

    # Cohens W
    Cohens_w = np.sqrt(Chi_square / sample_size)

    # maximum-corrected pearson chi_square
    q_chi = min(Expected)
    max_chi_Square = (sample_size*(sample_size-q_chi))/q_chi
    max_corrected_lambda = Chi_square/max_chi_Square
    
    # maximum-corrected wilks g square
    q_wilks = min(Expected_Proportions)
    max_wilks_G = -2*sample_size*np.log(q_wilks)
    max_corrected_gamma = Wilks_G_Square / max_wilks_G

    # Chance_Corrected measure of effect size
    Variance = (1/ levels_number) * (np.sum((np.array(Observed_Proportions) - (Expected_Proportions))**2))
    Mean = (1/levels_number**2) * (np.sum(([(elem1 - elem2)**2 for elem1 in np.array(Observed_Proportions) for elem2 in Expected_Proportions])))
    Chance_Corrected_R = Variance / Mean

    results = {}

    results["Chi Square"] = round(Chi_square, 7)
    results["Degrees of Freedom"] = round(degrees_of_freedom, 7)
    results["p value Chi Square"] = round(p_value_chi_square)
    results["Wilks' G Square"] = round(Wilks_G_Square, 7)
    results["max_corrected_lambda"] = round(max_corrected_lambda,4)
    results["max_corrected_gamma"] = around(max_corrected_gamma,4)
    results["Chance_Corrected_R"] = Chance_Corrected_R
    results["Variance of R"] = around(Variance,4)
    results["Mean of R"] = Mean

    return results


################################################################################
####################       Medians    ##########################################
################################################################################

#### General Functions for Medians

# 1.  Function for confidence intervals for Hodges-Lehmann based on Wilcox test (both within and between subject)
def Hodges_Lehmann_based_on_wilcox_test(x, y, confidence_level=confidence_level):
    alpha = 1-confidence_level
    sample_size_1 = len(x)
    sample_size_2 = len(y) if y is not None else 0
    if y is None:
        def Hodges_Lhemann_Estimation_differences_sample(Value, Z_score):
            Differnce_Vector = x - Value
            sample_size_1 = len(Differnce_Vector)
            Ranked_Data = rankdata(np.abs(Differnce_Vector))
            Difference = np.sum(Ranked_Data[Differnce_Vector > 0]) - sample_size_1 * (sample_size_1 + 1) / 4
            Number_of_Ties = np.histogram(Ranked_Data, bins=np.arange(1, len(Ranked_Data) + 2))[0]
            Variance = np.sqrt(sample_size_1 * (sample_size_1 + 1) * (2 * sample_size_1 + 1) / 24 - np.sum((Number_of_Ties ** 3 - Number_of_Ties) / 48))
            return Difference / Variance - Z_score
        UpperCi = root_scalar(Hodges_Lhemann_Estimation_differences_sample, bracket=(min(x), max(x)), xtol=1e-4, args=(norm.ppf((1-confidence_level) / 2),)).root
        LowerCi = root_scalar(Hodges_Lhemann_Estimation_differences_sample, bracket=(min(x), max(x)), xtol=1e-4, args=(norm.ppf(1 - (1-confidence_level) / 2),)).root
        Hodges_Lehmann_Estimator = root_scalar(Hodges_Lhemann_Estimation_differences_sample, bracket=(min(x), max(x)), xtol=1e-4, args=(0,)).root
    
    else:
        alpha = 1 - confidence_level
        Minimum_Value = min(x) - max(y)
        Maximum_Value = max(x) - min(y)
        def Hodges_Lhemann_Estimation_two_samples(Value, Zscore):
            Ranked_Data = rankdata(np.concatenate([x - Value, y]))
            Number_of_Ties = np.histogram(Ranked_Data, bins=np.arange(1, len(Ranked_Data) + 2))[0]
            Difference = (np.sum(Ranked_Data[:sample_size_1]) - sample_size_1 * (sample_size_1 + 1) / 2 - sample_size_1 * sample_size_2 / 2)
            Varaince = np.sqrt((sample_size_1 * sample_size_2 / 12) * ((sample_size_1 + sample_size_2 + 1) - np.sum((Number_of_Ties ** 3 - Number_of_Ties) / ((sample_size_1 + sample_size_2) * (sample_size_1 + sample_size_2 - 1)))))
            return Difference / Varaince - Zscore
        LowerCi = root_scalar(Hodges_Lhemann_Estimation_two_samples, bracket=(Minimum_Value, Maximum_Value), xtol=1e-4, args=(norm.ppf(1 - alpha / 2),)).root
        UpperCi = root_scalar(Hodges_Lhemann_Estimation_two_samples, bracket=(Minimum_Value, Maximum_Value), xtol=1e-4, args=(norm.ppf(alpha / 2),)).root
        Hodges_Lehmann_Estimator = root_scalar(Hodges_Lhemann_Estimation_two_samples, bracket=(Minimum_Value, Maximum_Value), xtol=1e-4, args=(0,)).root
    return Hodges_Lehmann_Estimator, LowerCi, UpperCi

def hltest(x, y, confidence_level = confidence_level):
    res = Hodges_Lehmann_based_on_wilcox_test(x, y, confidence_level=confidence_level)
    return res

# 2. Function for the confidence intervels for median difference and ratios using Price and Bonett method
def CI_price_bonett(x,y,confidence_interval):
    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)
    Median_sample_1 = np.median(x)
    sample_size_1 = len(x)
    sorted_x = np.sort(x)
    c1 = np.round((sample_size_1 + 1) / 2 - sample_size_1**0.5)
    X1 = sorted_x[int(sample_size_1 - c1)]
    X2 = sorted_x[int(c1 - 1)]
    Z1 = norm.ppf(1 - binom.cdf(c1 - 1, sample_size_1, 0.5))
    Variance_Sample_1 = (((X1 - X2) / (2 * Z1))**2)

    Median_Sample_2 = np.median(y)
    sample_size_2 = len(y)
    sorted_y = np.sort(y)
    c2 = np.round((sample_size_2 + 1) / 2 - sample_size_2**0.5)
    Y1 = (sorted_y[int(sample_size_2 - c2)])
    Y2 = (sorted_y[int(c2 - 1)])
    Z2 = norm.ppf(1 - binom.cdf(c2 - 1, sample_size_2, 0.5))
    Variance_Sample_2 = ((Y1 - Y2) / (2 * Z2))**2

    Standard_Error_Indpednent_Differnece_Price_Bonett = np.sqrt(Variance_Sample_1 + Variance_Sample_2)

    #### Confidence Intervals ratio of means
    log_X1 = np.log(sorted_x[int(sample_size_1 - c1)])
    log_X2 = np.log(sorted_x[int(c1 - 1)])
    log_Y1 = np.log(sorted_y[int(sample_size_2 - c2)])
    log_Y2 = np.log(sorted_y[int(c2 - 1)])
    log_Z1 = norm.ppf(1 - binom.cdf(c1 - 1, sample_size_1, 0.5))
    log_Variance_Sample_1 = (((log_X1 - log_X2) / (2 * log_Z1))**2)
    log_Z2 = norm.ppf(1 - binom.cdf(c2 - 1, sample_size_2, 0.5))
    log_Variance_Sample_2 = ((log_Y1 - log_Y2) / (2 * log_Z2))**2
    
    log_Standard_Error_Indpednent_Differnece_Price_Bonett = np.sqrt(log_Variance_Sample_1+log_Variance_Sample_2)

    values_lower_than_medians = np.sum((x < Median_sample_1) & (y<Median_Sample_2))
    Median_Probability = (values_lower_than_medians + 0.25) / (sample_size_1 + 1) if sample_size_1 % 2 == 0 else (values_lower_than_medians + 0.25) / sample_size_1
    Covariance_log = (4*Median_Probability - 1)*np.sqrt(log_Variance_Sample_1)*np.sqrt(log_Variance_Sample_2)
    Log_Standard_Error_Within_Ratio = np.sqrt(log_Variance_Sample_1 + log_Variance_Sample_2 - 2*Covariance_log)
    Covariance = (4*Median_Probability - 1)*np.sqrt(Variance_Sample_1)*np.sqrt(Variance_Sample_2)
    Standrd_Error_Within_Difference = np.sqrt(Variance_Sample_1 + Variance_Sample_2 - 2*Covariance)
    ratio_of_medians = Median_sample_1 / Median_Sample_2

    # Confidence Intervals independent difference
    LowerCi_Price_Bonett_between_difference = (Median_sample_1 - Median_Sample_2) - zcrit * Standard_Error_Indpednent_Differnece_Price_Bonett
    UpperCi_Price_Bonett_between_difference = (Median_sample_1 - Median_Sample_2) + zcrit * Standard_Error_Indpednent_Differnece_Price_Bonett

    # Confidence Intervals indpendent ratio_of medians
    LowerCi_Price_Bonett_between_ratio = ratio_of_medians * math.exp(-zcrit * log_Standard_Error_Indpednent_Differnece_Price_Bonett)
    UpperCi_Price_Bonett_between_ratio = ratio_of_medians * math.exp(zcrit * log_Standard_Error_Indpednent_Differnece_Price_Bonett)

    # Confidence Intervals depednent difference
    LowerCi_Price_Bonett_within_difference = (Median_sample_1 - Median_Sample_2) - zcrit * Standrd_Error_Within_Difference
    UpperCi_Price_Bonett_within_difference = (Median_sample_1 - Median_Sample_2) + zcrit * Standrd_Error_Within_Difference

    # Confidence Intervals dependent ratio_of medians
    LowerCi_Price_Bonett_within_ratio = ratio_of_medians * math.exp(-zcrit * Log_Standard_Error_Within_Ratio)
    UpperCi_Price_Bonett_within_ratio = ratio_of_medians * math.exp(zcrit * Log_Standard_Error_Within_Ratio)

    
    results = {}
    
    results["Median Sample 1"] = Median_sample_1
    results["Sample Size 1"] = sample_size_1
    results["Variance Sample 1"] = Variance_Sample_1
    results["Median Sample 2"] = Median_Sample_2
    results["Sample Size 2"] = sample_size_2
    results["Variance Sample 2"] = Variance_Sample_2
    results["Standard Error Independent Difference"] = Standard_Error_Indpednent_Differnece_Price_Bonett
    results["Log Standard Error"] = log_Standard_Error_Indpednent_Differnece_Price_Bonett
    results["Values Lower Than Medians"] = values_lower_than_medians
    results["Median Probability"] = Median_Probability
    results["Covariance (log)"] = Covariance_log
    results["Log Standard Error Within Ratio"] = Log_Standard_Error_Within_Ratio
    results["Covariance"] = Covariance
    results["Standard Error Within Difference"] = Standrd_Error_Within_Difference
    results["Ratio of Medians"] = ratio_of_medians

    results["Lower CI Independent Difference"] = LowerCi_Price_Bonett_between_difference
    results["Upper CI Independent Difference"] = UpperCi_Price_Bonett_between_difference

    results["Lower CI Independent Ratio of Medians"] = LowerCi_Price_Bonett_between_ratio
    results["Upper CI Independent Ratio of Medians"] = UpperCi_Price_Bonett_between_ratio

    results["Lower CI Dependent Difference"] = LowerCi_Price_Bonett_within_difference
    results["Upper CI Dependent Difference"] = UpperCi_Price_Bonett_within_difference

    results["Lower CI Dependent Ratio of Medians"] = LowerCi_Price_Bonett_within_ratio
    results["Upper CI Dependent Ratio of Medians"] = UpperCi_Price_Bonett_within_ratio

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str





def one_sample_median(x, population_median = 0, confidence_level = 0.95):
    Median = np.median(x)
    Mean = np.mean(x)
    absolute_deviations_from_median = abs(x-Median)
    mean_absolute_deviation = np.mean(absolute_deviations_from_median)
    normal_corrected_mean_absolout_deviation = mean_absolute_deviation * 1.2533
    median_absolute_deviation = np.median(absolute_deviations_from_median)
    normal_corrected_median_absoloute_deviation = 1.4826*median_absolute_deviation
    Inter_Quartile_Range = iqr(x)
    Standard_Deviation = np.std(x, ddof = 1)
    sample_size = len(x)
    
    Standrd_Error_Median = np.sqrt(math.pi/2) * (Standard_Deviation/np.sqrt(sample_size)) # Note - there are many other ways to estimate the median - for simplicity, we chose this one
    Standrd_Error_Median_AD = np.sqrt(2/math.pi) * (Standard_Deviation/np.sqrt(sample_size)) # Gia & Hung, 2001
    Standrd_Error_IQR = Standard_Deviation / (2*np.sqrt(sample_size)*0.31777)

    # Another Estimator for the dispertion around median Qn (Rousseeuw and Croux, 1993)
    def pairwise_differences(x):      # a function to calcualte all pairwise comparison
        return [b - a for a, b in itertools.combinations(x, 2)]
    Pairwise_Vector = pairwise_differences(x)
    Qn = 2.2219 * (np.quantile(abs(np.array(Pairwise_Vector)),0.25))

    # biweight standard Error Goldberg & Iglewicz, 1992; Lax, 1985)

    # Effect Sizes 
    # 1. Thompson 2007
    d_Mdn = abs(Median -  population_median) / Standard_Deviation

    # 2. Median Dispersion based measures
    d_Mdn_AD = abs(Median -  population_median) / median_absolute_deviation
    d_Mdn_AD_corrected = abs(Median -  population_median)  / normal_corrected_median_absoloute_deviation
    d_IQR = abs(Median -  population_median)  / Inter_Quartile_Range
    d_Qn = abs(Median -  population_median)  / Qn

    # Signifcance of the median

    # One Sample Median Significant Test
    t_median = (Median -  population_median) / Standrd_Error_Median
    P_value_median = calculate_p_value_from_t_score(t_median, sample_size-1)
    

    # One sample Sign Test
    Sign_Test_vector = np.where(x > population_median, 1, np.where(x < population_median, 0, np.nan))
    K = sum(Sign_Test_vector)
    p_value_sign = binom.cdf(k=K, n=sample_size, p=0.5)
    p_val_wilcoxon = sign_test_wilcoxon_method(x, y=None)


    results = {}

    results["Median"] = round((Median), 4)
    results["Median_Absolute_Deviation (MAD)"] = around((median_absolute_deviation), 4)
    results["Median Absolute Deviation (normal corrected)"] = around((normal_corrected_median_absoloute_deviation), 4)
    results["Mean Absolute_Deviation (MAD)"] = around((mean_absolute_deviation), 4)
    results["Mean Absolute Deviation (normal corrected)"] = around((normal_corrected_mean_absolout_deviation), 4)
    results["Qn Median Estimator"] = around((Qn), 4)
    results["Inter_Quartile_Range"] = around((Inter_Quartile_Range), 4)
    results["Standard_Deviation"] = round((Standard_Deviation), 4)
    results["Standrd_Error_Median"] = round((Standrd_Error_Median), 4)
    
    # Significant Tests
    results["p_value_sign"] = around((p_value_sign), 4)
    results["p_value_median"] = around((P_value_median), 4)


    # Effect Sizes
    results["Delta Median (Thompson, 2007)"] = round((d_Mdn), 4)
    results["Delta Median Absouloute Deviation"] = round((d_Mdn_AD), 4)
    results["Delta Median Absouloute Deviation Corrected"] = round((d_Mdn_AD_corrected), 4)
    results["Delta IQR"] = round((d_IQR), 4)
    results["Delta Qn"] = round((d_Qn), 4)

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

def two_independnet_medians(x,y, confidence_level = 0.95):
    Median_sample_1 = np.median(x)
    Mean_sample_1 = np.mean(x)
    absolute_deviations_from_median_sample_1 = abs(x-Median_sample_1)
    mean_absolute_deviation_sample_1 = np.mean(absolute_deviations_from_median_sample_1)
    normal_corrected_mean_absolout_deviation_sample_1 = mean_absolute_deviation_sample_1 * 1.2533
    median_absolute_deviation_sample_1 = np.median(absolute_deviations_from_median_sample_1)
    normal_corrected_median_absoloute_deviation_sample_1 = 1.4826*median_absolute_deviation_sample_1
    Inter_Quartile_Range_sample_1 = iqr(x)
    Standard_Deviation_sample_1 = np.std(x, ddof = 1)
    sample_size_sample_1 = len(x)
    Standrd_Error_Median_sample_1 = np.sqrt(math.pi/2) * (Standard_Deviation_sample_1/np.sqrt(sample_size_sample_1)) # Note - there are many other ways to estimate the median - for simplicity, we chose this one
    Standrd_Error_Median_AD_sample_1 = np.sqrt(2/math.pi) * (Standard_Deviation_sample_1/np.sqrt(sample_size_sample_1)) # Gia & Hung, 2001
    Standrd_Error_IQR_sample_1 = Standard_Deviation_sample_1 / (2*np.sqrt(sample_size_sample_1)*0.31777)
    
    Median_sample_2 = np.median(y)
    Mean_sample_2 = np.mean(y)
    absolute_deviations_from_median_sample_2 = abs(y-Median_sample_2)
    mean_absolute_deviation_sample_2 = np.mean(absolute_deviations_from_median_sample_2)
    normal_corrected_mean_absolout_deviation_sample_2 = mean_absolute_deviation_sample_2 * 1.2533
    median_absolute_deviation_sample_2 = np.median(absolute_deviations_from_median_sample_2)
    normal_corrected_median_absoloute_deviation_sample_2 = 1.4826*median_absolute_deviation_sample_2
    Inter_Quartile_Range_sample_2 = iqr(y)
    Standard_Deviation_sample_2 = np.std(y, ddof = 1)
    sample_size_sample_2 = len(y)
    Standrd_Error_Median_sample_2 = np.sqrt(math.pi/2) * (Standard_Deviation_sample_2/np.sqrt(sample_size_sample_2)) # Note - there are many other ways to estimate the median - for simplicity, we chose this one
    Standrd_Error_Median_AD_sample_2 = np.sqrt(2/math.pi) * (Standard_Deviation_sample_2/np.sqrt(sample_size_sample_2)) # Gia & Hung, 2001
    Standrd_Error_IQR_sample_2 = Standard_Deviation_sample_2 / (2*np.sqrt(sample_size_sample_2)*0.31777)

    # Effect Sizes for the difference betweem Medians
    Median_Difference = Median_sample_1 - Median_sample_2

    # 1. Based on SD's (Thompson, 2007)
    d_mdns = Median_Difference / np.sqrt((Standard_Deviation_sample_1**2 + Standard_Deviation_sample_2**2) / 2)

    # 2. Based on Pooled MAD Ricca & Blaine
    Median_Absolute_Deviations_Pooled = ((sample_size_sample_1-1)*median_absolute_deviation_sample_1 + (sample_size_sample_2-1)*median_absolute_deviation_sample_2) / (sample_size_sample_1 + sample_size_sample_2 - 2)
    Median_Absolute_Deviations_Pooled_Corrected = ((sample_size_sample_1-1)*normal_corrected_median_absoloute_deviation_sample_1 + (sample_size_sample_2-1)*normal_corrected_median_absoloute_deviation_sample_2) / (sample_size_sample_1 + sample_size_sample_2 - 2)
    d_mad_pooled = Median_Difference / Median_Absolute_Deviations_Pooled
    d_mad_pooled_corrected = Median_Difference / Median_Absolute_Deviations_Pooled

    
    # Ratio of the medians
    Ratio_of_the_Medians = Median_sample_1 / Median_sample_2

    # Quantile Shift effect size group2 is the reference group
    Quantile_1_Group1 = np.mean(Median_sample_1<=x)
    Quantile_2_Group1 = np.mean(Median_sample_2<=x)
    Quantile_Measure_Effect_Size_Median_Group1 = Quantile_1_Group1 - Quantile_2_Group1
    Quantile_1_Group2 = np.mean(Median_sample_1<=y)
    Quantile_2_Group2 = np.mean(Median_sample_2<=y)
    Quantile_Measure_Effect_Size_Median_Group2 = Quantile_2_Group2 - Quantile_1_Group2

    trimmed_mean_1 = stats.trim_mean(x, 0.2)
    trimmed_mean_2 = stats.trim_mean(y, 0.2)
    Quantile_1_trimmed_mean = np.mean(trimmed_mean_1<=x)
    Quantile_2_trimmed_mean = np.mean(trimmed_mean_2<=x)
    Quantile_Measure_Effect_Size_Trimmed_Mean = Quantile_1_trimmed_mean - Quantile_2_trimmed_mean

    # Quantile Shift of typical differences **Check why did Rand limits the function to n>10
    Pairwise_Comparisons = np.subtract.outer(x, y)
    Median_of_Comparisons = np.median(Pairwise_Comparisons)
    Trimmed_Mean_of_Comparisons = np.median(Pairwise_Comparisons)
    Quantile_Symmetric_Measure_Effect_Size = np.mean(Pairwise_Comparisons - Median_of_Comparisons <= Median_of_Comparisons)
    Quantile_Symmetric_Measure_Effect_Size_trimmed_mean = np.mean(Pairwise_Comparisons - Trimmed_Mean_of_Comparisons <= Trimmed_Mean_of_Comparisons)

    

    # Significant Tests for 2 medians
    Chi_Square_Statistic,p_value = median_test(x, y)[0:2]

    # Hodges Lehmann Estimator - I need to take the relvant parameters
    averages_vector_indepednent = [np.abs(xi - yi) for xi in x for yi in y]
    Hodges_Lehmann_estimator_independent_samples = np.median(averages_vector_indepednent)    
    CI_HL_between = Hodges_Lehmann_based_on_wilcox_test(x,y,confidence_level) 

    # Price and Bonett CI'sfor ratio of median and difference between medians 
    ci_between_price_bonett = CI_price_bonett(x,y,confidence_level)
 


    results = {}

    results["Median_Sample_1"] = Median_sample_1
    results["Mean_Sample_1"] = Mean_sample_1
    results["Mean_Absolute_Deviation_Sample_1"] = mean_absolute_deviation_sample_1
    results["Normal_Corrected_Mean_Absolute_Deviation_Sample_1"] = normal_corrected_mean_absolout_deviation_sample_1
    results["Median_Absolute_Deviation_Sample_1"] = median_absolute_deviation_sample_1
    results["Normal_Corrected_Median_Absolute_Deviation_Sample_1"] = normal_corrected_median_absoloute_deviation_sample_1
    results["Inter_Quartile_Range_Sample_1"] = Inter_Quartile_Range_sample_1
    results["Standard_Deviation_Sample_1"] = Standard_Deviation_sample_1
    results["Sample_Size_Sample_1"] = sample_size_sample_1
    results["Standrd_Error_Median_Sample_1"] = Standrd_Error_Median_sample_1
    results["Standrd_Error_Median_AD_Sample_1"] = Standrd_Error_Median_AD_sample_1
    results["Standrd_Error_IQR_Sample_1"] = Standrd_Error_IQR_sample_1

    results["Median_Sample_2"] = Median_sample_2
    results["Mean_Sample_2"] = Mean_sample_2
    results["Mean_Absolute_Deviation_Sample_2"] = mean_absolute_deviation_sample_2
    results["Normal_Corrected_Mean_Absolute_Deviation_Sample_2"] = normal_corrected_mean_absolout_deviation_sample_2
    results["Median_Absolute_Deviation_Sample_2"] = median_absolute_deviation_sample_2
    results["Normal_Corrected_Median_Absolute_Deviation_Sample_2"] = normal_corrected_median_absoloute_deviation_sample_2
    results["Inter_Quartile_Range_Sample_2"] = Inter_Quartile_Range_sample_2
    results["Standard_Deviation_Sample_2"] = Standard_Deviation_sample_2
    results["Sample_Size_Sample_2"] = sample_size_sample_2
    results["Standrd_Error_Median_Sample_2"] = Standrd_Error_Median_sample_2
    results["Standrd_Error_Median_AD_Sample_2"] = Standrd_Error_Median_AD_sample_2
    results["Standrd_Error_IQR_Sample_2"] = Standrd_Error_IQR_sample_2

    results["Median_Difference"] = Median_Difference
    results["Mean_Absolute_Deviations_Pooled"] = Mean_Absolute_Deviations_Pooled
    results["Effect_Size_Median"] = d_mdn
    results["Effect_Size_Median_AD_Pooled"] = d_mad_pooled
    results["Effect_Size_Median_AD_Pooled_Corrected"] = d_mad_pooled_corrected

    results["Quantile Measure (Group 1 as rference group) "] = Quantile_Measure_Effect_Size_Median_Group1
    results["Quantile Measure (Group 2 as reference group) "] = Quantile_Measure_Effect_Size_Median_Group2
    results["Quantile Measure Typical Difference"] = Quantile_Symmetric_Measure_Effect_Size

    results["Ratio_of_the_Medians"] = Ratio_of_the_Medians # Check out Bonnete and Price 2020 for confidence intervals  
    results["Moods Median Test Chi Square Statitic"] = Ratio_of_the_Medians   
    results["Moods Median Test p-value"] = Ratio_of_the_Medians 


    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

def two_dependent_medians(x,y, confidence_level = 0.95):
    Median_sample_1 = np.median(x)
    Mean_sample_1 = np.mean(x)
    absolute_deviations_from_median_sample_1 = abs(x-Median_sample_1)
    mean_absolute_deviation_sample_1 = np.mean(absolute_deviations_from_median_sample_1)
    normal_corrected_mean_absolout_deviation_sample_1 = mean_absolute_deviation_sample_1 * 1.2533
    median_absolute_deviation_sample_1 = np.median(absolute_deviations_from_median_sample_1)
    normal_corrected_median_absoloute_deviation_sample_1 = 1.4826*median_absolute_deviation_sample_1
    Inter_Quartile_Range_sample_1 = iqr(x)
    Standard_Deviation_sample_1 = np.std(x, ddof = 1)

    Median_sample_2 = np.median(y)
    Mean_sample_2 = np.mean(y)
    absolute_deviations_from_median_sample_2 = abs(y-Median_sample_2)
    mean_absolute_deviation_sample_2 = np.mean(absolute_deviations_from_median_sample_2)
    normal_corrected_mean_absolout_deviation_sample_2 = mean_absolute_deviation_sample_2 * 1.2533
    median_absolute_deviation_sample_2 = np.median(absolute_deviations_from_median_sample_2)
    normal_corrected_median_absoloute_deviation_sample_2 = 1.4826*median_absolute_deviation_sample_2
    Inter_Quartile_Range_sample_2 = iqr(y)
    Standard_Deviation_sample_2 = np.std(y, ddof = 1)

    # Median of the Difference and Related Measures
    Difference_Between_Samples = x-y
    Median_of_the_difference = np.median(Difference_Between_Samples)
    absolute_deviations_from_median_of_difference = abs(Difference_Between_Samples-Median_of_the_difference)
    mean_absolute_deviation_of_difference = np.mean(absolute_deviations_from_median_of_difference)
    normal_corrected_mean_absolout_deviation = mean_absolute_deviation_of_difference * 1.2533
    median_absolute_deviation_of_difference = np.median(absolute_deviations_from_median_of_difference)
    normal_corrected_median_absoloute_deviation = 1.4826*median_absolute_deviation_of_difference
    Inter_Quartile_Range_of_difference = iqr(Difference_Between_Samples)
    Standard_Deviation_of_difference = np.std(Difference_Between_Samples, ddof = 1)
    sample_size = len(Difference_Between_Samples)

    # Difference Between Medians and Confidence Interval (Price Bonett, 2021)
    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)
    sorted_x = np.sort(Difference_Between_Samples)
    c1 = np.floor((sample_size + 1) / 2 - sample_size**0.5)
    X1 = sorted_x[int(sample_size - c1)]
    X2 = sorted_x[int(c1 - 1)]
    Z1 = norm.ppf(1 - binom.cdf(c1 - 1, sample_size, 0.5))
    Standrad_Error_Difference_price_Bonett = np.sqrt(((X1 - X2) / (2 * Z1))**2) 
    values_lower_medians = np.sum (((x < Median_sample_1) + (y < Median_sample_2)) == 2)  


    Standrd_Error_Median_of_difference = np.sqrt(math.pi/2) * (Standard_Deviation_of_difference/np.sqrt(sample_size)) # Note - there are many other ways to estimate the median - for simplicity, we chose this one
    Standrd_Error_Median_AD = np.sqrt(2/math.pi) * (Standard_Deviation_of_difference/np.sqrt(sample_size)) # Gia & Hung, 2001
    Standrd_Error_IQR = Standard_Deviation_of_difference / (2*np.sqrt(sample_size)*0.31777)

    # Another Estimator for the dispertion around median Qn (Rousseeuw and Croux, 1993)
    def pairwise_differences(x):      # a function to calcualte all pairwise comparison
        return [b - a for a, b in itertools.combinations(x, 2)]
    Pairwise_Vector = pairwise_differences(x)
    Qn = 2.2219 * (np.quantile(abs(np.array(Pairwise_Vector)),0.25))

    # Effect Sizes 
    # 1. Thompson 2007
    d_Mdn = abs(Median_of_the_difference) / Standard_Deviation_of_difference

    # 2. Median Dispersion based measures
    d_Mdn_AD = abs(Median_of_the_difference) / median_absolute_deviation_of_difference
    d_Mdn_AD_corrected = (Median_of_the_difference)  / normal_corrected_median_absoloute_deviation
    d_IQR = (Median_of_the_difference)  / Inter_Quartile_Range_of_difference
    d_Qn = (Median_of_the_difference)  / Qn

    #3. Ratio of the means and confidence intervals
    Ratio_of_the_Medians = Median_sample_1 / Median_sample_2
    
    #4. Wilcoxon Quantile Shift measure of effect size
    Quantile_Shift_Effect_Size = np.mean(Difference_Between_Samples - Median_of_the_difference <= Median_of_the_difference)

    # Two depdendent Samples Median Significant Test
    t_median = (Median_of_the_difference) / Standrd_Error_Median_of_difference
    P_value_median = calculate_p_value_from_t_score(t_median, sample_size-1)
    
    # One sample Sign Test
    Sign_Test_vector = np.where(Difference_Between_Samples > 0, 1, np.where(Difference_Between_Samples < 0, 0, np.nan))
    K = sum(Sign_Test_vector)
    p_value_sign = binom.cdf(k=K, n=sample_size, p=0.5)

    # Wilcox Qs for dependent Samples 
    Wilcox_Qs = np.mean(Difference_Between_Samples - Median_of_the_difference <= Median_of_the_difference)

    # Hodges-Lehemann Estimator Dependent Samples
    averages_vector_paired = [(Difference_Between_Samples[i] + Difference_Between_Samples[j]) / 2
    for i in range(len(Difference_Between_Samples))
    for j in range(i, len(Difference_Between_Samples))]
    Hodges_Lehmann_estimator_Paired_samples = np.median(averages_vector_paired)
    HL_CI = Hodges_Lehmann_based_on_wilcox_test(Difference_Between_Samples, y = None, confidence_level=confidence_level)

    # ****Here I need to choose the relevant ouput parameters from the function 
    CIs = CI_price_bonett(x,y,confidence_level)







    









     















